# This Notebook Will Create the DataFrame Used to Create the Model

## 1. Scrape the PDFs and Save to a Master DataFrame

In [1]:
from os import listdir
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import pandas as pd
import pickle

In [5]:
list_of_pdfs = listdir("Scraped_PDFs/")

In [12]:
length = len(list_of_pdfs)

In [26]:
# Code from Stack Overflow to use PDFminer to import PDF to text reworked into a function


def pdf2txt(path):
    """
    A function to convert a PDF to a string, 
    takes the file path to the PDF as the 
    argument and returns the PDF as a string.
    """
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(path, 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()
    
    return(text)

In [27]:
path = 'SampleData/SamplePDF.pdf'
text = pdf2txt(path)

In [28]:
text  #WORKS!!
type(text)

str

#### Run the above function, adding each PDF to the DataFrame

In [29]:
# Run the function, adding each one to a DF
df = pd.DataFrame(columns = ["Opinion"])

In [43]:
error_list=[]

def make_df(directory):
    
    global error_list
    
    for count, filename in enumerate(listdir(directory)):

        if filename.endswith(".pdf"):
            try: 
                df.loc[count] = pdf2txt(directory+'/'+filename)
            except:
                error_list.append(filename)
                
        else:
            continue
            

In [44]:
%%time
make_df("Scraped_PDFs")

CPU times: user 4h 16min 50s, sys: 55.1 s, total: 4h 17min 45s
Wall time: 4h 19min 30s


In [45]:
df

,Opinion
0,\n\nAn unpublished opinion of the North ...
2,An unpublished opinion of the North Caro...
3,An unpublished opinion of the North Caro...
4,NO. COA11-246 \n\nNORTH CAROLINA COURT OF APPE...
1,An unpublished opinion of the North Caro...
...,...
28842,"AMY TERASAKA, Employee v. AT&T, Employer, SELF..."
28843,An unpublished opinion of the North Caro...
28844,An unpublished opinion of the North Caro...
28845,NO. COA01-932\n\nNORTH CAROLINA COURT OF APPEA...


In [46]:
error_list

['08154.pdf',
 '02900.pdf',
 '07110.pdf',
 '03014.pdf',
 '05705.pdf',
 '03567.pdf',
 '08813.pdf',
 '02494.pdf',
 '38907.pdf',
 '29822.pdf',
 '02641.pdf',
 '03577.pdf',
 '35166.pdf',
 '34287.pdf',
 '03402.pdf',
 '34286.pdf',
 '02913.pdf',
 '03204.pdf',
 '05477.pdf',
 '34285.pdf',
 '03398.pdf',
 '06481.pdf',
 '03499.pdf',
 '36191.pdf',
 '03103.pdf',
 '02631.pdf',
 '06682.pdf',
 '05389.pdf',
 '03288.pdf',
 '08040.pdf',
 '03666.pdf',
 '03528.pdf',
 '32481.pdf',
 '03678.pdf',
 '23755.pdf',
 '27453.pdf',
 '03083.pdf',
 '03040.pdf',
 '03732.pdf',
 '03492.pdf',
 '05418.pdf',
 '39483.pdf',
 '07223.pdf',
 '03496.pdf',
 '02950.pdf',
 '03469.pdf',
 '03332.pdf',
 '03640.pdf',
 '08073.pdf',
 '03494.pdf',
 '06831.pdf',
 '03154.pdf',
 '03419.pdf',
 '03545.pdf',
 '39381.pdf',
 '03745.pdf',
 '03209.pdf',
 '03180.pdf',
 '31286.pdf',
 '02894.pdf',
 '02853.pdf',
 '08774.pdf',
 '03345.pdf',
 '08158.pdf',
 '03233.pdf',
 '03623.pdf',
 '03637.pdf',
 '03636.pdf',
 '03193.pdf',
 '03385.pdf',
 '31525.pdf',
 '0375

In [51]:
# Save the new DF to a data file
with open('master_df.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(df, filehandle)

In [52]:
# Save the error file to a data file
with open('error_list.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(error_list, filehandle)

## 2. Check the Error List for Any Potentially Relevant Opinions
If any of the PDFs that came up on the error list contain the phrase "summary judgment," they may be important to the project. I manually reviewed the PDFs on the error list and discovered eleven potentially relevant PDFs (Nos. 2631, 2913, 3154, 3385, 3419, 3545, 3623, 3637, 3678, 32481, and 39483).

In [2]:
# Open the error_list
infile = open('error_list.data','rb')
error_list = pickle.load(infile)
infile.close()

In [51]:
# Open the dataframe
infile2 = open('master_df.data', 'rb')
df = pickle.load(infile2)
infile2.close()

In [53]:
df.describe(), df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28772 entries, 0 to 28846
Data columns (total 1 columns):
Opinion    28772 non-null object
dtypes: object(1)
memory usage: 449.6+ KB


(                                                  Opinion
 count                                               28772
 unique                                              28663
 top     NO. COA05-1369\nNO. COA05-1370\nNO. COA05-1371...
 freq                                                    4, None)

In [54]:
# Having reviewed the error_list PDFs, the following will create a Series from 
# those containing the significant language to append to the master dataframe
err1 = """STATE OF NORTH CAROLINA v. MARIO DEANDRE TAYLOR, Defendant\n NO. COA07-391\n Filed: 5 August 2008\n Kidnapping–during robbery–insufficient evidence of separate offense\n The evidence was not sufficient to support convictions for second-degree kidnapping\n where defendant and others entered a McDonald’s, made the patrons and workers lie down, and\n took the manager to the back to open the safe. The evidence establishes only the elements of\n robbery with the one added component of the victims being required to lie down, which was a\n mere technical asportation.\n Appeal by defendant from judgments entered 14 July 1999 by\n Judge Donald Jacobs in Durham County Superior Court. Heard in the\n Court of Appeals 30 October 2007.\n Attorney General Roy Cooper, by Assistant Attorney General\n Charlie E. Reece, for the State.\n Jarvis John Edgerton, IV for defendant-appellant.\n GEER, Judge.\n Defendant Mario Deandre Taylor appeals from his convictions of\n one count of robbery with a dangerous weapon and 10 counts of\n second degree kidnapping. Defendant primarily challenges the trial\n court's denial of his motion to dismiss his second degree\n kidnapping charges, arguing that the State failed to produce\n sufficient evidence of confinement, restraint, or removal beyond\n that which was inherent in the robbery with a dangerous weapon.\n Because we agree with defendant that the State failed to meet its\n burden, as required by State v. Fulcher, 294 N.C. 503, 243 S.E.2d\n 338 (1978), and its progeny, of establishing an act of confinement,\n -2-\n restraint, or removal separate and apart from the robbery, we\n vacate defendant's second degree kidnapping convictions.\n Facts\n The State's evidence tended to establish the following facts.\n On the evening of 14 February 1998, defendant and another man\n entered a McDonald's restaurant wearing masks. Defendant, who held\n a 9mm pistol, ordered everyone to lie face down on the floor. When\n a cashier remained standing, defendant pointed his gun at her and\n again ordered her to lay down on the floor. Defendant and the\n other man took the restaurant manager to the back of the restaurant\n where the safe was located and ordered her to open it. A third\n man, who had subsequently entered the restaurant, remained in the\n lobby watching the customers and employees. The cashier, however,\n ran out of the restaurant.\n After defendant and the second man finished collecting the\n cash from the safe, the three men ran out of the restaurant through\n a side door. The cashier saw the men leave the restaurant and\n identified defendant at trial as one of the perpetrators because he\n had been wearing the same clothes a few days earlier when he came\n into the restaurant to fill out an application.\n Defendant was charged with one count of robbery with a\n dangerous weapon and 13 counts of second degree kidnapping. During\n the trial, the court dismissed two of the second degree kidnapping\n charges. The court dismissed the count relating to the manager\n because her asportation to the back of the restaurant to open the\n safe was "part and parcel" of the robbery. With respect to the\n -3-\n second count, the court found that no evidence had been presented\n at all as to that alleged victim.\n The jury convicted defendant of robbery with a dangerous\n weapon and 10 counts of second degree kidnapping; it acquitted him\n of one count of second degree kidnapping. At sentencing, the trial\n court made findings regarding aggravating and mitigating factors.\n As an aggravating factor, the court found that defendant had joined\n with more than one other person in committing the robbery with a\n dangerous weapon and the kidnapping, but had not been indicted for\n conspiracy. In mitigation, the court found that defendant had a\n support system in the community and that he had voluntarily\n cooperated with the police. The court concluded that the\n aggravating factors outweighed the mitigating factors and,\n therefore, imposed aggravated sentences of (1) 120 to 153 months\n imprisonment for the robbery with a dangerous weapon conviction,\n (2) 92 to 110 months for one of his kidnapping convictions (running\n consecutively), (3) 92 to 110 months for a second kidnapping\n conviction (running consecutively), and (4) 92 to 110 months for\n the remaining eight kidnapping convictions (running concurrently\n with the second kidnapping sentence). Each of defendant's\n kidnapping sentences included a 60-month firearm enhancement\n pursuant to N.C. Gen. Stat. § 15A-1340.16(A) (2007).\n Defendant appealed in open court on 14 July 1999. While\n defendant was granted appellate counsel, his appeal did not\n progress for six years. We granted his petition for writ of\n certiorari on 21 February 2006.\n -4-\n Discussion\n Defendant's primary argument on appeal is that the trial court\n erred in denying his motion to dismiss the second degree kidnapping\n charges. Defendant maintains that the State presented insufficient\n evidence of confinement, restraint, or removal separate from that\n which was inherent in the robbery with a dangerous weapon and,\n therefore, he cannot be convicted of both offenses under State v.\n Fulcher, 294 N.C. 503, 523, 243 S.E.2d 338, 351 (1978).\n In Fulcher, our Supreme Court stated:\n It is self-evident that certain felonies\n (e.g., forcible rape and armed robbery) cannot\n be committed without some restraint of the\n victim. We are of the opinion, and so hold,\n that G.S. 14-39 was not intended by the\n Legislature to make a restraint, which is an\n inherent, inevitable feature of such other\n felony, also kidnapping so as to permit the\n conviction and punishment of the defendant for\n both crimes. To hold otherwise would violate\n the constitutional prohibition against double\n jeopardy. Pursuant to the above mentioned\n principle of statutory construction, we\n construe the word "restrain," as used in G.S.\n 14-39, to connote a restraint separate and\n apart from that which is inherent in the\n commission of the other felony.\n Id.\n The Supreme Court further clarified the "separate act"\n requirement in State v. Irwin, 304 N.C. 93, 103, 282 S.E.2d 439,\n 446 (1981), holding that removal of an employee at knifepoint from\n the front to the rear of a pharmacy to open the safe and obtain\n drugs was "an inherent and integral part of the attempted armed\n robbery," and, therefore, the removal was legally insufficient to\n -5-\n convict the defendant of a separate charge of kidnapping. The\n Court also noted that the defendant did not expose the victim "to\n greater danger than that inherent in the armed robbery itself, nor\n [was the victim] subjected to the kind of danger and abuse the\n kidnapping statute was designed to prevent." Id. As a result, the\n Court concluded that the defendant's removal of the victim was "a\n mere technical asportation" requiring dismissal of the kidnapping\n charge. Id.\n The Court more recently addressed this issue in State v.\n Ripley, 360 N.C. 333, 626 S.E.2d 289 (2006). In Ripley, the Court\n held:\n [A] trial court, in determining whether a\n defendant's asportation of a victim during the\n commission of a separate felony offense\n constitutes kidnapping, must consider whether\n the asportation was an inherent part of the\n separate felony offense, that is, whether the\n movement was "a mere technical asportation."\n If the asportation is a separate act\n independent of the originally committed\n criminal act, a trial court must consider\n additional factors such as whether the\n asportation facilitated the defendant's\n ability to commit a felony offense, or whether\n the asportation exposed the victim to a\n greater degree of danger than that which is\n inherent in the concurrently committed felony\n offense.\n Id. at 340, 626 S.E.2d at 293-94.\n Viewed in the light most favorable to the State, the evidence\n in this case indicates that defendant entered the McDonald's\n carrying a handgun, which he pointed at the customers and employees\n as he ordered them to lie face down on the floor. Defendant and\n another man found the manager and took her to the back of the\n -6-\n restaurant to open the safe while a third man stood guard over the\n people on the floor.\n The State contends that the robbery of the McDonald's occurred\n at the safe located in the back office of the restaurant, and,\n therefore, the restraint of the customers and employees in the\n lobby was unnecessary to the commission of the robbery. We,\n however, consider the present case to be controlled by State v.\n Beatty, 347 N.C. 555, 495 S.E.2d 367 (1998), in which the Supreme\n Court held that kidnapping charges, based on similar circumstances,\n should have been dismissed.\n In Beatty, a group of men approached the owner of a restaurant\n outside an open door to the restaurant, put a gun to his head, and\n told him to go inside and open the safe. Id. at 557, 495 S.E.2d at\n 368. Once inside, the robbers saw two restaurant employees. One\n employee, Poulos, "was on his knees washing the floor at the\n front," while the second, Koufaloitis, "stood three to four feet\n from the safe cleaning the floor in the back." Id., 495 S.E.2d at\n 368-69. At that point, "[o]ne robber put a gun to Poulos' head and\n stood beside him during the robbery. An unarmed robber put duct\n tape around Koufaloitis' wrists and told him to lie on the floor."\n Id., 495 S.E.2d at 369.\n The Supreme Court upheld the kidnapping conviction with\n respect to Koufaloitis, but not as to Poulos. Id. at 560, 495\n S.E.2d at 370. The Court explained that "[w]hen defendant bound\n [Koufaloitis'] wrists and kicked him in the back, he increased the\n victim's helplessness and vulnerability beyond what was necessary\n -7-\n to enable him and his comrades to rob the restaurant." Id. at 559,\n 495 S.E.2d at 370. On the other hand, "[w]ith regard to victim\n Poulos, the evidence shows only that one of the robbers approached\n the victim, pointed a gun at him, and stood guarding him during the\n robbery. The victim did not move during the robbery, and the\n robbers did not injure him in any way." Id. at 560, 495 S.E.2d at\n 370. The Court explained further: "The only evidence of restraint\n of this victim was the threatened use of a firearm. This restraint\n is an essential element of robbery with a dangerous weapon under\n N.C.G.S. § 14-87, and defendant's use of this restraint exposed the\n victim to no greater danger than that required to complete the\n robbery with a dangerous weapon." Id.\n In this case, as in Beatty, the robbery took place at a safe\n in the back of a restaurant, while the victims were restrained in\n the front by another robber guarding them with a gun, without any\n of the victims being bound or injured in any way. Because the\n restaurant's occupants were not bound, once the robbery was\n complete and the perpetrators had run out of the restaurant, the\n occupants were not further restrained. Compare State v. Morgan,\n 183 N.C. App. 160, 167, 645 S.E.2d 93, 99 (2007) (upholding\n kidnapping conviction when "[t]he evidence shows that the three\n robbers bound the victims with duct tape, took money and cellular\n telephones, and left the victims bound when they left the hotel\n room"), appeal dismissed and disc. review denied, 362 N.C. 241, 660\n S.E.2d 536 (2008).\n -8-\n The sole distinction between this case and Beatty is that the\n victims were required to lie down on the floor. In Ripley, 360\n N.C. at 340, 626 S.E.2d at 294 (quoting Irwin, 304 N.C. at 103, 282\n S.E.2d at 446), however, our Supreme Court concluded:\n [T]he asportation of the [victims] from one\n side of the motel lobby door to the other was\n not legally sufficient to justify defendant's\n convictions of second-degree kidnapping. The\n moment defendant's accomplice drew his\n firearm, the robbery with a dangerous weapon\n had begun. The subsequent asportation of the\n victims was "a mere technical asportation"\n that was an inherent part of the robbery\n defendant and his accomplices were engaged in.\n We do not believe that defendant's order, at gunpoint, that the\n victims lie down on the floor is materially different than the\n Ripley robbers' order, also at gunpoint, that the victims move from\n outside the door to the lobby to inside the door. Accordingly,\n under Ripley, we hold that the act of requiring the victims to lie\n down is a mere technical asportation insufficient to sustain a\n charge of kidnapping separate from the robbery.\n This conclusion is supported by this Court's decision in State\n v. Ross, 133 N.C. App. 310, 515 S.E.2d 252 (1999). In Ross, the\n record indicated "that, upon entering the apartment, [a robber]\n pointed the shotgun at [the two victims] and ordered them to step\n away from the apartment door and get on the floor." Id. at 313,\n 515 S.E.2d at 254. Although one of the victims backed from the\n living room into the kitchen before lying down, the Court held that\n the evidence was insufficient to establish a removal separate from\n the robbery when the robbers did not order the victim to move to\n the kitchen, but rather only ordered him to "back up and get on the\n -9-\n floor." Id. The evidence of restraint or removal was no greater\n in this case.\n The State, however, points to State v. Brice, 126 N.C. App.\n 788, 486 S.E.2d 719 (1997), a decision rendered a year before\n Beatty. In Brice, one defendant was in the bedroom robbing two\n male victims, while a second robber was outside the house demanding\n money from another male victim. A third robber was in the living\n room with the female victim. The third robber threatened the woman\n with a gun and ordered her to lie face down on the floor, causing\n her to become ill. Id. at 790, 486 S.E.2d at 720. This Court\n explained in Ross that "[i]n Brice, our Court held that terrorizing\n the woman in the living room was not an inherent part of the\n robbery taking place in the bedroom." Ross, 133 N.C. App. at 314,\n 515 S.E.2d at 255. As the Court acknowledged in Brice, this\n terrorization was not necessary to carry out the robbery of either\n the victims in the bedroom or the victim outside the house. Brice,\n 126 N.C. App. at 791, 486 S.E.2d at 720. We believe, however, that\n this case more closely resembles Beatty and Ross.\n State v. Davidson, 77 N.C. App. 540, 335 S.E.2d 518 (1985),\n disc. review denied, 315 N.C. 393, 338 S.E.2d 882 (1986), also\n relied upon by the State, is likewise inapposite. In Davidson, the\n defendants entered a retail store, and, at gunpoint, took the\n store's occupants from the front of the store to a dressing room in\n the rear of the store; bound their heads, arms, and legs; took\n their valuables; and then took cash and merchandise from the store.\n Id. at 541, 335 S.E.2d at 519. In upholding the kidnapping\n -10-\n convictions, this Court concluded: "Removal of the victims to the\n dressing room [where none of the stolen property was kept] thus was\n not an inherent and integral part of the robbery." Id. at 543, 335\n S.E.2d at 520. The "removal" was the critical factor.\n The State, however, points to this Court's statement in\n Davidson that the removal "was a separate course of conduct\n designed to remove the victims from the view of passersby who might\n have hindered the commission of the crime." Id. The State\n contends that the conduct in this case necessarily must have been\n for the same purpose. In making this argument, the State overlooks\n the fact that there must still have been "a separate course of\n conduct." Id.\n In this case, in contrast to Davidson, no removal occurred.\n The only conduct presented by the State as being apart from the\n robbery was the guarding of victims with a gun while face down on\n the floor. While the removal of the victims was not necessary to\n the robbery in Davidson, both the use of the firearm and the\n presence of the individual victims were necessary to the robbery\n with a dangerous weapon conviction. Under N.C. Gen. Stat. § 14-\n 87(a) (2007), a person is guilty of robbery with a dangerous weapon\n if that person, "having in possession or with the use or threatened\n use of any firearms or other dangerous weapon, implement or means,\n whereby the life of a person is endangered or threatened,\n unlawfully takes or attempts to take personal property from another\n or from any place of business, residence or banking institution or\n any other place where there is a person or persons in attendance .\n -11-\n . . ." (Emphasis added.) Consistent with the statute, the State's\n indictments in this case alleged the threatened use of firearms and\n the taking of McDonald's property while the alleged kidnapping\n victims were present.\n In sum, the State's evidence of kidnapping established only\n the elements of the crime of robbery with a dangerous weapon with\n the lone added component of the victims' being required to lie down\n on the floor. Under Ripley, that lone act is a mere technical\n asportation. As a result, unlike Davidson, the State presented no\n additional evidence of restraint, confinement, or removal beyond\n that necessary to commit the robbery.\n We, therefore, hold that the evidence in the record is\n insufficient to support defendant's convictions for second degree\n kidnapping under Fulcher, and the trial court should have granted\n defendant's motion to dismiss those charges. Because we are\n vacating defendant's second degree kidnapping convictions, we do\n not address defendant's additional arguments relating to those\n convictions. Defendant does not make any arguments on appeal\n regarding his robbery with a dangerous weapon conviction and thus,\n as to that conviction, we find no error.\n Vacated in part; no error in part.\n Judges WYNN and STEELMAN concur."""
err2 = """Pinpoint added 191 #3 p. 411\n CHARLES W. STONE; MARIE STONE; MONA M. KEECH; MARK DEARMON; MASON\n P. THOMAS, JR.; MARGARET KAY HOVIOUS; JEANNETTE M. DEAN; WILLIAM\n R. FOSTER; R. ROSS HAILEY, JR.; THOMAS F. EAMON; FLINT BENSON;\n DONNIE G. PERRY; W.R. McCLURE; and MARY SINGLETON; on behalf of\n themselves and all others similarly situated, Plaintiffs, v. THE\n STATE OF NORTH CAROLINA; MICHAEL F. EASLEY, in his capacity as\n Governor of North Carolina; ROBERT POWELL, in his capacity as\n State Controller of North Carolina; DAVID T. McCOY, in his\n capacity as State Budget Officer of North Carolina; RICHARD H.\n MOORE, in his capacity as Treasurer of North Carolina; and the\n BOARD OF TRUSTEES of the TEACHERS' AND STATE EMPLOYEES'\n RETIREMENT SYSTEM OF NORTH CAROLINA, a body politic and\n corporate; Defendants\n NO. COA07-718\n Filed: 5 August 2008\n 1. Governor; Pensions and Retirement–executive order–state employees’ retirement\n system–employer contributions escrowed–unconstitutionality\n An executive order signed by the governor directing that state employers send the\n employer portion of retirement contributions for the state employees’ retirement system to the\n State Controller for placement into an escrow account for the purpose of ensuring a balanced\n state budget “diverted” the funds in violation of N.C. Const. art. V, § 6(2) even though the\n employer contributions had not yet been received by the retirement system.\n 2. Pensions and Retirement–state employees’ retirement system–actuarially sound\n funding–contractual right\n Vested state employees have a contractual right to have their retirement systems funded\n in an actuarially sound manner.\n 3. Pensions and Retirement–state employees’ retirement system–escrow of employer\n contributions–impairment of contract\n The diversion of employer contributions from the state employees’ retirement system into\n an escrow account pursuant to an executive order signed by the governor impaired the\n contractual rights of vested members to a retirement system funded in an actuarially sound\n manner because, at the time the employer contributions were escrowed, it was unclear when, or\n even whether, the diverted funds would be repaid, and the integrity and security of the retirement\n system were threatened.\n 4. Pensions and Retirment–state employees’ retirement system–escrow of employer\n contribution–not reasonable and necessary\n The escrow of the employer contribution to the state employees’ retirement system was\n not reasonable and necessary to serve the important public purpose of avoiding a constitutionally\n prohibited budget deficit and violated the contract clause of the U.S. Constitution. A balanced\n -2-\n budget could have been achieved in another way. U.S. Const. art. I, § 10.\n 5. Penstions and Retirement--state employees’ retirement system–employer\n contribution escrowed–no penalty\n The trial court did not err by granting summary judgment for the State Treasurer and the\n board of trustees of the state employees’ retirement system on a claim for a writ of mandamus to\n compel compliance with N.C.G.S. § 135-8(f)(3), which imposes a penalty when contributions to\n the state employees’ retirement system are not received. The statute speaks in terms of default\n by an employer, but in the present case the employer contributions were escrowed as the result\n of an executive order. Moreover, the Treasurer and board of trustees had routinely waived the\n imposition of the fine when it was determined that there was no intent to not remit the\n contributions in a timely manner.\n Appeal by Plaintiffs and Defendants from order entered 27\n February 2007, incorporating by reference an order entered 6\n September 2006, said orders entered by Judge Joseph R. John, Sr. in\n Superior Court, Wake County. Heard in the Court of Appeals 15\n January 2008.\n Blanchard, Miller, Lewis & Styers, P.A., by E. Hardy Lewis and\n Karen M. Kemerait, for Plaintiffs.\n Attorney General Roy Cooper, by Special Deputy Attorney\n General Alexander McC. Peters, Special Deputy Attorney General\n Joyce Rutledge, and Assistant Attorney General Robert M.\n Curran, for Defendants.\n Thomas A. Harris for the State Employees Association of North\n Carolina, Inc., amicus curiae.\n McGEE, Judge.\n The Governor of the State of North Carolina, Michael F. Easley\n (the Governor), signed Executive Order No. 3 on 8 February 2001.\n Executive Order No. 3 provided, in pertinent part:\n [B]y the authority vested in me as Governor by\n Article III, Sec. 5(3) of the North Carolina\n Constitution to insure that a deficit is not\n incurred in the administration of the budget\n for fiscal year 2001, IT IS ORDERED:\n -3-\n . . . .\n The Office of the State Controller, as advised\n by the State Budget Officer, is directed to\n receive the employer portion of retirement\n contributions for all State funded retirement\n systems and to escrow such funds in a special\n reserve as established by [the Office of State\n Budget, Planning, and Management ("OSBPM")].\n Before taking such action, OSBPM is directed\n to confirm with the State Treasurer that such\n action will not impair the actuarial integrity\n of the state retirement system. Return of all\n such receipts shall be made to the retirement\n system, if possible, after determination that\n such funds are not necessary to address the\n deficit.\n In compliance with Executive Order No. 3, Edward Renfrow, the State\n Controller at the time, issued a memorandum on 15 February 2001 to\n all chief fiscal officers, vice chancellors, business managers, and\n local education authorities affiliated with employers participating\n in State-funded retirement systems. Specifically, he directed that\n all such employers send the funds allocated as employer\n contributions for the Teachers' and State Employees' Retirement\n System of North Carolina (the Retirement System) to an escrow\n account (the escrow account) in the Office of the State Controller.\n Between February and June 2001, State employers sent\n $208,362,861.00 of Retirement System employer contributions to the\n escrow account. The Governor extended the terms of Executive Order\n No. 3 to include employer contributions for July and August 2001,\n and State employers sent an additional $16,511,854.00 of Retirement\n System employer contributions to the escrow account during that\n period of time. The amount of $16,511,854.00 was returned on 30\n November 2001 to the Retirement System, and the amount of\n -4-\n $82,612,901.00 was returned on 7 December 2001 to the Retirement\n System and to two other retirement systems from which funds had\n been seized. The two other retirement systems were the Legislative\n Retirement System and the Judicial Retirement System. As of 31\n December 2001, a total of $129,924,859.00 of Retirement System\n employer contributions that had been sent to the escrow account had\n not been repaid to the Retirement System.\n Plaintiffs initiated this action on 14 June 2002 by filing a\n complaint for declaratory judgment and a petition for writ of\n mandamus. Plaintiffs alleged that the State of North Carolina; the\n Governor; Robert Powell, in his capacity as State Controller of\n North Carolina; and David T. McCoy, in his capacity as State Budget\n Officer of North Carolina, violated the Contract Clause of the\n United States Constitution; Article V, section 6(2) of the North\n Carolina Constitution; and Article I, sections 1 and 19 of the\n North Carolina Constitution. Plaintiffs also sought a writ of\n mandamus to require the Governor, Robert Powell, and David T. McCoy\n to permanently desist from the seizure and diversion of employer\n contributions, and to return to the Retirement System all funds\n that were appropriated, paid, seized, and diverted. Plaintiffs\n further sought a writ of mandamus to compel Richard H. Moore, in\n his capacity as Treasurer of North Carolina (the Treasurer), and\n the Board of Trustees (the Board) of the Retirement System to\n comply with N.C. Gen. Stat. § 135-8(f)(3).\n Defendants filed a motion to dismiss Plaintiffs' complaint on\n 19 August 2002. Pursuant to Rule 2.1(a) of the General Rules of\n -5-\n Practice, the Chief Justice of the North Carolina Supreme Court\n designated the case as exceptional and assigned the case to\n Emergency Judge Joseph R. John, Sr. Plaintiffs filed a first\n amended complaint for declaratory judgment, petition for writ of\n mandamus, and motion for class certification dated 12 February\n 2004.\n The trial court entered an order certifying a class on 27\n February 2004, and defined the class as follows: "[A]ll North\n Carolina teachers and State employees who were members of the\n Teachers' and State Employees' Retirement System of North Carolina,\n as provided in [N.C. Gen. Stat. §§] 135-3 and 135-4, at any time\n during the period 7 February 2001 to 7 August 2001, inclusive."\n The trial court also entered an order denying Defendants' motion to\n dismiss on 27 February 2004. Defendants filed an answer, dated 27\n April 2004, to the first amended complaint.\n Following substantial discovery, Plaintiffs and Defendants\n filed cross-motions for summary judgment on 20 February 2006.\n Plaintiffs subsequently filed a motion to amend their first amended\n complaint, seeking to add a paragraph and amend an existing\n paragraph to add a claim under Article I, section 6 of the North\n Carolina Constitution. In their motion, Plaintiffs stated that\n Defendants did not object to Plaintiffs' motion to amend. The\n trial court entered an order on 26 July 2006 amending both\n Plaintiffs' first amended complaint and Defendants' answer to\n Plaintiffs' first amended complaint.\n The trial court entered an order on the cross-motions for\n -6-\n summary judgment on 6 September 2006, granting summary judgment for\n Plaintiffs on their "claim for declaratory judgment establishing\n that the actions of [D]efendants State of North Carolina, the\n Governor of North Carolina, the State Controller of North Carolina,\n and the State Budget Officer of North Carolina violate[d] Article\n V, Section 6 of the North Carolina Constitution[.]" Regarding\n Plaintiffs' claim for declaratory judgment that the actions of\n these Defendants violated the Contract Clause of the United States\n Constitution, the trial court ordered the following:\n [T]he Court will reserve ruling to allow the\n parties to determine whether sufficient\n stipulated facts can be provided to the Court\n to allow for ruling without a trial. Counsel\n are directed to report to the Court, within 15\n days of the entry of this order, as to whether\n an additional hearing on this issue is\n necessary, and whether at such hearing the\n parties will put on evidence or submit\n stipulated facts.\n The trial court also entered summary judgment for Defendants on\n Plaintiffs' remaining claims for declaratory judgment and on\n Plaintiffs' two remaining claims for writ of mandamus, one of which\n is the subject of Plaintiffs' appeal.\n The parties filed a joint statement of stipulations regarding\n Plaintiffs' Contract Clause claim on 18 October 2006, and the trial\n court entered a final order on substantive claims on 27 February\n 2007. The trial court ordered: "The 6 September 2006 order is\n incorporated by reference in its entirety, such that the present\n order shall serve as the final judgment of the trial court in this\n case with regard to the substantive claims raised in the\n complaint[.]" The trial court also stated:\n -7-\n 3. The Plaintiff Class has met its burden of\n proving the existence of a contract. The\n Plaintiff Class likewise has met its burden of\n proving that the State's actions impaired the\n contract.\n 4. With regard to the third prong,\n [D]efendants have met their burden of proving\n that the Governor's maintaining compliance\n with his constitutional responsibility to\n ensure a balanced budget constitutes an\n important public purpose. However,\n [D]efendants have failed to carry their burden\n of proving that the diversion of employer\n contributions was "reasonable" or "necessary"\n in service of that purpose. In so ruling, the\n Court observes that the provisions for the\n 'inviolability' of retirement system funds\n contained in the Constitution of North\n Carolina indicate a public policy that would\n favor protection of these funds under the\n circumstances of the present case.\n The trial court granted summary judgment for Plaintiffs on their\n Contract Clause claim. The trial court also certified the matter\n for immediate appeal:\n Further, the Court finds that the instant case\n involves multiple parties and multiple claims,\n that the Court has entered final judgment as\n to certain claims and certain parties, that\n the Court has entered final judgment as to all\n substantive claims raised in the complaint,\n and that there is no just reason for delay of\n the parties' respective appeals; therefore,\n pursuant to N.C.G.S. [§] 1A-1, Rule 54(b), the\n Court certifies this matter for immediate\n appeal.\n Plaintiffs appeal the grant of summary judgment to the\n Treasurer and to the Board on Plaintiffs' claim for writ of\n mandamus to compel compliance with N.C. Gen. Stat. § 135-8(f)(3).\n The grant of summary judgment to Plaintiffs on their claims for\n declaratory judgment under Article V, section 6(2) of the North\n Carolina Constitution and under the Contract Clause of the United\n -8-\n States Constitution is appealed by the State of North Carolina; the\n Governor; Robert Powell; and David T. McCoy. For the reasons set\n forth herein, we affirm the orders of the trial court.\n Standard of Review\n "It is well settled that de novo review is ordinarily\n appropriate in cases where constitutional rights are implicated."\n Piedmont Triad Reg'l Water Auth. v. Sumner Hills, Inc., 353 N.C.\n 343, 348, 543 S.E.2d 844, 848 (2001). "We review a trial court's\n order for summary judgment de novo to determine whether there is a\n 'genuine issue of material fact' and whether either party is\n 'entitled to judgment as a matter of law.'" Robins v. Town of\n Hillsborough, 361 N.C. 193, 196, 639 S.E.2d 421, 423 (2007)\n (citations omitted). In reviewing a summary judgment order, we\n consider the evidence in the light most favorable to the nonmoving\n party. Bruce-Terminix Co. v. Zurich Ins. Co., 130 N.C. App. 729,\n 733, 504 S.E.2d 574, 577 (1998).\n Defendants' Appeal\n I.\n [1] Defendants the State of North Carolina, the Governor,\n Robert Powell, and David T. McCoy argue the trial court erred by\n granting summary judgment for Plaintiffs on the ground that\n Executive Order No. 3 violated Article V, section 6(2) of the North\n Carolina Constitution. N.C. Const. art. V, § 6(2) provides:\n Neither the General Assembly nor any public\n officer, employee, or agency shall use or\n authorize to be used any part of the funds of\n the Teachers' and State Employees' Retirement\n System or the Local Governmental Employees'\n -9-\n Retirement System for any purpose other than\n retirement system benefits and purposes,\n administrative expenses, and refunds; except\n that retirement system funds may be invested\n as authorized by law, subject to the\n investment limitation that the funds of the\n Teachers' and State Employees' Retirement\n System and the Local Governmental Employees'\n Retirement System shall not be applied,\n diverted, loaned to, or used by the State, any\n State agency, State officer, public officer,\n or public employee.\n Defendants argue that the protections of Article V, section\n 6(2) of the North Carolina Constitution do not apply to their\n actions because the employer contributions at issue in the present\n case were not yet part of the funds of the Retirement System.\n Specifically, Defendants argue that\n no monies appropriated by the General Assembly\n for salaries and related expenses of employees\n are considered employer contributions to the\n Retirement System unless and until they are\n actually remitted to and received by the\n Retirement System, and only then are they\n placed in a Retirement System fund subject to\n the protections of Article V, § 6.\n Plaintiffs counter that a violation of Article V, section 6(2) of\n the North Carolina Constitution occurs where "monies identified as\n employer contributions, and paid in the exact amounts and on the\n exact schedule required for employer contributions by the\n [Retirement] System, are diverted to another use before being\n deposited into the [Retirement] System[.]" Plaintiffs specifically\n focus on the language in Article V, section 6(2) that states that\n Retirement System funds shall not be "diverted."\n We thus determine whether Defendants' actions "diverted"\n Retirement System funds in violation of Article V, section 6(2) of\n -10-\n the North Carolina Constitution. "Issues concerning the proper\n construction of the Constitution of North Carolina 'are in the main\n governed by the same general principles which control in\n ascertaining the meaning of all written instruments.'" State ex\n rel. Martin v. Preston, 325 N.C. 438, 449, 385 S.E.2d 473, 478\n (1989) (quoting Perry v. Stancil, 237 N.C. 442, 444, 75 S.E.2d 512,\n 514 (1953)). "In interpreting our Constitution — as in\n interpreting a statute — where the meaning is clear from the words\n used, we will not search for a meaning elsewhere." Id. at 449, 385\n S.E.2d at 479.\n The term "divert" is defined as follows: "To turn aside from\n a direction or course[.]" Webster's II New College Dictionary 339\n (3d ed. 2005). In the present case, the funds at issue were\n intended for the Retirement System. However, in compliance with\n Executive Order No. 3, the State Controller ordered those funds to\n be deposited in the escrow account. State employers did as\n directed and, as the trial court found, the funds were used\n entirely "for purposes other than retirement system benefits and\n purposes, administrative expenses, and refunds." By these actions,\n Defendants turned the funds aside from their intended destination,\n which was the Retirement System.\n Defendants argue that "the use of the word 'diverted' is\n consistent with the Constitution's protection against misuse of the\n funds which are in the possession of and controlled by the\n Treasurer." Therefore, Defendants argue, the North Carolina\n Constitution does not protect employer contributions not yet\n -11-\n deposited in Retirement System accounts. However, Article V,\n section 6(2) of the North Carolina Constitution not only precludes\n retirement system funds from being "applied," "loaned to," or "used\n by" the State, but also precludes those funds from being "diverted"\n by the State. Even if the terms other than "diverted" apply only\n in the context of funds already held in Retirement System accounts,\n which we do not decide, "we follow the maxims of statutory\n construction that words of a statute are not to be deemed useless\n or redundant[.]" Town of Pine Knoll Shores v. Evans, 331 N.C. 361,\n 366, 416 S.E.2d 4, 7 (1992) (emphasis added). Therefore, we must\n give effect to the term "diverted." See Preston, 325 N.C. at 449,\n 385 S.E.2d at 478 (stating: "'The will of the people as expressed\n in the Constitution is the supreme law of the land. In searching\n for this will or intent all cognate provisions are to be brought\n into view in their entirety and so interpreted as to effectuate the\n manifest purposes of the instrument. The best way to ascertain the\n meaning of a word or sentence in the Constitution is to read it\n contextually and to compare it with other words and sentences with\n which it stands connected.'" (quoting State v. Emery, 224 N.C. 581,\n 583, 31 S.E.2d 858, 860 (1944) (citations omitted))). Applying the\n plain meaning of the term "diverted," we hold that the prohibition\n against seizure of the Retirement System's funds applies to, and\n includes, those funds appropriated and intended for the Retirement\n System, but not yet deposited therein. We thus hold that\n Defendants diverted assets of the Retirement System and, by doing\n so, Defendants violated Article V, section 6(2) of the North\n -12-\n Carolina Constitution. Therefore, we hold the trial court did not\n err by granting Plaintiffs' motion for summary judgment.\n II.\n These Defendants also argue the trial court erred by granting\n summary judgment for Plaintiffs on the ground that Executive Order\n No. 3 violated the Contract Clause of the United States\n Constitution. The Contract Clause of the United States\n Constitution provides that "[n]o state shall . . . pass\n any . . . law impairing the obligation of contracts[.]" U.S.\n Const. art. I, § 10. In order to determine whether there has been\n a violation of the Contract Clause, a court must ascertain the\n following: "(1) whether a contractual obligation is present, (2)\n whether the state's actions impaired that contract, and (3) whether\n the impairment was reasonable and necessary to serve an important\n public purpose." Bailey v. State of North Carolina, 348 N.C. 130,\n 140-41, 500 S.E.2d 54, 60 (1998). We address these requirements in\n the following three subsections.\n A.\n [2] It is well settled, and Defendants do not contest, that a\n contractual relationship exists between vested State employees and\n the State's retirement systems, and that vested State employees\n have contractual rights to their retirement benefits. See id. at\n 150, 500 S.E.2d at 66 (holding that "the relationship between the\n Retirement Systems and employees vested in the system is\n contractual in nature, [and] the right to benefits exempt from\n state taxation is a term of such contract"); Faulkenbury v.\n -13-\n Teachers' and State Employees' Ret. Sys., 345 N.C. 683, 690, 483\n S.E.2d 422, 427 (1997) (recognizing that vested state employees\n have contractual rights to disability benefits calculated pursuant\n to the method in place when they vested); Simpson v. N.C. Local\n Gov't Employees' Retirement System, 88 N.C. App. 218, 224, 363\n S.E.2d 90, 94 (1987), aff'd per curiam, 323 N.C. 362, 372 S.E.2d\n 559 (1988) (holding that "members of the North Carolina Local\n Governmental Employees' Retirement System[] ha[ve] a contractual\n right to rely on the terms of the retirement plan as these terms\n existed at the moment their retirement rights became vested").\n Defendants argue, however, that Plaintiffs do not have a\n contractual right to a retirement system that is funded in an\n actuarially sound manner, as concluded by the trial court.\n Defendants also argue that "[P]laintiffs have neither alleged nor\n shown that they are not receiving the benefits to which they are\n entitled, nor can they show that their benefits have in any way\n been harmed or are in danger of being harmed by action taken\n pursuant to Executive Order No. 3."\n In Bailey, our Supreme Court recognized that the determination\n that a contractual relationship exists does not end the inquiry;\n "[t]his Court must determine whether the tax exemption was a\n condition or term included in the retirement contract." Bailey,\n 348 N.C. at 146, 500 S.E.2d at 63; see also Robertson v.\n Kulongoski, 466 F.3d 1114, 1117 (9th Cir. 2006), cert. denied, 550\n U.S. ___, 167 L. Ed. 2d 1092 (2007) (stating that "'[t]he first\n sub-inquiry is not whether any contractual relationship whatsoever\n -14-\n exists between the parties, but whether there was a "contractual\n agreement regarding the specific . . . terms allegedly at issue"'"\n (citations omitted)). In Bailey, our Supreme Court held that the\n trial court's finding that the plaintiffs had a contractual right\n to the tax exemption at issue was supported by evidence of\n creation of various statutory tax exemptions\n by the legislature, the location of those\n provisions alongside the other statutorily\n created benefit terms instead of within the\n general income tax code, the frequency of\n governmental contract making, communication of\n the exemption by governmental agents in both\n written and oral form, use of the exemption as\n inducement for employment, mandatory\n participation, reduction of periodic wages by\n contribution amount (evidencing compensation),\n loss of interest for those not vesting,\n establishment of a set time period for\n vesting, and the reliance of employees upon\n retirement compensation in exchange for their\n services. Thus, it is clear the tax exemption\n was a term or condition of benefits of the\n Retirement Systems to which [the] plaintiffs\n have a contractual right.\n Bailey, 348 N.C. at 146, 500 S.E.2d at 63.\n In the present case, our Court must determine whether, as a\n term of their contracts for retirement benefits, Plaintiffs were\n entitled to have the Retirement System funded in an actuarially\n sound manner. An "actuarially sound retirement system" is defined\n as a "retirement plan that contains sufficient funds to pay future\n obligations, as by receiving contributions from employees and the\n employer to be invested in accounts to pay future benefits."\n Black's Law Dictionary 39 (8th ed. 2004).\n We first examine the statutes in effect at the time of the\n diversion of the employer contributions. See Wells v. Consolidated\n -15-\n Jud'l Ret. Sys. of N.C., 136 N.C. App. 671, 673, 526 S.E.2d 486,\n 488-89 (2000), aff'd, 354 N.C. 313, 553 S.E.2d 877, reh'g denied,\n 354 N.C. 580, 559 S.E.2d 553 (2001) (stating that "[t]he\n [retirement] contract is embodied in state statute and governed by\n statutory provisions as they existed at the time the employee's\n contractual rights vested"). Specifically, we consider N.C. Gen.\n Stat. § 135-8(d)(1), which deals with the method of financing the\n Retirement System, and which provides as follows:\n On account of each member there shall be paid\n in the pension accumulation fund by employers\n an amount equal to a certain percentage of the\n actual compensation of each member to be known\n as the "normal contribution," and an\n additional amount equal to a percentage of his\n actual compensation to be known as the\n "accrued liability contribution." The rate\n per centum of such contributions shall be\n fixed on the basis of the liabilities of the\n Retirement System as shown by actuarial\n valuation.\n N.C. Gen. Stat. § 135-8(d)(1) (2001) (emphasis added). An\n actuarial valuation is determined by an actuary, who is a\n "statistician who determines the present effects of future\n contingent events; esp., one who calculates insurance and pension\n rates on the basis of empirically based tables." Black's Law\n Dictionary 39 (8th ed. 2004). N.C. Gen. Stat. § 135-8(d)(3) (2001)\n provides, in pertinent part: "Upon certification by the actuary\n engaged by the Board of Trustees that the accrued liability\n contribution rate may be reduced without impairing the Retirement\n System, the Board of Trustees may cause the accrued liability\n contribution rate to be reduced." This statute demonstrates that\n contributions to the Retirement System can be reduced only if the\n -16-\n State's actuary certifies that such a reduction will not impair the\n Retirement System. However, as we discuss in subsection B below,\n the State's actuary in the present case did not make such a\n certification.\n We next consider N.C. Gen. Stat. § 135-6 (2001), which governs\n the administration of the Retirement System. N.C. Gen. Stat. §\n 135-6(g) (2001) requires the Board of Trustees to "engage such\n actuarial and other service as shall be required to transact the\n business of the Retirement System." N.C. Gen. Stat. § 135-6(h)\n (2001) also requires the following: "The Board of Trustees shall\n keep in convenient form such data as shall be necessary for\n actuarial valuation of the various funds of the Retirement System,\n and for checking the experience of the System." N.C. Gen. Stat. §\n 135-6(i) (2001) provides:\n The Board of Trustees shall keep a record of\n all of its proceedings which shall be open to\n public inspection. It shall publish annually\n a report showing the fiscal transactions of\n the Retirement System for the preceding year,\n the amount of the accumulated cash and\n securities of the System, and the last balance\n sheet showing the financial condition of the\n System by means of an actuarial valuation of\n the assets and liabilities of the Retirement\n System.\n The following statutes, N.C. Gen. Stat. § 135-6(m)-(o) (2001), all\n envision an active role for an actuary in the Retirement System:\n (m) Immediately after the establishment of the\n Retirement System the actuary shall make such\n investigation of the mortality, service and\n compensation experience of the members of the\n System as he shall recommend and the Board of\n Trustees shall authorize, and on the basis of\n such investigation he shall recommend for\n adoption by the Board of Trustees such tables\n -17-\n and such rates as are required in subsection\n (n), subdivisions (1) and (2), of this\n section. The Board of Trustees shall adopt\n tables and certify rates, and as soon as\n practicable thereafter the actuary shall make\n a valuation based on such tables and rates of\n the assets and liabilities of the funds\n created by this Chapter.\n (n) In 1943, and at least once in each\n five-year period thereafter, the actuary shall\n make an actuarial investigation into the\n mortality, service and compensation experience\n of the members and beneficiaries of the\n Retirement System, and shall make a valuation\n of the assets and liabilities of the funds of\n the System, and taking into account the result\n of such investigation and valuation, the Board\n of Trustees shall:\n (1) Adopt for the Retirement System such\n mortality, service and other tables as\n shall be deemed necessary; and\n (2) Certify the rates of contributions\n payable by the State of North Carolina on\n account of new entrants at various ages.\n (o) On the basis of such tables and interest\n assumption rate as the Board of Trustees shall\n adopt, the actuary shall make an annual\n valuation of the assets and liabilities of the\n funds of the System created by this Chapter.\n Upon review of these statutes, it is clear that Plaintiffs had\n a contractual right to the funding of the Retirement System in an\n actuarially sound manner. Therefore, we hold that the right to\n have the Retirement System funded in an actuarially sound manner is\n a term or condition included in Plaintiffs' retirement contracts.\n See Bailey, 348 N.C. at 146, 500 S.E.2d at 63 (holding that "the\n tax exemption was a term or condition of benefits of the Retirement\n Systems to which [the] plaintiffs have a contractual right").\n Moreover, the record in this case, on which the trial court\n -18-\n relied in granting summary judgment for Plaintiffs, contains\n several examples of representations made to Plaintiffs that\n demonstrate that Plaintiffs have a contractual right to have the\n Retirement System funded in an actuarially sound manner. A\n pamphlet in the record entitled, "1975 YOUR RETIREMENT SYSTEM - how\n it works," which was distributed to State employees, stated the\n following:\n YOUR EMPLOYER'S CONTRIBUTIONS\n Your employer contributes the major part\n of the cost of the benefits.\n Your employer is currently contributing\n at the rate of 9.12% of all salaries\n subject to retirement deductions.\n Your employer contributes to your\n retirement until you retire, regardless\n of age. The contributions are based on\n actuarial calculations so that your\n benefits can be provided on a sound\n basis.\n Similarly, a 1 July 1996 pamphlet in the record entitled, "Your\n Retirement Benefits" stated the following: "The State bases\n contributions on the calculations prepared by an actuary." The\n pamphlet further stated the following:\n Funded Status\n The Retirement System has been labeled as\n "actuarially sound" because of the consistent\n use over the years of:\n [--] actuarial assumptions based on\n experience,\n [--] an approved actuarial funding method, and\n [--] the recognition of all promised benefits\n in the actuarial liabilities.\n -19-\n Our decision is further supported by numerous decisions of\n courts in other jurisdictions, which have held that vested state\n employees have a contractual right to have their retirement systems\n funded in an actuarially sound manner. See Municipality of\n Anchorage v. Gallion, 944 P.2d 436 (Alaska 1997); Dadisman v.\n Moore, 384 S.E.2d 816 (W. Va. 1989); Valdes v. Cory, 189 Cal. Rptr.\n 212 (Cal. Ct. App. 1983); Sgaglione v. Levitt, 337 N.E.2d 592 (N.Y.\n 1975); Weaver v. Evans, 495 P.2d 639 (Wash. 1972); Dombrowski v.\n City of Philadelphia, 245 A.2d 238 (Pa. 1968); State Teachers'\n Retirement Board v. Giessel, 106 N.W.2d 301 (Wis. 1960).\n B.\n [3] These Defendants next argue that "even if a contractual\n right to an actuarially sound retirement system exists, there is no\n impairment of that contractual right if there is no impairment or\n diminishment of accrued, vested benefits." In support of their\n argument, Defendants rely upon RPEC v. Charles, 62 P.3d 470 (Wash.\n 2003), and Halstead v. City of Flint, 338 N.W.2d 903 (Mich. Ct.\n App. 1983). However, these cases are distinguishable.\n In RPEC, the Washington Supreme Court held, as we do in the\n present case, that retirees and State employees did "have vested\n contractual rights to the systematic funding of the retirement\n system to maintain actuarial soundness." RPEC, 62 P.3d at 483.\n However, the Court also held that "there is no indication that the\n lowered [employer] contribution rates render the system actuarially\n unsound." Id. at 484. Consequently, the Court held that the\n "appellants have not met their burden of proof that a question of\n -20-\n fact exists as to whether the system is actuarially unsound, i.e.,\n the modifications made in EHB 2487 have not been shown to affect\n Retirees' and Employees' vested pension right." Id. Importantly,\n however, the State Actuary in RPEC had determined that the lowered\n employer contribution rates would not render the system actuarially\n unsound: "The stated reason for reducing the rates was that the\n 1998 valuation from the State Actuary determined that the funding\n goals expressed in former RCW 41.45.010 (1998) could still be met\n using lower contribution rates, primarily because of investment\n returns on the pension funds that were higher than anticipated."\n Id. at 476.\n In contrast to RPEC, the record shows the State's actuary in\n the case before us, Edward A. Macdonald (Mr. Macdonald), stated the\n following in a 6 February 2001 letter to the Deputy Treasurer and\n Director of the Retirement Systems Division of the State Treasurer:\n "The employer rate cannot be reduced effective February 1, 2001 in\n an actuarially sound manner. . . . The System is not being funded\n in an actuarial[ly] sound manner since the actual contributions\n will be less than the annual required contributions." Mr.\n Macdonald also testified at a deposition as follows:\n Q And have you since learning about the case\n formed any opinions concerning issues that\n have been raised in this lawsuit?\n A I had opinions prior to the lawsuit being\n filed.\n Q Can you tell me generally what those\n opinions are?\n A That the escrowing of the money was not\n actuarially sound for the system. I believe I\n -21-\n wrote at least one letter regarding that back\n probably three or four, probably three years\n ago.\n Q Do you have any other opinions concerning\n the escrow of the money?\n A That eventually it ought to be repaid. I\n mean, you know --\n Q Will its having been repaid, assuming that\n happens, will its having been repaid change\n your opinion about whether the action in\n escrowing the money was actuarial[ly] sound?\n A That action is actuarially unsound at that\n time, and repaying the money doesn't really\n change that.\n Defendants, however, point to Mr. Macdonald's deposition\n testimony that his opinion regarding the actuarially unsound manner\n of funding the Retirement System "does not mean the system is\n actuarially unsound. It just means during that fiscal year the\n contribution that was made was not satisfactory of the fund, which\n should have been funded." Mr. Macdonald also testified that\n "maintaining an operating System in an actuarially unsound manner\n does not necessarily render the entire System actuarially\n unsound[.]"\n The courts of other states have previously rejected arguments\n similar to the argument of Defendants. In Dadisman, the West\n Virginia Supreme Court cited Valdes, Weaver, and Dombrowski for the\n following proposition:\n Those cases found that even where a unilateral\n reduction in the state's share of pension\n contributions, as earned by State employees,\n does not result in out-of-pocket losses for\n plan participants, they still have a vested\n interest in the integrity and security of the\n funds available to pay future benefits. See,\n -22-\n e.g., Valdes, 139 Cal. App. 3d at 785, 189\n Cal. Rptr. 222. We agree with this reasoning.\n Dadisman, 384 S.E.2d at 828. We also find this reasoning\n compelling. At the point in time when the employer contributions\n were escrowed in this case, the Retirement System was not being\n funded in an actuarially sound manner. At that time, it was\n unclear when, or even whether, the diverted funds would be repaid.\n Accordingly, the actuarially unsound diversion of funds threatened\n the integrity and security of the Retirement System. Therefore, we\n hold that by diverting the funds, Defendants' actions impaired the\n contractual right of Plaintiffs to a retirement system funded in an\n actuarially sound manner. See Bailey, 348 N.C. at 150-51, 500\n S.E.2d at 66; see also Public Employee Pensions in Times of Fiscal\n Distress, 90 Harv. L. Rev. 992, 1006 (March 1977) (stating that\n "[o]n several occasions, governments have failed to continue the\n actuarial appropriations they had promised to make to the pension\n system, and courts have uniformly held these missed appropriations\n to be contract violations").\n Defendants also rely upon Halstead, in which the Michigan\n Court of Appeals cited the Michigan State Constitution, which\n provided that "accrued financial benefits of the state's or a\n political subdivision's pension plan are a contractual obligation\n which cannot be diminished or impaired." Halstead, 338 N.W.2d at\n 905-06. The Michigan Court of Appeals also recognized that\n "'[a]ccrued financial benefits' have been defined as the right to\n receive certain pension payments upon retirement based on service\n performed." Id. at 906. However, the Court went further in\n -23-\n holding that "[b]ecause there is no evidence that ordinance § 35-\n 16.3 diminishes or impairs the full payment of [the] plaintiffs'\n accrued financial benefits, the ordinance does not violate the\n constitutional proscription against impairment of contracts." Id.\n Halstead is readily distinguishable from the present case because\n it presented the narrow issue of whether the legislative enactment\n violated the constitutional proscription against diminishing\n accrued financial benefits. In contrast, the issue in the case\n before us is whether Defendants' actions violated Plaintiffs'\n contractual right to a retirement system funded in an actuarially\n sound manner.\n C.\n [4] These Defendants also argue that even assuming Plaintiffs\n had a contractual right to a retirement system funded in an\n actuarially sound manner and that Defendants' actions impaired that\n contract, any impairment was reasonable and necessary to serve an\n important public purpose. Defendants contend that "it has been\n stipulated that the escrow of employer contributions mandated by\n Executive Order No. 3 was for the sole purpose of fulfilling the\n constitutional requirement to balance the State budget."\n Therefore, Defendants argue, the escrow of the employer\n contributions was reasonable and necessary to serve the important\n public purpose of avoiding a constitutionally prohibited budget\n deficit.\n Article III, section 5(3) of the North Carolina Constitution\n provides:\n -24-\n The total expenditures of the State for the\n fiscal period covered by the budget shall not\n exceed the total of receipts during that\n fiscal period and the surplus remaining in the\n State Treasury at the beginning of the period.\n To insure that the State does not incur a\n deficit for any fiscal period, the Governor\n shall continually survey the collection of the\n revenue and shall effect the necessary\n economies in State expenditures[.]\n N.C. Const. art. III, § 5(3). As our Court has recognized, "[t]his\n provision clearly places a duty upon the Governor to balance the\n budget and prevent a deficit." County of Cabarrus v. Tolson, 169\n N.C. App. 636, 638, 610 S.E.2d 443, 445, disc. review denied, 359\n N.C. 630, 616 S.E.2d 229 (2005).\n We agree with the trial court and with Defendants that the\n avoidance of a constitutionally prohibited budget deficit is\n clearly an important public purpose. However, we also agree with\n the trial court that the escrow of the funds in the present case\n was not reasonable and necessary to achieve that purpose. As we\n recognized in our earlier discussion of Article V, section 6(2) of\n the North Carolina Constitution, retirement funds specifically\n receive special constitutional protection in North Carolina.\n Article V, section 6(2) of the North Carolina Constitution plainly\n provides that the State may not use retirement funds for any\n purpose other than "retirement system benefits and purposes,\n administrative expenses, and refunds[.]" N.C. Const. art. V, §\n 6(2). This constitutional provision demonstrates the strong public\n policy of North Carolina in favor of the inviolability of\n retirement funds. See John V. Orth, The North Carolina State\n Constitution 127 (1993) (stating: "By constitutional amendment,\n -25-\n approved in 1950 and carried forward into the 1971 Constitution,\n teachers and state employees secured constitutional protection for\n their retirement funds; in 1971 local government employees were\n given the same protection. Money in the funds may not even be\n loaned to the state." (emphasis added)). Defendants argue that the\n constitutional protection of Retirement System funds extends only\n to those funds actually held by the Retirement System and that it\n is not against public policy to use funds not yet received by the\n Retirement System to balance the budget. Again, we reject this\n argument. As we held above, the escrow of the employer\n contributions was a diversion of Retirement System funds that was\n prohibited by the North Carolina Constitution.\n Our Supreme Court in Bailey considered whether the impairment\n at issue in that case was reasonable and necessary to serve an\n important public purpose. Bailey, 348 N.C. at 151-53, 500 S.E.2d\n at 66-67. Our Supreme Court recognized:\n "In applying this standard, . . . complete\n deference to a legislative assessment of\n reasonableness and necessity is not\n appropriate because the State's self-interest\n is at stake. A governmental entity can always\n find a use for extra money, especially when\n taxes do not have to be raised. If a State\n could reduce its financial obligations\n whenever it wanted to spend the money for what\n it regarded as an important public purpose,\n the Contract Clause would provide no\n protection at all."\n Id. at 151-52, 500 S.E.2d at 66 (quoting United States Trust Co. v.\n New Jersey, 431 U.S. 1, 25-26, 52 L. Ed. 2d 92, 112, reh'g denied,\n 431 U.S. 975, 53 L. Ed. 2d 1073 (1977)). Our Supreme Court thus\n held:\n -26-\n While it is clear that the state interest in\n this case — complying with a Supreme Court\n ruling — was important, what is equally clear\n is that the method chosen was not necessary to\n achieve the state interest asserted. In\n Davis, the Supreme Court did not tell North\n Carolina that it was required to tax state and\n local employees; nor did it set forth any\n mandatory scheme of compliance. Davis v.\n Michigan Dep't of Treasury, 489 U.S. 803, 103\n L. Ed. 2d 891. The Court merely held that\n federal retirees had to be treated the same as\n state and local retirees. Id. There are\n numerous ways that the State could have\n achieved this goal without impairing the\n contractual obligations of [the] plaintiffs.\n Id. at 152, 500 S.E.2d at 67.\n As in Bailey, we cannot say in the case before us that "the\n method chosen" of diverting employer contributions to the\n Retirement System was "necessary to achieve the state interest\n asserted." See id. A balanced budget could have been achieved in\n another way without diverting Retirement System funds that have\n been afforded special constitutional protection.\n Our Supreme Court further held in Bailey that\n the method chosen was not reasonable under the\n circumstances. The legislature sought a\n "revenue neutral" approach to complying with\n the Davis decision, meaning that legislators\n would be faced with neither raising taxes nor\n cutting other programs in order to comply.\n However, this convenient approach impaired\n vested rights of current and future state and\n local retirees to whom the State had made\n promises of exemption in consideration of\n their many years of public service.\n Id.\n Similarly, in the present case, instead of seeking a tax\n increase or cuts in other State programs that did not enjoy special\n constitutional protection, Defendants diverted the employer\n -27-\n contributions to the Retirement System. Our Court cannot say that\n this diversion, which impaired the contractual right of Plaintiffs\n to a retirement system funded in an actuarially sound manner, was\n reasonable.\n For the reasons stated above, we hold that the trial court did\n not err by granting summary judgment to Plaintiffs regarding their\n claim for declaratory judgment under the Contract Clause. Because\n we affirm the judgment of the trial court for Plaintiffs on the\n merits of their declaratory judgment claims under Article V,\n section 6(2) of the North Carolina Constitution and under the\n Contract Clause of the United States Constitution, we need not\n address Plaintiffs' cross-assignments of error.\n Plaintiffs' Appeal\n [5] Plaintiffs argue the trial court erred by granting summary\n judgment to the Treasurer and to the Board on Plaintiffs' claim for\n writ of mandamus to compel compliance with N.C. Gen. Stat. § 135-\n 8(f)(3). N.C. Gen. Stat. § 135-8(f)(3) provides:\n In the event the employee or employer\n contributions required under this section are\n not received by the date set by the Board of\n Trustees, the Board shall assess the employer\n with a penalty of 1% per month with a minimum\n penalty of twenty-five dollars ($25.00). If\n within 90 days after request therefor by the\n Board any employer shall not have provided the\n System with the records and other information\n required hereunder or if the full accrued\n amount of the contributions provided for under\n this section due from members employed by an\n employer or from an employer other than the\n State shall not have been received by the\n System from the chief fiscal officer of such\n employer within 30 days after the last due\n date as herein provided, then, notwithstanding\n anything herein or in the provisions of any\n -28-\n other law to the contrary, upon notification\n by the Board to the State Treasurer as to the\n default of such employer as herein provided,\n any distributions which might otherwise be\n made to such employer from any funds of the\n State shall be withheld from such employer\n until notice from the Board to the State\n Treasurer that such employer is no longer in\n default.\n N.C. Gen. Stat. § 135-8(f)(3) (2007) (emphases added). Based upon\n the first sentence of the statute, Plaintiffs argue that because\n the employer contributions diverted pursuant to Executive Order No.\n 3 were not "received" by the Retirement System when they were due,\n the Board should have assessed a one percent per month penalty to\n those employers. Plaintiffs further argue that pursuant to the\n second sentence of the statute, all State funds should have been\n withheld from employers whose employer contributions were not\n received by the Retirement System "within 30 days after the last\n due date as [therein] provided[.]" We disagree.\n The statute speaks in terms of a default by an employer. The\n term "default" is defined as the "[f]ailure to perform a task or\n fulfill an obligation, esp. failure to meet a financial\n obligation." Webster's II New College Dictionary 301 (3d ed.\n 2005); see also Black's Law Dictionary 449 (8th ed. 2004) (defining\n "default" as "[t]he omission or failure to perform a legal or\n contractual duty; esp., the failure to pay a debt when due"). In\n the present case, the State employers did not default on their\n obligation to remit their employer contributions to the Retirement\n System. State employers calculated the amount of their employer\n contributions to the Retirement System, but then were ordered to\n -29-\n pay those amounts into the escrow account. As the trial court\n found,\n the fact that the employer contributions\n affected by Executive Order No. 3 were not\n received by the Retirement System was neither\n the result of the intent of any employers to\n withhold contributions nor of negligence on\n the part of any employers, but rather was the\n result of an intervening executive order that\n employers were bound to follow unless and\n until directed otherwise by competent\n authority.\n Moreover, as the trial court found,\n [f]or several years prior to and following the\n issuance of Executive Order No. 3, the\n Department of State Treasurer had routinely\n waived the imposition of the 1% per month fine\n provided in N.C. Gen. Stat. § 135-8(f)(3) for\n employers from whom the System had not\n received required payments when due when the\n Department of State Treasurer determined, in\n its discretion, that the employer had\n demonstrated lack of intent to fail to remit\n the contributions in a timely manner.\n N.C. Gen. Stat. § 135-6(f) (2007) provides: "The Board of Trustees\n shall also, from time to time, in its discretion, adopt rules and\n regulations to prevent injustices and inequalities which might\n otherwise arise in the administration of this Chapter." The\n Board's practice of waiving penalties under circumstances where\n employers were not at fault for failing to remit employer\n contributions to the Retirement System is entirely consistent with\n the Board's statutory discretion to adopt rules "to prevent\n injustices." Accordingly, for all the reasons stated above, the\n Treasurer and the Board were not required to punish those employers\n whose employer contributions were not deposited in the Retirement\n System. Therefore, we hold the trial court did not err by granting\n -30-\n summary judgment to the Treasurer and to the Board on Plaintiffs'\n claim for writ of mandamus.\n Affirmed.\n Judges WYNN and CALABRIA concur."""
err3 = """NATIONWIDE MUTUAL FIRE INSURANCE COMPANY, Plaintiff, v.\n KONSTANTIN MNATSAKANOV, LIANA MNATSAKANOV, AMIRAN MNATSAKANOV and\n MELISSA C. MCCALISTER, Defendants\n NO. COA07-1004\n Filed: 5 August 2008\n 1. Appeal and Error–-preservation of issues--appellate rules violations--assignments of\n error abandoned\n Defendant’s assignments of error that violated N.C. R. App. P. 28(b)(6) were not\n considered and were deemed abandoned.\n 2. Insurance--homeowners-–effective date of restriction--dog bite\n The trial court erred by entering summary judgment for plaintiff insurance company\n because a genuine issue of material fact existed as to whether a restriction of coverage for a\n homeowners policy for any occurrence caused by insured’s dog became effective on the date the\n restriction was signed by the insured or on the date of the policy’s renewal and thus whether the\n policy covered a claim under the homeowners policy for a dog bite that occurred after the\n restriction was signed but before the renewal date.\n Appeal by defendant from order entered 2 May 2007 by Judge\n Linwood O. Foust in Mecklenberg County Superior Court. Heard in\n the Court of Appeals 20 February 2008.\n Baucom, Claytor, Benton, Morgan & Wood, P.A., by Rex C.\n Morgan, for plaintiff-appellee.\n Price, Smith, Hargett, Petho & Anderson, by Wm. Benjamin\n Smith, for defendant-appellant.\n BRYANT, Judge.\n Defendants Konstantin Mnatsakanov, Liana Mnatsakanov, Amiran\n Mnatsakanov, and Melissa McCalister appeal from an order granting\n summary judgment for Nationwide Mutual Fire Insurance Company. For\n -2-\n the reasons stated herein, we reverse and remand this case to the\n trial court.\n On 15 July 2005, Konstantin Mnatsakanov received a letter from\n Nationwide notifying him that his Homeowner’s Policy would not\n renew on 31 October 2005 because he had a Rottweiler dog on the\n premises. On 15 July 2005, Konstantin Mnatsakanov met with\n Nationwide agent Gary Griffith and requested that his policy not be\n cancelled on 31 October 2005 but that it be renewed. After\n contacting Nationwide Underwriting, Griffith advised Konstantin\n that the policy would be renewed if Konstantin agreed to a\n restriction of coverage for any occurrence caused by his dog as set\n forth on the “Restriction of Individual Policies” (Endorsement H-\n 7030A). Konstantin agreed to the restriction. Both he and\n Griffith signed the “Restriction of Individual Policies” on 15 July\n 2005. The “Restriction of Individual Policies,” (H-7030-A)\n exempted from coverage any claim brought against the insured\n “caused by any animal, owned or in the care of the insured.”\n However, the restriction did not state an effective date.\n Melissa McCalister filed a claim for personal injuries that\n occurred 13 October 2005 when she was bitten by a dog owned by the\n Mnatsakanovs. The Mnatsakanovs requested coverage under the\n Nationwide policy for the claim asserted by McCalister. Nationwide\n filed a Declaratory Judgment action naming the Mnatsakanovs and\n -3-\n McCalister as defendants and asking the trial court to determine if\n the insurance policy covered McCalister’s claim. Nationwide\n thereafter moved for summary judgment asking the trial court to\n find as a matter of law that the policy excluded coverage for the\n dog bite injury suffered by McCalister on the Mnatsakanov’s\n property.\n The trial court found that the effective date of the\n “Restriction of Individual Policies” was 15 July 2005, the date it\n was signed by Nationwide Policyholder Konstantin Mnatsakanov and\n Nationwide agent Gary Griffith. The trial court found that in\n exchange for agreeing to the restriction, Nationwide promised not\n to cancel the Mnatsakanov’s policy on 31 October 2005 but renew the\n Policy for another year. The trial court found that Nationwide’s\n agreement to not cancel the Mnatsakanov’s policy on 31 October 2005\n and renew the policy for another year constituted adequate\n consideration for the Restriction of Individual Policies signed by\n Mr. Mnatsakanov and Nationwide Agent Griffith.\n Based on those findings, the trial court concluded that the\n effective date of the modification of the Nationwide Policy as set\n forth in the “Restriction of Individual Policies” was 15 July 2005;\n the renewal of the policy from 31 October 2005 through 31 October\n 2006 constituted adequate consideration for the 15 July 2005\n modification of the policy; and the language set forth in the\n -4-\n “Restriction of Individual Policies” effectively excluded any\n liability coverage or medical payments coverage for injuries\n sustained by McCalister. On these grounds, the trial court granted\n summary judgment in favor of Nationwide.\n Konstantin Mnatsakanov, Liana Mnatsakanov, Amiran Mnatsakanov\n and Melissa McCalister (collectively “defendants”) appealed.\n ____________________________________\n Defendants present six issues on appeal: whether the trial\n court committed reversible error by (I) granting summary judgment\n for Nationwide; (II) finding as fact that the effective date of the\n endorsement of the “Restriction of Individual Policies” was 15 July\n 2005; (III) finding as fact that Nationwide agreed not to cancel\n the Mnatsakanov’s policy in exchange for signing the restriction on\n 15 July 2005; (IV) finding that no coverage existed for Melissa\n McCalister’s injury claim; (V) concluding that the effective date\n of the restriction on the policy was 15 July 2005; and (VI) finding\n that the renewal of the policy was consideration for restricting\n the policy on the date it was signed.\n [1] Because the dispositive issue is whether there was a\n genuine issue of material fact as to the effective date of the\n endorsement of the “Restriction of Individual Policies,” and\n because many of defendant’s other issues violate our appellate\n rules, we do not reach those other issues. See N.C.R. App. P.\n -5-\n 28(b)(6) (2007) (“assignments of error not set out in the\n appellant’s brief, or in support of which no reason or argument is\n stated or authority cited, are deemed abandoned”).\n Standard of Review\n Where a motion for summary judgment has been granted, the two\n critical questions on appeal are whether, on the basis of the\n materials presented to the trial court, (1) there is no genuine\n issue of material fact, and (2) the moving party is entitled to\n judgment as a matter of law. North River Ins. Co. v. Young, 117\n N.C. App. 663, 667, 453 S.E.2d 205, 208 (1995).\n An issue is material if the facts alleged are\n such as to constitute a legal defense or are\n of such nature as to affect the result of the\n action, or if the resolution of the issue is\n so essential that the party against whom it is\n resolved may not prevail.\n Kessing v. Mortgage Corp., 278 N.C. 523, 534, 180 S.E.2d 823, 830\n (1971). Moreover, the evidence presented by the parties must be\n viewed in the light most favorable to the non-movant. Bruce-\n Terminix Co. v. Zurich Ins. Co., 130 N.C. App. 729, 733, 504 S.E.2d\n 574, 577 (1998). “Review of summary judgment on appeal is\n necessarily limited to whether the trial court’s conclusion as to\n these questions of laws were correct ones.” Young, 117 N.C. App.\n at 667, 453 S.E.2d at 208. Hence, the standard of review of an\n order granting summary judgment is de novo. Diggs v. Novant\n -6-\n Health, Inc., 177 N.C. App. 290, 294, 628 S.E.2d 851, 855 (2006).\n ___________________________________\n [2] Defendants argue the trial court erred by finding and\n concluding that the effective date of the “Restriction of\n Individual Policies” was 15 July 2005. We agree.\n “The heart of a contract is the intention of the parties,\n which is to be ascertained from the expressions used, the subject\n matter, the end in view, the purpose sought, and the situation of\n the parties at the time.” Gould Morris Elec. Co. v. Atlantic Fire\n Ins. Co., 229 N.C. 518, 520, 50 S.E.2d 295, 297 (1948). “It is\n essential to the formation of any contract that there be mutual\n assent of both parties to the terms of the agreement so as to\n establish a meeting of the minds.” Creech v. Melnik, 347 N.C. 520,\n 527, 495 S.E.2d 907, 911-12 (1998). “This rule of contract law is\n founded on the proposition that there can be no contract without a\n meeting of the minds.” Cunningham v. Brown, 51 N.C. App. 264, 270,\n 276 S.E.2d 718, 723 (1981). “Whether mutual assent is established\n and whether a contract was intended between parties are questions\n for the trier of fact.” Creech, 347 N.C. at 527, 495 S.E.2d at\n 911.\n In the instant case, defendants contend that the “Restriction\n of Individual Policies” was written to apply beginning on 31\n October 2005, the renewal date of the policy. There was no\n -7-\n indication on the “Restriction of Individual Policies” form as to\n when the exclusion would take effect. Nationwide agent Griffith\n testified in his deposition that “the practice of Nationwide would\n be to send a letter to the insured letting him know that his\n coverage is continued.” Griffith stated he knew Nationwide had the\n ability to non-renew a policy, but not whether they had the right\n to cancel the policy immediately. When asked if there was any\n correspondence regarding renewal of coverage, he said, “renewal\n notices may or may not be in the file [for the Mnatsakanovs].”\n Agent Griffith also testified that he “could not answer for\n Nationwide if it was a requirement that the exclusion take effect\n in July for a renewal date in October.”\n To the contrary, Nationwide argues that the “Restriction of\n Individual Policies” was effective on 15 July 2005, the day\n Konstantin Mnatsakanov signed the exclusion. Nationwide argues\n Konstantin Mnatsakanov understood verbally and in writing, that the\n agreement was effective immediately. In his deposition, Agent\n Griffith testified that “it was explained to Mr. Mnatsakanov that\n effective immediately, 15 July 2005, there is no coverage for the\n dog, and he verbally expressed that he understood.” Moreover,\n Griffith testified that “Konstantin Mnatsakanov was informed by the\n underwriter while he was in the office that if he was willing to\n sign an endorsement provided by Nationwide, which he understood\n -8-\n completely, there would be no coverage with his signature for any\n liability involving the dog.” In summary, Nationwide argues that\n Griffith’s testimony provided that the exclusion signed 15 July\n 2005 was effective immediately because “the dog was a danger to\n society; [to] families living around the Mnatsakanovs; the sooner\n that Konstantin understood what would be covered and not covered,\n the better he understood it took place immediately.”\n Because the “Restriction of Individual Policies” excluded any\n liability for a claim or suit brought against an insured for any\n occurrence involving a dog and its effective date bears upon the\n determination that the Mnatsakanov’s Nationwide insurance policy\n covers damages for the dog bite injury to McCalister, we believe a\n genuine issue of material fact exists as to when the “Restriction\n of Individual Policies” was effective. Accordingly, we reverse the\n trial court’s grant of summary judgment and remand for further\n proceedings.\n Reversed and remanded.\n Judges HUNTER and JACKSON concur."""
err4 = """1 Bruce-Terminix Co. v. Zurich Ins. Co., 130 N.C. App. 729,\n 733, 504 S.E.2d 574, 577 (1998).\n JOY B. MURRAY Plaintiff, v. COUNTY OF PERSON; et al., Defendants\n NO. COA07-1260\n Filed: 5 August 2008\n 1. Appeal and Error–appealability--partial denial of summary\n judgment–governmental immunity\n An appeal from the denial of summary judgment involving governmental immunity was\n interlocutory but properly before the Court of Appeals.\n 2. Immunity–public duty doctrine–suit in individual capacity\n The public duty doctrine does not extend to government workers sued only in their\n individual capacities, and summary judgment was properly denied to defendants on that ground\n in an action against employees of a county health department arising from the failure of a septic\n system.\n 3. Immunity–public officers–not available\n Public officers immunity was not available to health department employees in the\n positions of Environmental Health Specialist and Environmental Health Supervisor, and the trial\n court correctly denied summary judgment for defendants on that issue in an action arising from\n the failure of a septic system.\n Appeal by defendants from judgment entered 24 July 2007 by\n Judge Richard W. Stone in Superior Court, Person County. Heard in\n the Court of Appeals 18 March 2008.\n Alan S. Hicks, P.A., by Alan S. Hicks, for plaintiff-appellee.\n Womble Carlyle Sandridge & Rice, PLLC, by James R. Morgan, Jr.\n and Robert T. Numbers, II, for defendants-appellants Clayton,\n Kelly, and Sarver.\n WYNN, Judge.\n The standard of review for a motion for summary judgment is\n whether there is any genuine issue of material fact and whether the\n moving party is entitled to a judgment as a matter of law.1 Here,\n -2-\n the defendants argue the trial court erred by partially denying\n their motion for summary judgment because they are entitled to the\n protection of the public duty doctrine and public officers’\n immunity. Because we hold that neither the public duty doctrine\n nor public officers’ immunity protects the defendants from\n liability, we affirm the trial court’s partial denial of the\n defendants’ motion for summary judgment.\n On 4 September 2002, Herman Rouse, Plaintiff Joy Murray’s\n contractor and builder, applied for an improvement permit from the\n Person County Health Department. On 6 November 2002, Defendant\n Adam Sarver, an Environmental Health Specialist for the Person\n County Health Department, conducted a site evaluation on Ms.\n Murray’s property and issued an improvement permit approving the\n installation of an innovative wastewater treatment system on the\n property. The improvement permit stated that “[n]either Person\n County nor the Environmental Health Specialist warrants that the\n septic tank system will continue to function satisfactorily in the\n future or that the water supply will remain potable.”\n On 13 March 2003, an innovative wastewater treatment system\n was installed on Ms. Murray’s property. On 19 March 2003, Mr.\n Sarver issued an Operation Permit, indicating that the system had\n been installed in compliance with statutory law.\n The construction of Ms. Murray’s home was completed in March\n 2003 and she moved into the home in April 2003. Shortly after she\n moved in, Ms. Murray noticed water surfacing on her property and\n -3-\n she notified Mr. Rouse. Mr. Rouse visited Ms. Murray’s property\n and informed her that she had a problem with her septic system.\n Ms. Murray reported the problems with her wastewater system to\n the county, and over the next several months, Mr. Sarver, along\n with Defendant Harold Kelly, another Environmental Health\n Specialist, and Defendant Janet Clayton, an Environmental Health\n Supervisor, made numerous unsuccessful attempts to repair Ms.\n Murray’s wastewater system. These attempts involved multiple\n inspections and observations of the wastewater system, the issuance\n of permits for the installation of a new line, and eventually, the\n installation of a new innovative system. However, the new\n innovative wastewater treatment system, installed in February 2004,\n also failed.\n On 15 June 2006, Ms. Murray initiated this action against\n Person County and the Person County Health Department; and against\n Mr. Sarver, Ms. Clayton, and Mr. Kelly, individually and in their\n official capacities. She alleged negligence, negligent\n misrepresentation, and negligent infliction of emotional distress\n in the issuance of permits for the installation and repair of her\n wastewater treatment system.\n On 29 May 2007, Defendants filed a motion for summary judgment\n as to all of Ms. Murray’s claims. The trial court heard\n Defendants’ motion on 24 July 2007 and granted Defendants’ motion\n as to all claims against Person County and the Person County Health\n Department; and Mr. Sarver, Ms. Clayton, and Mr. Kelly in their\n official capacities. The trial court also granted summary judgment\n -4-\n on Ms. Murray’s claim of negligent infliction of emotional\n distress. However, the trial court denied summary judgment as to\n Ms. Murray’s claims for negligence and negligent misrepresentation\n against Mr. Sarver, Ms. Clayton, and Mr. Kelly in their individual\n capacities.\n [1] On appeal, Mr. Sarver, Ms. Clayton, and Mr. Kelly\n (collectively “Defendants”) argue the trial court erred by\n partially denying their motion for summary judgment. Specifically,\n Defendants contend that they are entitled to the protection of the\n public duty doctrine and public officers’ immunity. Though\n interlocutory, Defendants’ appeal from the denial of summary\n judgment is properly before this Court because appeals which\n present defenses of governmental or sovereign immunity, like the\n public duty doctrine or public officers’ immunity, have been held\n by this Court to be immediately appealable as affecting a\n substantial right. Schlossberg v. Goins, 141 N.C. App. 436, 439,\n 540 S.E.2d 49, 52 (2000), disc. review denied, 355 N.C. 215, 560\n S.E.2d 136 (2002); Derwort v. Polk County, 129 N.C. App. 789,\n 790-91, 501 S.E.2d 379, 380 (1998).\n The standard of review from the denial of summary judgment is\n de novo. Moody v. Able Outdoor, Inc., 169 N.C. App. 80, 83, 609\n S.E.2d 259, 261 (2005). We review whether there is any genuine\n issue of material fact and whether the moving party is entitled to\n a judgment as a matter of law. Bruce-Terminix Co. v. Zurich Ins.\n Co., 130 N.C. App. 729, 733, 504 S.E.2d 574, 577 (1998). Though we\n view the evidence presented by the parties in the light most\n -5-\n favorable to the non-movant, summary judgment is appropriate when\n “(1) an essential element of plaintiff’s claim is nonexistent[,]\n (2) plaintiff cannot produce evidence to support an essential\n element of his claim, or (3) plaintiff cannot surmount an\n affirmative defense which would bar the claim.” Gibson v. Mutual\n Life Ins. Co. of New York, 121 N.C. App. 284, 286, 465 S.E.2d 56,\n 58 (1996).\n [2] Defendants first argue the trial court erred by denying\n summary judgment on the claims of negligence and negligent\n misrepresentation in their individual capacities because they are\n protected from liability by the public duty doctrine. We disagree.\n The public duty doctrine “provides that governmental entities\n and their agents owe duties only to the general public, not to\n individuals, absent a ‘special relationship’ or ‘special duty’\n between the entity and the injured party.” Stone v. North Carolina\n Dept. of Labor, 347 N.C. 473, 477-78, 495 S.E.2d 711, 714 (citation\n omitted), cert. denied, 525 U.S. 1016, 142 L. Ed. 2d 449 (1998).\n “Because the governmental entity owes no particular duty to any\n individual claimant, it cannot be held liable for negligence . . .\n .” Id. at 482, 495 S.E.2d at 716. The purpose of the public duty\n doctrine is “to prevent an overwhelming burden of liability on\n governmental agencies with limited resources.” Id. at 481, 495\n S.E.2d at 716 (internal citations omitted).\n Although the public duty doctrine was initially adopted in the\n context of municipal law enforcement, Braswell v. Braswell, 330\n N.C. 363, 410 S.E.2d 897 (1991), reh’g denied, 330 N.C. 854, 413\n -6-\n 2 We note that in Watts, the action was brought before the\n Industrial Commission against an employee of the Health Department,\n the Health Department, and North Carolina Department of Environment\n and Natural Resources; however, the Deputy Commissioner dismissed\n the claim against the employee, as he was not a proper party.\n Watts, 182 N.C. App. at 180, 641 S.E.2d at 815.\n S.E.2d 550 (1992), our Supreme Court has extended the public duty\n doctrine “to claims against the State under the Tort Claims Act,”\n Stone, 347 N.C. at 482, 495 S.E.2d at 716, and “to state agencies\n required by statute to conduct inspections for the public’s general\n protection.” Lovelace v. City of Shelby, 351 N.C. 458, 461, 526\n S.E.2d 652, 654, reh’g denied, 352 N.C. 157, 544 S.E.2d 225 (2000).\n Additionally, this Court has held that “the Health Department, an\n agent of [North Carolina Department of Environment and Natural\n Resources], is a state agency required [by statute] to inspect site\n for suitability of wastewater treatment systems before issuing\n improvement permits . . . and therefore may avail itself of the\n protection afforded by the public duty doctrine.” Watts v. North\n Carolina Dept. of Env’t. and Natural Resources, 182 N.C. App. 178,\n 182, 641 S.E.2d 811, 816 (2007), disc. review granted, _ N.C. _,\n 660 S.E.2d 899.2\n However, our review of North Carolina case law has revealed no\n cases in which our courts have held that an employee of a health\n department is entitled to the protection of the public duty\n doctrine when sued only in his or her individual capacity in\n Superior Court. Our Supreme Court has explained: “A suit against\n a defendant in his individual capacity means that the plaintiff\n seeks recovery from the defendant directly; a suit against a\n -7-\n defendant in his official capacity means that the plaintiff seeks\n recovery from the entity of which the public servant defendant is\n an agent.” Meyer v. Walls, 347 N.C. 97, 110, 489 S.E.2d 880, 887\n (1997).\n Here, the only claims remaining against Defendants are in\n their individual capacities. Where a governmental worker is sued\n in his individual capacity, rather than applying the public duty\n doctrine, our courts have consistently applied public officers’\n immunity. See Isenhour v. Hutto, 350 N.C. 601, 609, 517 S.E.2d\n 121, 127 (1999) (“Once we determine the aggrieved party has\n sufficiently pled a claim against defendant in his or her\n individual capacity, we must determine whether that defendant is a\n public official or a public employee.”); Epps v. Duke Univ., Inc.,\n 122 N.C. App. 198, 205, 468 S.E.2d 846, 851-52 (1996) (“To sustain\n the personal or individual capacity suit, the plaintiff must\n initially make a prima facie showing that the defendant-official’s\n tortious conduct falls within one of the immunity exceptions[.]”);\n EEE-ZZZ Lay Drain Co. v. North Carolina Dept. of Human Res., 108\n N.C. App. 24, 28, 422 S.E.2d 338, 341 (1992) (“When a governmental\n worker is sued in his individual capacity, our courts have\n distinguished between whether the worker is an officer or an\n employee when assessing liability.”). We hold that the public duty\n doctrine does not extend to government workers sued only in their\n individual capacities. Accordingly, this assignment of error is\n overruled, and we now turn to a discussion of public officers’\n immunity.\n -8-\n [3] Defendants next argue the trial court erred by denying\n summary judgment on the claims of negligence and negligent\n misrepresentation in their individual capacities because they are\n entitled to public officers’ immunity. We disagree.\n It is well established that “[p]ublic officers are shielded\n from liability unless their actions are corrupt or malicious[;]”\n however, public employees can be held personally liable for mere\n negligence. EEE-ZZZ Lay Drain Co., 108 N.C. App. at 28-29, 422\n S.E.2d at 341 (citing Smith v. State, 289 N.C. 303, 331, 222 S.E.2d\n 412, 430 (1976)), overruled on other grounds, Meyer v. Walls, 347\n N.C. 97, 489 S.E.2d 880 (1997). In distinguishing between a public\n official and a public employee, our courts have held that “(1) a\n public office is a position created by the constitution or\n statutes; (2) a public official exercises a portion of the\n sovereign power; and (3) a public official exercises discretion,\n while public employees perform ministerial duties.” Isenhour, 350\n N.C. at 610, 517 S.E.2d at 127. Additionally, “an officer is\n generally required to take an oath of office while an agent or\n employee is not required to do so.” Pigott v. City of Wilmington,\n 50 N.C. App. 401, 403-04, 273 S.E.2d 752, 754, cert. denied, 303\n N.C. 181, 280 S.E.2d 453 (1981).\n This Court has previously determined that the positions of an\n Environmental Health Specialist and an Environmental Health\n Supervisor are public employees because the positions are not\n created by statute and they do not exercise sovereign power;\n rather, their duties are ministerial. Block v. Cty. of Person, 141\n -9-\n N.C. App. 273, 281-82, 540 S.E.2d 415, 421-22 (2000). Although\n Block was an appeal from the denial of a motion to dismiss and the\n current case is an appeal from a partial denial of a motion for\n summary judgment, we find the reasoning in Block persuasive. See\n Northern Nat. Life Ins. Co. v. Lacy J. Miller Mach. Co., Inc., 311\n N.C. 62, 76, 316 S.E.2d 256, 265 (1984) (holding that the Court of\n Appeals was not bound by the doctrine of stare decisis because the\n procedural issues in the case were substantially different from\n those in a similar case).\n In Block, this Court stated:\n Although defendants cite a number of statutes\n contained in Chapter 130A (Public Health) of\n the North Carolina General Statutes, there is\n no statutory or constitutional scheme that\n creates the positions of Environmental Health\n Specialist or Environmental Health Supervisor\n for a county health department. Only the\n position of Director of a county health\n department is set forth by statute. Nor does\n it appear that defendants . . . exercise any\n sovereign power; rather, their duties are\n ministerial. Our courts have held that a\n supervisor of the Department of Social\n Services is a public employee. Similarly, a\n supervisor for the Health Department is a\n public employee, as is a specialist, who is a\n subordinate of the supervisor.\n 141 N.C. App. at 281-82, 540 S.E.2d at 421-22.\n Although Defendants argue that they were acting as Registered\n Sanitarians, a position created by statute, we agree with the\n reasoning in Block, that “there is no statutory or constitutional\n scheme that creates the positions of Environmental Health\n Specialist or Environmental Health Supervisor for a county health\n department.” Id.; see also Hare v. Butler, 99 N.C. App. 693, 700,\n -10-\n 394 S.E.2d 231, 236 (1990) (holding that three Department of Social\n Services positions - Protective Services Investigation Supervisor,\n Program Administrator for Child and Family Services, and Assistant\n Director - were public employees because their positions were not\n created by statute nor did they exercise any sovereign power).\n Additionally, there is no evidence in the record that Defendants\n took oaths of office. See Pigott, 50 N.C. App. at 403-04, 273\n S.E.2d at 754. Accordingly, this assignment of error is overruled.\n Affirmed.\n Judges BRYANT and JACKSON concur."""
err5 = """ST. REGIS OF ONSLOW COUNTY, NORTH CAROLINA OWNERS ASSOCIATION,\n INC., Plaintiff, v. WILLIAM C. JOHNSON, Defendant\n NO. COA07-1295\n Filed: 5 August 2008\n 1. Judicial Sales--defendant not within county–notice not required\n Defendant was not entitled to receive personal notice of the impending execution sale of\n a condominium on the North Carolina coast because he was not located in Onslow county, there\n was no evidence that he had an agent in North Carolina, and the Sheriff complied with the\n statutory requirement that notice be sent by registered mail.\n 2. Judicial Sales–judgment docketed--ownership transferred before sale–no personal\n notice–no due process violation\n A New Jersey family trust which received ownership of a condominium on the North\n Carolina coast after a judgement but before the execution sale was not entitled to personal notice,\n nor were its due process rights affected. The judgment had been docketed and the trust had\n record notice of the judgment lien.\n 3. Judicial Sales–notice via registered letter–additional steps impractical\n The notice of an impending execution sale complied with due process requirements\n where the Sheriff provided notice via registered letter and did not become aware that the normal\n procedures for providing notice were ineffective until after the sale had been finalized. It was\n not practicable for the Sheriff, without knowledge of the non-receipt, to take additional\n reasonable steps to notify defendant of the impending sale of the property.\n Appeal by Defendant from order entered 2 May 2007 by Judge\n Jack W. Jenkins in Onslow County Superior Court. Heard in the\n Court of Appeals 19 March 2008.\n Edwin L. West, III, for Defendant-Appellant.\n Ward and Smith, P.A., by J. Michael Fields and John P. Crolle,\n for Plaintiff-Appellee.\n STEPHENS, Judge.\n This case concerns the adequacy of the means employed by the\n Sheriff of Onslow County to provide notice to Defendant William C.\n Johnson of the pending execution sale of his condominium unit.\n -2-\n I. FACTS\n Defendant was the owner of condominium unit 2107 located in\n the St. Regis condominium complex in Onslow County, North Carolina.\n Plaintiff St. Regis of Onslow County, North Carolina Owners\n Association, Inc., filed a Claim of Lien against Defendant to\n enforce assessments due and owed to Plaintiff for homeowner’s dues\n for the condominium unit. Plaintiff subsequently filed a complaint\n against Defendant on 25 March 1999 to recover the delinquent\n assessments.\n On 6 March 2000, judgment was entered against Defendant upon\n Plaintiff’s motion for summary judgment. By this order and\n judgment, Plaintiff was entitled to recover from Defendant\n $10,063.66, plus court costs, attorney’s fees of $551.54, and\n interest at a rate of eight percent per annum from the date of the\n filing of the complaint. Plaintiff was also allowed to foreclose\n its lien on the property and to sell the property to satisfy\n Defendant’s debt to Plaintiff. Also on 6 March 2000, the judgment\n lien was docketed in Onslow County in Judgment Docket Book 87 on\n page 236.\n On 4 April 2003, approximately three years after the entry of\n judgment, Defendant deeded the subject real property to the Johnson\n Family Trust (“Trust”). The deed on its face requested that the\n Register of Deeds mail the recorded deed to Defendant at 39 Pitney\n Lane, Jackson, New Jersey 08527.\n On 5 January 2006, the Onslow County Clerk of Superior Court\n (“Clerk”) issued a Writ of Execution to the Sheriff of Onslow\n -3-\n County (“Sheriff”). On 30 January 2006, the Sheriff mailed, via\n registered mail, a letter and a Notice of Sale of Real Property\n Under an Execution (“Notice”) to Defendant at 39 Pitney Lane,\n Jackson, New Jersey 08527. The letter stated: “Under and by\n virtue of a Judgment rendered against Defendant in the referenced\n action, an execution was issued by the Court on the 5[th] day of\n January, 2006, and directed to the Sheriff of Onslow County.” The\n Notice further provided: “The sale will be held on the 6[th] day of\n March, 2006, at 11:30 o’clock a.m., at the Onslow County\n Courthouse.”\n On 27 January 2006, the Sheriff posted the Notice at the\n Onslow County Courthouse in the area designated by the Clerk for\n the posting of notices. On 3 February, and again on 9 February\n 2006, the United States Postal Service notified Defendant of the\n Sheriff’s registered mail envelope, but Defendant did not claim the\n envelope. On 20 February and 1 March 2006, the Sheriff published\n the Notice in the Jacksonville Daily News.\n On 6 March 2006, the Sheriff conducted the execution sale.\n Floyd B. McKissick, Jr., the President of the Plaintiff owners\n association, submitted the winning bid of $87,000. On 10 March\n 2006, Mr. McKissick paid the purchase price, and on 31 March 2006,\n a Sheriff’s Deed conveying the property to Mr. McKissick was\n recorded. On 27 April 2006, the registered mail envelope\n containing the letter and Notice was returned to the Sheriff marked\n “unclaimed.”\n -4-\n On 5 March 2007, Defendant, along with the Trust, trustees\n Karen Gillen and William M. Johnson, and trust beneficiaries\n William Charles Johnson, Jr., Christopher Michael Johnson, and\n Stacy Lynn Johnson (collectively “Movants”) filed a Motion to Set\n Aside Execution Sale, Order Confirming Execution Sale, and the\n Sheriff’s Deed Issued to the Execution Sale Purchaser pursuant to\n North Carolina Civil Procedure Rule 60. On 16 April 2007, Movants\n filed an Amended Motion.\n On 23 April 2007, a hearing was conducted on the motion, and\n an order denying the motion was entered on 2 May 2007. From this\n order, Defendant, joined by the Rule 60 Movants, appeals.\n II. DISCUSSION\n By Defendant’s nine assignments of error, he argues the trial\n court erred in denying his motion for relief pursuant to N.C. Gen.\n Stat. § 1A-1, Rule 60(b)(6) because the notice of the execution\n sale provided by the Sheriff did not meet due process requirements.\n Under Rule 60(b), the trial court may “relieve a party or his\n legal representative from a final judgment, order, or proceeding”\n for the reasons specified in Rule 60(b)(1) – (5). N.C. Gen. Stat.\n § 1A-1, Rule 60(b) (2005). Rule 60(b)(6) permits the trial court\n to grant relief for any other reason “justifying relief from the\n operation of the judgment.” Id. This provision “authorizes the\n trial judge to exercise his discretion in granting or withholding\n the relief sought.” Kennedy v. Starr, 62 N.C. App. 182, 186, 302\n S.E.2d 497, 499-500, disc. review denied, 309 N.C. 321, 307 S.E.2d\n 164 (1983).\n -5-\n On appeal, this Court’s review of a trial court’s Rule 60(b)\n ruling “is limited to determining whether the trial court abused\n its discretion.” Vaughn v. Vaughn, 99 N.C. App. 574, 575, 393\n S.E.2d 567, 568, disc. review denied, 327 N.C. 488, 397 S.E.2d 238\n (1990). An abuse of discretion is shown only when the court’s\n decision “is manifestly unsupported by reason or is so arbitrary\n that it could not have been the result of a reasoned decision.”\n State v. McDonald, 130 N.C. App. 263, 267, 502 S.E.2d 409, 413\n (1998) (citation omitted).\n The Due Process Clause of the Fifth Amendment to the United\n States Constitution, as applied to the states by the Fourteenth\n Amendment, and Article 1, section 19, of the North Carolina\n Constitution, prohibit the government from depriving any person of\n his or her property without due process of law. Due process does\n not require that a property owner receive actual notice before the\n government may take his property. Dusenbery v. United States, 534\n U.S. 161, 151 L. Ed. 2d 597 (2002). Rather, due process requires\n the government to provide “notice reasonably calculated, under all\n the circumstances, to apprise interested parties of the pendency of\n the action and afford them an opportunity to present their\n objections.” McLean v. McLean, 233 N.C. 139, 143, 63 S.E.2d 138,\n 146 (1951) (quoting Mullane v. Cent. Hanover Bank & Trust Co., 339\n U.S. 306, 314, 94 L. Ed. 865, 873 (1950)). “Whether a party has\n adequate notice is a question of law.” Trivette v. Trivette, 162\n N.C. App. 55, 58, 590 S.E.2d 298, 302 (2004).\n -6-\n [1] Defendant first argues that Movants were entitled to\n receive “personal notice” of the impending sale of the property.\n Specifically, Defendant argues that the language of N.C. Gen. Stat.\n § 1-339.54 makes it clear that Defendant, as the judgment debtor,\n was entitled to personal notice of the sale of the condominium\n unit. We disagree.\n Pursuant to N.C. Gen. Stat. § 1-339.54, the sheriff must\n comply with the following procedure for notifying a judgment debtor\n of the sale of real property:\n [T]he sheriff shall, at least ten days before\n the sale of real property,\n (1) If the judgment debtor is found in the\n county, serve a copy of the notice of sale on\n him personally, or\n (2) If the judgment debtor is not found in the\n county,\n a. Send a copy of the notice of sale by\n registered mail to the judgment debtor at\n his last address known to the sheriff,\n and\n b. Serve a copy of the notice of sale on\n the judgment debtor’s agent, if there is\n in the county a person known to the\n sheriff to be an agent who has custody or\n management of, or who exercises control\n over, any property in the county\n belonging to the judgment debtor.\n N.C. Gen. Stat. § 1-339.54 (2005). Additionally, the sheriff must\n comply with the following procedure for posting and publishing\n notice of an execution sale of real property:\n (a) The notice of sale of real property shall:\n (1) Be posted, in the area designated by\n the clerk of superior court for the\n posting of notices in the county in which\n -7-\n the property is situated, for at least 20\n days immediately preceding the sale; and\n (2) Be published once a week for at least\n two successive weeks:\n a. In a newspaper qualified for\n legal advertising published in the\n county; or\n b. If no newspaper qualified for\n legal advertising is published in\n the county, in a newspaper having\n general circulation in the county.\n (b) When the notice of sale is published in a\n newspaper:\n (1) The period from the date of the first\n publication to the date of the last\n publication, both dates inclusive, shall\n not be less than seven days, including\n Sundays; and\n (2) The date of the last publication\n shall be not more than 10 days preceding\n the date of the sale.\n N.C. Gen. Stat. § 1-339.52 (2005).\n As Defendant was not located in Onslow County, he was not\n entitled to have a copy of the Notice served on him personally\n pursuant to N.C. Gen. Stat. § 1-339.54(1). Furthermore, there was\n no evidence that Defendant had an agent in Onslow County upon whom\n the Sheriff was required to personally serve a copy of the Notice\n pursuant to N.C. Gen. Stat. § 1-339.54(2)b. Accordingly, here, at\n least ten days before the sale of the property, the Sheriff was\n required to “[s]end a copy of the [N]otice [] by registered mail to\n [Defendant] at his last address known to the [S]heriff[.]” N.C.\n Gen. Stat. § 1-339.54(2)a. The Sheriff complied with this mandate\n by sending the Notice via registered mail on 30 January 2006, more\n -8-\n than 30 days prior to the sale of the property on 6 March 2006, to\n 39 Pitney Lane, Jackson, New Jersey 08527, Defendant’s last address\n known to the Sheriff.\n [2] Defendant also alleges that the Trust, as the owner of the\n unit at the time of the sale, and the Trust beneficiaries, as\n parties with legally protected interests, were entitled to receive\n personal notice of the impending execution sale. Again, we\n disagree.\n In Scott v. Paisley, 271 U.S. 632, 70 L. Ed. 1123 (1926), the\n plaintiff purchased a tract of land that was subject to a priorrecorded\n security deed executed by the previous owner. After the\n plaintiff purchased the land, the previous owner defaulted on the\n note secured by the property. Without notice to the plaintiff, the\n creditor brought suit against the former owner of the land.\n Judgment was entered in the creditor’s favor and the sheriff\n subsequently executed on the land. After the required\n advertisement of the sale, the property was sold at public sale in\n satisfaction of the judgment. The plaintiff brought suit to set\n aside the sale because she was not provided with notice of the\n sale. In concluding that the validity of the sale was not\n affected, nor were the plaintiff’s due process rights violated, by\n the fact that notice of the sale was not given to the plaintiff,\n the United States Supreme Court stated:\n A purchaser of land on which there is a prior\n security deed acquires his interest in the\n property subject to the right of the holder of\n the secured debt to exercise the statutory\n power of sale. There is no established\n principle of law which entitles such a\n -9-\n purchaser to notice of the exercise of this\n power.\n Id. at 636, 70 L. Ed. at 1125.\n Here, the Warranty Deed transferring the condominium unit from\n Defendant to the Trust erroneously asserted that the property was\n “free and clear of all encumbrances[.]” In fact, the property was\n encumbered by a judgment lien which allowed Plaintiff to foreclose\n its lien and exercise its power of sale to satisfy Defendant’s\n debt. As in Scott, the Trust and the beneficiaries acquired their\n interests in the property subject to the right of Plaintiff to\n exercise its statutory power of sale. Furthermore, docketing a\n judgment puts third parties on notice of the existence of a\n judgment, and transferees are bound to look into the proper dockets\n to examine for judgment liens. Jones v. Currie, 190 N.C. 260, 129\n S.E. 605 (1925). While in Scott no judgment was docketed and the\n defendant’s power of sale did not arise until after the plaintiff\n had purchased the property, here, the judgment had been docketed on\n 6 May 2000 in Onslow County in Judgment Docket Book 87 on page 236.\n Thus, unlike the plaintiff in Scott who had no record notice of the\n defendant’s power of sale, here the Trust had record notice of the\n judgment lien allowing Plaintiff to exercise its power of sale to\n satisfy Defendant’s debt. Accordingly, as in Scott, the Trust and\n the beneficiaries were not entitled to personal notice of the\n execution sale and the validity of the sale of the condominium was\n not affected, nor were Movants’ due process rights violated, by the\n fact that such notice was not given to the Trust or the\n beneficiaries.\n -10-\n [3] Defendant next argues that N.C. Gen. Stat. § 1-339.54\n “run[s] afoul” of due process requirements in that, under the\n circumstances, the Sheriff was obligated to take “further\n reasonable steps” to notify Defendant of the impending property\n sale.\n As stated above, individuals whose property interests are at\n stake are entitled to “notice reasonably calculated, under all the\n circumstances, to apprise [them] of the pendency of the action.”\n Mullane, 339 U.S. at 314, 94 L. Ed. at 873. Whether a particular\n method of notice is reasonable depends on the particular\n circumstances and “[t]he means employed must be such as one\n desirous of actually informing the absentee might reasonably adopt\n to accomplish it.” Id. at 315, 94 L. Ed. at 874.\n Thus, the issue is whether the notice in this case was\n “reasonably calculated under all the circumstances” to apprise\n Defendant of the pendency of the execution sale. The Sheriff sent\n registered mail addressed to Defendant at Defendant’s last known\n and actual address. Additionally, the Sheriff posted the Notice at\n the Onslow County Courthouse in the area designated by the Clerk\n for the posting of notices, and published the Notice in the\n Jacksonville Daily News on two occasions.\n Although Defendant contends that “the attempt at providing\n notice solely by means of one attempt at sending notice via\n registered mail was constitutionally inadequate to accord with due\n process requirements under the law[,]” use of the postal service to\n send a letter to a party is well-recognized as an adequate means of\n -11-\n 1 E.g., Mullane, 339 U.S. at 319, 94 L. Ed. at 876 (“[T]he\n mails today are recognized as an efficient and inexpensive means of\n communication.”); Walker v. City of Hutchinson, 352 U.S. 112, 116,\n 1 L. Ed. 2d 178, 182 (1956) (“[T]he notice by publication here\n falls short of the requirements of due process. . . . Even a\n letter would have apprised [appellant] that his property was about\n to be taken . . . .”); Schroeder v. City of New York, 371 U.S. 208,\n 214, 9 L. Ed. 2d 255, 260 (1962) (“[T]he city was constitutionally\n obliged to make at least a good faith effort to give [notice]\n personally to the appellant – an obligation which the mailing of a\n single letter would have discharged.”); Mennonite Bd. of Missions\n v. Adams, 462 U.S. 791, 798, 77 L. Ed. 2d 180, 187 (1983) (“When\n the mortgagee is identified in a mortgage that is publicly\n recorded, constructive notice by publication must be supplemented\n by notice mailed to the mortgagee’s last known available address,\n or by personal service.”).\n effecting notice upon known addressees when notice by publication\n has been found to be insufficient.1 Dusenbery, 534 U.S. 161, 151\n L. Ed. 2d 597. Defendant further claims that “Jones v. Flowers []\n makes it clear that, if nothing else, one charged with providing\n notice as to the impending loss of another’s property must send one\n last notice, via regular United States mail, if the provider cannot\n figure out a better way of providing personal notice.” We are of\n the opinion, however, that Defendant’s interpretation and reliance\n on Jones v. Flowers, 547 U.S. 220, 164 L. Ed. 2d 415 (2006), is\n misplaced.\n In Flowers, the Arkansas Commissioner of State Lands\n (“Commissioner”) mailed a certified letter to the defendant to\n notify him of his tax delinquency. The letter stated that unless\n the defendant redeemed the property, the property would be subject\n to public sale two years later. No one was home to sign for the\n letter and nobody retrieved the letter from the post office within\n the next fifteen days. The post office returned the unopened\n -12-\n letter to the Commissioner marked “unclaimed.” Id. at 224, 164 L.\n Ed. 2d at 424.\n Two years later, the Commissioner published a notice of public\n sale in the newspaper several weeks before the sale. No bids were\n submitted, permitting the State to negotiate a private sale of the\n property. Several months later, Linda Flowers submitted a purchase\n offer. The Commissioner then mailed another certified letter to\n the defendant, attempting to notify him that his house would be\n sold to Flowers if he did not redeem the property. Like the first\n letter, the second was also returned to the Commissioner marked\n “unclaimed.” Id. The property was subsequently sold to Flowers.\n The defendant filed suit against the Commissioner and Flowers,\n alleging that the Commissioner’s failure to provide notice of the\n tax sale and of the defendant’s right to redeem his property\n resulted in the taking of his property without due process. The\n United States Supreme Court held that “when mailed notice of a tax\n sale is returned unclaimed, the State must take additional\n reasonable steps to attempt to provide notice to the property owner\n before selling his property, if it is practicable to do so.” Id.\n at 225, 164 L. Ed. 2d at 425. The Court reasoned that “despite the\n fact that the letters were reasonably calculated to reach their\n intended recipients when delivered to the postman[,]” id. at 229,\n 164 L. Ed. 2d at 428, when the notice provider becomes aware that\n normal procedures are ineffective in providing notice, this\n triggers an obligation “to take additional steps to effect notice.”\n Id. at 230, 164 L. Ed. 2d at 428. The Court then determined that\n -13-\n “[u]nder the circumstances presented [], additional reasonable\n steps were available to the State.” Id. at 225, 164 L. Ed. 2d at\n 425.\n As in Flowers, the Sheriff in this case sent notice of the\n pending property sale to Defendant through the United States Postal\n Service via registered letter. However, unlike in Flowers where\n the unclaimed letters were returned before the sale of the property\n at issue, triggering the state’s obligation “to take additional\n steps to effect notice” before the sale of the property, id., in\n this case, the unclaimed letter was returned almost two months\n after the execution sale of the condominium, and one month after\n the Sheriff’s Deed was recorded. Thus, similar to Dusenbery and\n Mullane where the government heard nothing back indicating that\n their attempts at notice had gone awry, the Sheriff did not become\n aware that the normal procedures for providing notice were\n ineffective until after the sale had been finalized. Here, as in\n Flowers, the letter was reasonably calculated to reach Defendant\n when it was delivered to the postman, but, unlike Flowers, under\n the circumstances presented in this case, it was not practicable\n for the Sheriff, without knowledge of the non-receipt, to take\n additional reasonable steps to notify Defendant of the impending\n sale of the property.\n Accordingly, the notice provided to Defendant complied with\n due process requirements. The use of the mail addressed to\n Defendant at his last known and actual address was clearly\n acceptable for much the same reason the United States Supreme Court\n -14-\n and the North Carolina Appellate Courts have approved mailed notice\n in the past. Dusenbery, 534 U.S. 161, 151 L. Ed. 2d 597. E.g.,\n Henderson Cty. v. Osteen, 292 N.C. 692, 708, 235 S.E.2d 166, 176\n (1977) (“[N]otice of the execution sale . . . sent by registered or\n certified mail to the listing taxpayer at his last known\n address . . ., in conjunction with the posting and publication also\n required by the statute, would, in our opinion, be sufficient to\n satisfy the fundamental concept of due process of law . . . .”);\n Hardy v. Moore Cty., 133 N.C. App. 321, 515 S.E.2d 84 (1999)\n (concluding that due process was satisfied where notice of a tax\n foreclosure sale of plaintiff’s property was mailed to plaintiff’s\n last known address and published in the local newspaper); Myers v.\n H. McBride Realty, Inc., 93 N.C. App. 689, 379 S.E.2d 70 (1989)\n (concluding that due process requirements were complied with where\n the sheriff sent notice of an execution sale via certified mail to\n the plaintiff’s address listed on the execution notice and to an\n address where plaintiff owned real property).\n We conclude that the Sheriff’s actions were “reasonably\n calculated, under all the circumstances, to apprise [Defendant] of\n the action.” Mullane, 339 U.S. at 314, 94 L. Ed. at 873. Due\n process requires no more.\n The order of the trial court denying Defendant’s Rule 60(b)\n motion is\n AFFIRMED.\n Judges McGEE and TYSON concur."""
err6 = """EULA P. STREET, Plaintiff v. BASIL W. STREET, Defendant\n NO. COA07-1452\n Filed: 5 August 2008\n 1. Appeal and Error--appealability--denial of summary judgment--final judgment on\n merits\n Plaintiff’s arguments in a divorce case regarding the order denying her claim for\n summary judgment were not considered because the Court cannot consider an appeal of the\n denial of a summary judgment motion once a final judgment on the merits has been made.\n 2. Divorce--equitable distribution--property contract--affirmative defense catchall\n provision\n The trial court erred in a divorce case by disregarding a property contract on the basis\n that it must have been pled as an affirmative defense under the catchall provision in N.C.G.S. §\n 8C-1, Rule 8(c) because: (1) the contract governing property is not an affirmative defense, but\n instead was a piece of evidence to be considered in settling the action for equitable distribution;\n (2) no counterclaim was brought by defendant to which an affirmative defense would need to be\n made; (3) it would not have been appropriate for plaintiff to specifically plead an affirmative\n defense when defendant neither disputed the property contract nor brought any new claim of his\n own; and (4) the trial court cited to neither statutory nor case law to support its holding, and the\n Court of Appeals found none.\n 3. Divorce--equitable distribution--validity of property contract--findings of fact\n A divorce case is remanded to the trial court so that the validity of the property contract\n and any other evidence as to equitable distribution may be considered because: (1) the trial court\n made no findings of fact in any of its orders as to the validity of the property contract; and (2) the\n record reflected no evidence on the question.\n Appeal by plaintiff from orders entered 26 September 2005, 23\n August 2006, and 22 August 2007 by Judge Mike Gentry in Person\n County District Court. Heard in the Court of Appeals 14 May 2008.\n Manning, Fulton & Skinner, P.A., by Michael S. Harrell, for\n plaintiff-appellant.\n No brief for defendant-appellee.\n HUNTER, Judge.\n -2-\n Eula P. Street (“plaintiff”) appeals from three orders\n pertaining to her divorce from Basil W. Street (“defendant”).\n After careful review, we reverse and grant a new trial.\n Plaintiff and defendant married in 1985. In 1989, the parties\n entered into a contract (“the property contract”) stating that each\n party “release[d], renounce[d], and quitclaim[ed] all interest in\n any real property hereafter acquired by [other party].” The\n parties separated in 1997 and were divorced in 1999. Equitable\n distribution proceedings were then instigated.\n In 2005, plaintiff moved for summary judgment on her claim for\n equitable distribution. That motion was denied on 26 September\n 2005; that order is one of the three from which plaintiff appeals.\n On 23 August 2006, the court filed an order holding that the\n property contract constituted an “affirmative defense” and\n therefore “should have been specifically pled by Plaintiff.”\n Because it was not, the trial court classified the parties’ former\n home as marital property for purposes of equitable distribution.\n This is the second order from which plaintiff appeals.\n A final equitable distribution order was filed on 22 August\n 2007. Therein, plaintiff was ordered to pay defendant $14,500.00\n -- half the total equity the couple had in the marital home -- “for\n his interest in the marital property of the parties[.]” This is\n the final order from which plaintiff appeals.\n [1] As to plaintiff’s arguments regarding the order denying\n plaintiff’s claim for summary judgment, we note only that “[t]his\n Court cannot consider an appeal of denial of the summary judgment\n -3-\n motion now that a final judgment on the merits has been made[.]”\n WRI/Raleigh, L.P. v. Shaikh, 183 N.C. App. 249, 252, 644 S.E.2d\n 245, 246 (2007). We therefore do not consider plaintiff’s\n arguments as to that order.\n As to the other two orders, plaintiff’s argument in essence is\n that the court erred in (1) holding that the property contract must\n have been pled as an affirmative defense and (2) refusing to give\n the property contract effect on that basis. Because we agree with\n the first, we do not address the second.\n [2] In its August 2006 order holding that the property\n contract must have been pled as an affirmative defense, the court\n states that the contract was “improperly and untimely filed”\n because it was “an affirmative defense within the meaning of Rule\n 8(c) of the North Carolina Rules of Civil Procedure such that it\n should have been specifically pled by Plaintiff.” [R p. 51] Rule\n 8(c) lists twenty-one specific affirmative defenses that must be\n pled, none of which apply in this case. N.C. Gen. Stat. § 1A-1,\n Rule 8(c) (2007). Following this list is the following catch-all:\n “[A]ny other matter constituting an avoidance or affirmative\n defense.” Id. There are two errors in the trial court’s\n conclusion that this catch-all applies to the property contract at\n issue in this case. First, the contract governing property is not\n an affirmative defense; rather, it is a piece of evidence to be\n considered in settling the action for equitable distribution.\n Second, and more important, no counterclaim was brought by\n defendant to which an affirmative defense would need to be made.\n -4-\n Plaintiff brought an action for equitable distribution, and\n defendant filed an answer admitting almost all of plaintiff’s\n allegations and asking only for visitation and “such orders of\n equitable distribution as is [sic] appropriate.” He neither\n disputed the property contract nor brought any new claim of his\n own. As such, it would not have been appropriate for plaintiff to\n specifically plead an affirmative defense.\n This appears to be the only basis on which the trial court\n disregarded the property contract -- that is, that the property\n contract was not timely filed because it was an affirmative\n defense. The trial court cited only to Rule 8(c) in support of\n this holding, and as discussed above, that rule does not apply.\n The trial court cited to neither statutory nor case law to support\n the holding, and this Court has found none. Thus, the property\n contract was improperly disregarded by the trial court.\n [3] The next question, then, is whether the property contract\n is valid. We cannot make such a determination on the record before\n us; the trial court made no findings of fact in any of its orders\n as to the validity of the property contract, and the record\n reflects no evidence on the question. As such, we remand this case\n to the trial court so that the validity of the property contract --\n and any other evidence as to equitable distribution -- may be\n considered.\n Reversed; new trial.\n Judges STEELMAN and STEPHENS concur."""
err7 = """EAGLE ENGINEERING, INC., Plaintiff, v. CONTINENTAL CASUALTY\n COMPANY, Defendant\n NO. COA07-1537\n Filed: 5 August 2008\n Insurance–professional liability--claims made and reported policy–summary judgment\n The trial court correctly affirmed summary judgment for defendant insurance company\n on plaintiff’s claim under a professional liability policy where plaintiff did not make its claim\n within the required 60 days of the policy period. The parties had a plain, unambiguous contract\n which required that the claim arise during a covered policy period and be made within the policy\n period or 60 days afterwards.\n Appeal by plaintiff from order entered 30 October 2007 by\n Judge Kimberly S. Taylor in Superior Court, Union County. Heard in\n the Court of Appeals 15 May 2008.\n James, McElroy & Diehl, P.A. by Richard B. Fennell, for\n plaintiff-appellant.\n Jones, Hewson & Woolard by Lawrence J. Goldman, for defendantappellee.\n STROUD, Judge.\n Plaintiff appeals from order entered 30 October 2007 which\n granted defendant’s motion for summary judgment. The dispositive\n question before this Court is whether the trial court erred in\n granting defendant’s motion for summary judgment. For the\n following reasons, we affirm.\n I. Background\n On 27 February 2007, plaintiff Eagle Engineering, Inc.\n (“Eagle”) filed a complaint against defendant Continental Casualty\n Company (“Continental”). Plaintiff alleged the following in the\n complaint:\n -2-\n 7. Eagle purchased from Defendant a\n Professional Liability and Pollution Incident\n Liability Insurance Policy (“the Coverage\n Agreement”) with policy number 11-405-03-06.\n The policy period for the Coverage Agreement\n was December 1, 2001, through December 1,\n 2004.\n 8. The Coverage Agreement provides, in\n pertinent part:\n A. We will pay all amounts in excess of\n the deductible up to the limit of liability\n that you become legally obligated to pay as a\n result of:\n 1. a wrongful act; or\n 2. a pollution incident arising\n out of your activities or the activities of\n any person or entity for whom you are liable,\n that results in a claim anywhere in the world,\n provided that on the knowledge date set forth\n on the Declarations no officer, director,\n principal, partner or insurance manager knew\n or could reasonably have expected that a claim\n would be made.\n B. A claim arising out of a wrongful\n act or pollution incident must be first made\n during the policy year or any applicable\n extended reporting period. A claim is\n considered first made when you receive notice\n of the claim or as set forth in accordance\n with Section VI. CONDITIONS, Item C., Your\n Rights and Duties in the Event of a\n Circumstance.\n 9. The Coverage Agreement also includes the\n following descriptive language of the type of\n coverage provided:\n Your professional liability and pollution\n incident liability insurance policy is written\n on a “claims-made” basis and applies only to\n those claims first made against you while this\n insurance is in force. No coverage exists for\n claims first made against you after the end of\n the policy term unless, and to the extent, an\n extended reporting period applies.\n 10. Shea Homes, LLC (“Shea”), is a\n residential home builder that does business\n throughout North Carolina.\n -3-\n 11. Shea filed counterclaims against Eagle in\n Union County Superior Court Case No. 03-CVS-\n 02057 on March 8, 2004, which were amended\n August 9, 2004. The gravamen of Shea’s\n counterclaims was that Eagle had improperly\n performed professional services in a Shea\n development and that this had led to property\n damage.\n 12. Eagle purchased similar insurance\n coverage from Defendant for the period from\n January 4, 2006, through January 4, 2007. The\n coverage language is identical to that set\n forth in the Coverage Agreement. Eagle was\n insured with a different carrier for the\n intervening period.\n 13. Eagle listed Shea’s counterclaims as a\n pending lawsuit in its application for the\n 2006 policy year. Defendant accepted the\n business, and Eagle’s premium.\n 14. Eagle began to ask Defendant to assume\n the defense and indemnification obligations\n surrounding Shea’s counterclaims as early as\n November, 2005.\n 15. Defendant refused, contending that the\n Coverage Agreement required notice to be\n received during a policy year in order to\n trigger coverage.\n 16. Eagle resolved Shea’s counterclaims on\n the eve of trial after failing to convince\n Defendant to indemnify against the\n counterclaims or even pick up the defense.\n Plaintiff brought a cause of action against defendant for breach of\n contract.\n On or about 9 May 2007, defendant filed an answer. On or\n about 17 August 2007, defendant filed a motion for summary\n judgment. An affidavit from James F. Alderson (“Alderson”), a\n claims consultant with Continental, referred to Exhibits A and B,\n Policy Declarations. Exhibit A is “[a] true and accurate copy of\n [the 11-405-03-06] policy” which ran from 1 December 2001 through\n -4-\n 1 December 2004. Exhibit B is “[a] true and accurate copy of [the\n 27-620-29-33] policy” which ran from 4 January 2006 through 4\n January 2007. Within Exhibit B is a “Conditions” section. This\n section provides,\n B. Your Duties If There Is A Claim\n If there is a claim, you must do the\n following:\n 1. promptly notify us in writing.\n . . .\n The notice must be given to us within a\n policy year or within 60 days after its\n expiration or termination[.]\n . . . .\n N. Extended Reporting Period\n 1. Automatic Extended Reporting Period\n If this Policy is canceled or nonrenewed\n by either us or by the first Named\n Insured, we will provide an automatic, noncancelable\n extended reporting period starting\n at the termination of the policy term if the\n first Named Insured has not obtained similar\n coverage. This automatic extended reporting\n period will terminate after 60 days.\n 2. Optional Extended Reporting Period\n If this Policy is canceled or nonrenewed\n by either us or by the first Named\n Insured, then the first Named Insured shall\n have the right to purchase an optional\n extended reporting period. Such right must be\n exercised by the first Named Insured within 60\n days of the termination of the policy term[.]\n . . . .\n 4. Extended Reporting Period\n Limitations\n No additional or optional extended\n reporting period shall apply to:\n a. any claim or proceedings\n pending at the inception date of such extended\n reporting period[.]\n The affidavit further provided,\n -5-\n 5. Continental Casualty Company did not\n provide insurance to Eagle Engineering, Inc.\n from January 4, 2005 through January 4, 2006.\n . . . .\n 9. From the information provided to the\n Continental Casualty Company by Eagle\n Engineering, Inc., the date of the claim for\n which Eagle Engineering seeks indemnification\n and a defense in this litigation was August 9,\n 2004. Notice of this claim was not submitted\n to CNA and Continental Casualty Company until\n October 5, 2006.\n On 20 September 2007, S. Stephen Goodwin, Jr., trial counsel\n for Eagle Engineering in the case with Shea, filed an affidavit\n which stated, “Any delays on Eagles part were either inadvertent or\n the result of difficulty obtaining information.” On this same date\n Frank L. “Skeet” Gray, III, P.E., a principle [sic] with plaintiff,\n also filed an affidavit stating, “[T]here was absolutely no\n purposeful intentional or deliberate decision by Eagle Engineering\n to delay notification to the Defendant.”\n On 30 October 2007, the trial court granted defendant’s motion\n for summary judgment because “there is no genuine issue as to any\n material fact that plaintiff did not provide notice of the claim\n within the policy period or within 60 days after the expiration of\n the policy period.” Plaintiff appeals. The dispositive question\n before this Court is whether the trial court erred in granting\n defendant’s motion for summary judgment. For the following\n reasons, we affirm.\n II. Summary Judgment\n Plaintiff argues that the trial court erred by granting\n summary judgment in favor of defendant. Plaintiff contends that\n -6-\n Continental’s position centers on its\n contention that the Policy at issue is a\n “claims made,” rather than an occurrence\n policy. Continental’s duties, then, depend on\n the definition of “claims made.” A claims\n made policy generally restricts coverage to\n those claims which are made against the\n insured during the policy period. . . . A pure\n claims-made policy allows a claim to be\n presented after the policy period has expired.\n . . . A “claims made and reported” policy, on\n the other hand, engrafts a second requirement\n on to the definition of a covered claim. It\n must be both made to the insured and reported\n to the carrier within the policy period.\n In Digh v. Nationwide Mut. Fire Ins. Co. this Court stated,\n Summary judgment is appropriate if the\n pleadings, depositions, answers to\n interrogatories, and admissions on file,\n together with the affidavits, if any, show\n that there is no genuine issue as to any\n material fact and that a party is entitled to\n a judgment as a matter of law. On appeal of a\n trial court's allowance of a motion for\n summary judgment, we consider whether, on the\n basis of materials supplied to the trial\n court, there was a genuine issue of material\n fact and whether the moving party is entitled\n to judgment as a matter of law. Evidence\n presented by the parties is viewed in the\n light most favorable to the non-movant.\n We begin by noting that insurance\n policies are considered contracts between two\n parties. It is the duty of the court to\n construe an insurance policy as it is written,\n not to rewrite it and thus make a new contract\n for the parties. Insurance contracts are\n construed according to the intent of the\n parties, and in the absence of ambiguity, we\n construe them by the plain, ordinary and\n accepted meaning of the language used.\n Digh v. Nationwide Mut. Fire Ins. Co., 187 N.C. App. 725, 727-28,\n 654 S.E.2d 37, 39 (2007) (internal citations, internal quotation\n marks, brackets, and heading omitted). Furthermore,\n a mere disagreement between the parties over\n the language of the insurance contract does\n -7-\n not create an ambiguity. Rather, no ambiguity\n exists unless, in the opinion of the court,\n the language of the policy is fairly and\n reasonably susceptible to either of the\n constructions for which the parties contend.\n Also, each provision of an insurance contract\n must be interpreted in view of the whole\n contract and not in isolation.\n Pennsylvania Nat.’l Mut. Ins. Co. v. Strickland, 178 N.C. App. 547,\n 550, 631 S.E.2d 845, 847 (2006) (internal citations, internal\n quotation marks, ellipses, and brackets omitted).\n Both Exhibit A, policy term of 1 December 2001 through 1\n December 2004 (“hereinafter “Policy 1"), and Exhibit B, policy term\n of 4 January 2006 through 4 January 2007, (hereinafter “Policy 2")\n refer to the attached “Professional Liability Policy.” The\n Professional Liability Policy provides in bold capitalized font at\n the top of the policy,\n Your professional liability and pollution\n incident liability insurance policy is written\n on a “claims-made” basis and applies only to\n those claims first made against you while this\n insurance is in force. No coverage exists for\n claims first made against you after the end of\n the policy term unless, and to the extent, an\n extended reporting period applies.\n The Professional Liability Policy further provides that notice of\n a claim “must be given . . . within a policy year or within 60 days\n after its expiration or termination.” Thus, the Professional\n Liability Policy requires plaintiff’s claim to have both arisen\n during a covered policy term and to be reported within a covered\n policy term or within 60 days thereafter.\n The exact date defendant was informed of plaintiff’s claim is\n in contention, but was somewhere between November 2005 and October\n -8-\n 2006. Plaintiff, however, contends, defendant was informed of the\n claim in November of 2005, and we will view the evidence in a light\n most favorable to plaintiff, the non-movant. See Dighat 727-28,\n 654 S.E.2d at 39.\n A. Policy 1\n As to Policy 1, neither November 2005 nor October 2006 falls\n within the policy term of 1 December 2001 through 1 December 2004\n nor within 60 days thereafter. Therefore, pursuant to the plain\n language of the policy, Policy 1 does not provide coverage for\n plaintiff’s claim.\n B. Policy 2\n As to Policy 2, Shea filed its counterclaims against plaintiff\n on 8 March 2004 and amended those claims 9 August 2004; 8 March\n 2004 and 9 August 2004 are not dates during which the 4 January\n 2006 through 4 January 2007 policy would have been “in force.”\n Therefore, Policy 2 does not provide coverage for plaintiff’s\n claim.\n C. Policies 1 and 2\n Viewing the contracts as a whole it is plain that plaintiff’s\n claim must have arisen during a covered policy period and plaintiff\n was required to make its claim within a covered policy period or 60\n days thereafter, which plaintiff failed to do. See Digh at 727-28,\n 654 S.E.2d at 39; Pennsylvania Nat.’l Mut. Ins. Co. at 550, 631\n S.E.2d at 847 (2006). There is no ambiguity within the policy nor\n any merit to plaintiff’s argument, and therefore this argument is\n overruled.\n -9-\n D. Prejudice\n Furthermore, plaintiff contends that there are “genuine issues\n of material fact . . . as to whether defendant was prejudiced by\n the timing of the notice in this case.” However, prejudice is\n irrelevant in light of the fact that the parties have a plain\n unambiguous contract setting out the terms of the agreement on this\n issue. See Digh at 727-28, 654 S.E.2d at 39; Pennsylvania Nat.’l\n Mut. Ins. Co. at 550, 631 S.E.2d at 847 (2006). This argument is\n without merit.\n III. Conclusion\n For the foregoing reasons, we affirm the trial court’s\n granting of defendant’s motion for summary judgment.\n AFFIRMED.\n Judges McCULLOUGH and TYSON concur."""
err8 = """JOSEPH CALVIN WILSON, Plaintiff, v. BARBARA BILTCLIFFE WILSON,\n Defendant\n NO. COA07-1524\n Filed: 5 August 2008\n 1. Civil Procedure--summary judgment hearing--notice\n The trial court did not err in a divorce case by concluding defendant wife received\n adequate and proper notice of the summary judgment hearing, even though defendant contends\n the notice of hearing only stated the date and not the time of the hearing, because: (1) defendant\n failed to show that she did not receive notice of hearing on plaintiff’s motion for summary\n judgment at least ten days prior to the hearing as required by N.C.G.S. § 1A-1, Rule 56(c); and\n (2) plaintiff’s notice of hearing was adequate in light of N.C.G.S. § 1A-1, Rule 56(c).\n 2. Divorce--absolute--subject matter jurisdiction--personal jurisdiction--proper notice\n and service\n The district court did not lack subject matter jurisdiction and personal jurisdiction even\n though defendant wife contends she was not properly served with the summons and complaint\n prior to the trial court’s entry of absolute divorce because: (1) in regard to subject matter\n jurisdiction, the court found that plaintiff had been a citizen and resident of North Carolina for\n more than six months next preceding the institution of this action, and plaintiff and defendant\n have lived separate and apart for more than one year without resuming the marital relationship;\n (2) in regard to personal jurisdiction, plaintiff filed an affidavit of service by certified mail on 7\n June 2007; (3) plaintiff’s verified complaint contained allegations consistent with the trial\n court’s order and was properly treated as an affidavit; and (4) competent evidence supported the\n court’s unchallenged findings of fact.\n Appeal by defendant from judgment entered 23 August 2007 by\n Judge Ben S. Thalheimer in Mecklenburg County District Court.\n Heard in the Court of Appeals 11 June 2008.\n No brief filed by plaintiff.\n Barbara Biltcliffe Wilson, pro-se, for defendant-appellant.\n TYSON, Judge.\n Barbara Biltcliffe Wilson (“defendant”) appeals from judgment\n entered, which granted Joseph Calvin Wilson (“plaintiff”) an\n absolute divorce from defendant. We affirm.\n -2-\n I. Background\n Plaintiff and defendant were married on or about 14 June 1964\n and separated on 30 July 2001. On 13 April 2006, plaintiff filed\n a verified complaint in which he sought “the bonds of matrimony\n heretofore existing between [p]laintiff and [d]efendant be\n dissolved, and that [p]laintiff and [d]efendant be granted an\n absolute divorce from each other.” Plaintiff failed to achieve\n service of process on defendant after the issuance of summonses on\n 13 April 2006, 24 May 2006, 11 September 2006, 5 December 2006, 28\n February 2007, and 24 April 2007.\n On 1 August 2007, plaintiff alleged service of process was\n accomplished on 22 May 2007 and moved for summary judgment. The\n hearing for summary judgment was held 20 August 2007. The district\n court granted plaintiff an absolute divorce from defendant and\n filed its judgment on 23 August 2007. Defendant appeals.\n II. Issues\n Defendant argues the district court erred when it entered its\n judgment because: (1) defendant was not given proper notice of the\n summary judgment hearing and (2) the trial court lacked\n jurisdiction.\n III. Standard of Review\n Summary judgment is appropriate if the\n pleadings, depositions, answers to\n interrogatories, and admissions on file,\n together with the affidavits, if any, show\n that there is no genuine issue as to any\n material fact and that a party is entitled to\n a judgment as a matter of law. On appeal of a\n trial court’s allowance of a motion for\n summary judgment, we consider whether, on the\n basis of materials supplied to the trial\n -3-\n court, there was a genuine issue of material\n fact and whether the moving party is entitled\n to judgment as a matter of law. Evidence\n presented by the parties is viewed in the\n light most favorable to the non-movant.\n Summey v. Barker, 357 N.C. 492, 496, 586 S.E.2d 247, 249 (2003)\n (internal citation and quotation omitted).\n “We review an order allowing summary judgment de novo. If the\n granting of summary judgment can be sustained on any grounds, it\n should be affirmed on appeal.” Wilkins v. Safran, 185 N.C. App.\n 668, 672, 649 S.E.2d 658, 661 (2007) (internal citation and\n quotation omitted).\n IV. Notice\n [1] Defendant asserts she did not receive adequate and proper\n notice of the summary judgment hearing because the notice of\n hearing only stated the date and not the time of the hearing. We\n disagree.\n Motions for summary judgment are governed by Rule 56 of the\n North Carolina Rules of Civil Procedure. N.C. Gen. Stat. § 1A-1,\n Rule 56 (2007). “The motion shall be served at least 10 days\n before the time fixed for the hearing.” N.C. Gen. Stat. § 1A-1,\n Rule 56(c). “Although Rule 56 makes no direct reference to notice\n of hearing, this Court has held that such notice also must be given\n at least ten (10) days prior to the hearing.” Barnett v. King, 134\n N.C. App. 348, 350, 517 S.E.2d 397, 399 (1999) (citing Calhoun v.\n Wayne Dennis Heating & Air Cond., 129 N.C. App. 794, 800, 501\n S.E.2d 346, 350 (1998), disc. rev. denied, 350 N.C. 92, 532 S.E.2d\n 524 (1999)).\n -4-\n Here, plaintiff filed his motion for summary judgment and\n notice of hearing on 1 August 2007. The notice of hearing states\n “that on the 20[th] day of Aug[ust], 2007 the [p]laintiff will\n request the Judge presiding in Courtroom No. 8110 of the Civil\n Courts Building to grant the relief requested in [p]laintiff’s\n Motion for Summary Judgment, namely entry of a Judgment of Absolute\n Divorce.” Attached to both the motion and notice were certificates\n of service signed by plaintiff’s counsel on 31 July 2007.\n Defendant has failed to show that she did not receive notice\n of hearing on plaintiff’s motion for summary judgment “at least ten\n (10) days prior to the hearing.” Barnett, 134 N.C. App. at 350,\n 517 S.E.2d at 399. Plaintiff’s notice of hearing was adequate and\n proper in light of Rule 56(c) of the North Carolina Rules of Civil\n Procedure. This assignment of error is overruled.\n V. Service of Process\n [2] Defendant argues the district court lacked subject matter\n and personal jurisdiction because she was not properly served with\n the summons and complaint prior to the trial court’s entry of\n absolute divorce. We disagree.\n A. Subject Matter Jurisdiction\n “The district court division is the proper division without\n regard to the amount in controversy, for the trial of civil actions\n and proceedings for . . . divorce . . . .” N.C. Gen. Stat. §\n 7A-244 (2005). “In North Carolina, subject matter jurisdiction for\n divorce involves not only bringing the matter in the correct court,\n but also the court’s finding residence by one of the parties for\n -5-\n the requisite length of time and verification of the pleadings.”\n 2 Suzanne Reynolds, Lee’s North Carolina Family Law § 7.25, at 88\n (5th ed. 1999) (citations omitted); see also N.C. Gen. Stat. §§\n 50-6, -8 (2005).\n Here, plaintiff filed a verified complaint with the\n Mecklenburg County District Court. The district court found: (1)\n “[p]laintiff has been a citizen and resident of the State of North\n Carolina for more than six months next preceding the institution of\n this action[]” and (2) “[p]laintiff and [d]efendant have lived\n separate and apart for more than one year next preceding the\n institution of this action without resuming the marital\n relationship.” The district court’s findings are supported by\n plaintiff’s verified complaint, which may be treated as an\n affidavit. See Page v. Sloan, 281 N.C. 697, 705, 190 S.E.2d 189,\n 194 (1972) (“A verified complaint may be treated as an affidavit if\n it (1) is made on personal knowledge, (2) sets forth such facts as\n would be admissible in evidence, and (3) shows affirmatively that\n the affiant is competent to testify to the matters stated therein.”\n Citations omitted)).\n The district court properly exercised jurisdiction over the\n subject matter of the parties’ divorce action. N.C. Gen. Stat. §§\n 7A-244, 50-6, 50-8. This assignment of error is overruled.\n B. Personal Jurisdiction\n Defendant asserts the district court did not acquire personal\n jurisdiction over her. We disagree.\n In any action commenced in a court of this\n State having jurisdiction of the subject\n -6-\n matter and grounds for personal jurisdiction\n as provided in G.S. 1-75.4, the manner of\n service of process within or without the State\n shall be as follows:\n (1) Natural Person. - Except as provided in\n subsection (2) below, upon a natural person by\n one of the following:\n . . . .\n c. By mailing a copy of the summons and of the\n complaint, registered or certified mail,\n return receipt requested, addressed to the\n party to be served, and delivering to the\n addressee.\n N.C. Gen. Stat. § 1A-1, Rule 4(j) (2007).\n Where the defendant appears in the action and\n challenges the service of the summons upon\n him, proof of the service of process shall be\n as follows:\n . . . .\n (4) Service by Registered or Certified Mail. -\n In the case of service by registered or\n certified mail, by affidavit of the serving\n party averring:\n a. That a copy of the summons and complaint\n was deposited in the post office for mailing\n by registered or certified mail, return\n receipt requested;\n b. That it was in fact received as evidenced\n by the attached registry receipt or other\n evidence satisfactory to the court of delivery\n to the addressee; and\n c. That the genuine receipt or other evidence\n of delivery is attached.\n N.C. Gen. Stat. § 1-75.10 (2007) (emphasis supplied).\n Here, plaintiff filed an affidavit of service by certified\n mail on 7 June 2007. The affidavit stated that “copies of a Civil\n Summons and Complaint were deposited in the U.S. Post Office for\n -7-\n mailing by certified mail, return receipt requested, addressed to\n [defendant] . . . .” Attached to plaintiff’s affidavit was: (1)\n a receipt for certified mail which showed item number 7160 3901\n 9848 8335 2054 was mailed on 3 May 2007 and (2) a track/confirm\n sheet from the U.S. Post Office which indicated item number 7160\n 3901 9848 8335 2054 was delivered to and signed for by defendant on\n 22 May 2007 at 11:36 a.m.\n It is clear from plaintiff’s affidavit of service by certified\n mail and attachments thereto with defendant’s signature appearing\n thereon that the summons and complaint were personally served upon\n defendant pursuant to Rule 4(j)(1)(c) of the North Carolina Rules\n of Civil Procedure.\n Our holding is also consistent with other jurisdictions, which\n have held service to be sufficient when reviewing facts similar to\n those at bar. See In re Estate of Riley, 847 N.E.2d 22, 27 (Ohio\n Ct. App. 2006) (“A signed return receipt constitutes evidence of\n delivery pursuant to Civ.R. 4.1(A), but the rule does not bar\n introduction of other evidence to establish certified mail\n delivery.” (Citation omitted)); see also Lauer v. City of New\n York, 656 N.Y.S.2d 93 (1997); compare Connally v. Connally, 233\n S.W.3d 168, 171-72 (Ark. App. 2006) (“We see no evidence in the\n record that [the defendant] signed for the package or that the\n [third party] was [the defendant’s] authorized agent, and, during\n oral argument, [the plaintiff’s] counsel could not direct us to any\n such evidence. We therefore agree with the trial court that [the\n defendant] was not served by commercial delivery.”).\n -8-\n The district court properly exercised personal jurisdiction\n over defendant. This assignment of error is overruled.\n VI. Sufficiency of the Evidence\n N.C. Gen. Stat. § 50-6 states:\n Marriages may be dissolved and the parties\n thereto divorced from the bonds of matrimony\n on the application of either party, if and\n when the husband and wife have lived separate\n and apart for one year, and the plaintiff or\n defendant in the suit for divorce has resided\n in the State for a period of six months.\n In its order granting plaintiff an absolute divorce from\n defendant, the district court found: (1) “[p]laintiff has been a\n citizen and resident of the State of North Carolina for more than\n six months next preceding the institution of this action[]” and (2)\n “[p]laintiff and [d]efendant have lived separate and apart for more\n than one year next preceding the institution of this action without\n resuming the marital relationship.” These findings are supported\n by allegations asserted in plaintiff’s verified complaint.\n “A verified complaint may be treated as an affidavit if it (1)\n is made on personal knowledge, (2) sets forth such facts as would\n be admissible in evidence, and (3) shows affirmatively that the\n affiant is competent to testify to the matters stated therein.”\n Page, 281 N.C. at 705, 190 S.E.2d at 194 (citations omitted).\n Here, plaintiff’s verified complaint meets the elements articulated\n by our Supreme Court in Page, contains allegations consistent with\n the trial court’s order and was properly treated as an affidavit.\n 281 N.C. at 705, 190 S.E.2d at 194.\n -9-\n Further, defendant has not assigned any error to these\n findings. “Where no exception is taken to a finding of fact by the\n trial court, the finding is presumed to be supported by competent\n evidence and is binding on appeal.” Koufman v. Koufman, 330 N.C.\n 93, 97, 408 S.E.2d 729, 731 (1991) (citations omitted).\n Uncontradicted and competent evidence supports the district court’s\n findings of fact, which in turn support its conclusion to grant\n plaintiff an absolute divorce from defendant.\n VII. Conclusion\n Defendant received adequate and proper notice of the hearing\n on plaintiff’s motion for summary judgment “at least ten (10) days\n prior to the hearing.” Barnett, 134 N.C. App. at 350, 517 S.E.2d\n at 399; N.C. Gen. Stat. § 1A-1, Rule 56(c).\n The district court of Mecklenburg County properly exercised\n subject matter and personal jurisdiction over the parties’ divorce\n action and defendant, respectively. Competent evidence supports\n the district court’s unchallenged findings of fact. The district\n court found and concluded plaintiff had met all statutory\n requirements and properly granted plaintiff an absolute divorce\n from defendant. The judgment appealed from is affirmed.\n Affirmed.\n Judges HUNTER and JACKSON concur."""
err9 = """ELM ST. GALLERY, INC., WILLIAM B. HEROY, ANNA R. HEROY,\n Individually and d/b/a HEROY STUDIOS and OLD PHOTO SPECIALISTS,\n INC., Plaintiffs, v. ROBERT M. WILLIAMS and SHELIA V. WILLIAMS,\n Defendants\n NO. COA08-10\n Filed: 5 August 2008\n 1. Negligence--fire damage--causation--mere conjecture, surmise, and speculation--\n summary judgment\n The trial court did not err in a negligence case arising from fire damage by granting\n defendants’ motion for summary judgment on the issue of the cause of the fire because: (1)\n plaintiffs’ assertion that the evidence pointed to an electrical fire originating from the right rear\n of defendants’ building was a mere conjecture, surmise, and speculation as to the cause of the\n fire; (2) the record was devoid of any evidence tending to support plaintiffs’ assertion when an\n inspector and two other experts were unable to determine the origin of the fire; and (3) plaintiffs\n failed to establish any inference that the alleged negligence by defendants was the actual or\n proximate cause of their injury.\n 2. Negligence--fire damage--proximate cause--delay taking corrective action to remedy\n condition--summary judgment\n The trial court did not err in a negligence case arising from fire damage by granting\n defendants’ motion for summary judgment on the issue of whether defendants negligently\n delayed taking corrective action to remedy the condition of their building after the fire occurred\n because: (1) assuming arguendo that plaintiffs could establish a duty and breach thereof,\n plaintiffs failed to produce any evidence tending to show or raise any inference that defendants’\n alleged negligence was the proximate cause of plaintiffs’ injury; (2) speculation or mere\n conjecture would be required to determine whether the damage to plaintiffs’ building resulted\n from defendants’ delay in the demolition, plaintiffs’ repairs to their building, or from some other\n source; and (3) plaintiffs’ unsubstantiated and contradictory allegations were insufficient to\n establish any inference that defendants’ alleged negligence was the actual or proximate cause of\n plaintiffs’ injury.\n Appeal by plaintiff from order entered 14 August 2007 by Judge\n John O. Craig, III in Guilford County Superior Court. Heard in the\n Court of Appeals 21 May 2008.\n Butler & Quinn, P.L.L.C., by W. Rob Heroy, for plaintiffappellants.\n Smith, James, Rowlett & Cohen, L.L.P., by Norman B. Smith, for\n defendant-appellees.\n TYSON, Judge.\n -2-\n Elm St. Gallery, Inc., William and Anna Heroy, individually\n and d/b/a Heroy Studios and Old Photo Specialists, Inc.\n (collectively, “plaintiffs”) appeal from an order entered granting\n Robert and Shelia Williams’ (collectively, “defendants”) motion for\n summary judgment. We affirm.\n I. Background\n Elm St. Gallery, Inc. is the owner of property located at 320\n South Elm Street, Greensboro, North Carolina (“plaintiffs’\n building”). Defendants formerly owned property located at 324\n South Elm Street (“defendants’ building”), which adjoined\n plaintiffs’ building by a shared party wall. On 24 October 2003,\n a fire occurred in defendants’ unoccupied building.\n At the time of the fire, William and Anna Heroy owned a\n photography business located on the first floor of plaintiffs’\n building. The second and third floors of plaintiffs’ building were\n rented as residential apartments. As a result of the fire,\n plaintiffs’ building sustained damage and the photography business\n and residential tenants were required to vacate the premises.\n Plaintiffs commenced repairs and renovations to the party wall\n and to their building. Plaintiffs demanded defendants demolish the\n remnants of their building to facilitate plaintiffs’ repairs.\n Defendants allegedly expressed an intent to comply with plaintiffs’\n requests, but failed to do so.\n Due to defendants’ inaction in demolishing the remnants of\n their building, the City of Greensboro issued a demolition order.\n -3-\n On 21 February 2005, defendants sold their property to a third\n party, who subsequently demolished the building.\n On 13 June 2006, plaintiffs filed an unverified complaint and\n alleged defendants had negligently maintained their building in\n such a condition that caused or contributed to the start and spread\n of the fire. On 16 June 2006, plaintiffs’ filed an amended\n complaint and further alleged defendants negligently delayed taking\n corrective action to remedy the condition of their building after\n the fire. Defendants filed an answer, denied all of plaintiffs’\n allegations, and raised the affirmative defenses of: (1)\n contributory negligence; (2) res judicata; and (3) failure to\n mitigate damages.\n On 26 April 2007, defendants moved for summary judgment on all\n of plaintiffs’ claims. Plaintiffs also filed a motion for summary\n judgment regarding defendants’ counterclaim. However, no\n counterclaim was asserted in defendants’ original answer and no\n amended answer is included as part of the record on appeal.\n The trial court granted plaintiffs’ motion for summary\n judgment regarding defendants’ counterclaim. The trial court also\n granted defendants’ motion for summary judgment on all of\n plaintiffs’ claims contained in their amended complaint.\n Plaintiffs appeal.\n II. Issues\n Plaintiffs argue genuine issues of material fact exist and the\n trial court erred by granting defendants’ motion for summary\n judgment.\n -4-\n III. Summary Judgment\n Plaintiffs argue the trial court erred by granting defendants’\n motion for summary judgment because genuine issues of material fact\n exist regarding: (1) the cause of the fire and (2) whether\n defendants negligently delayed taking corrective action to remedy\n the condition of their building after the fire occurred. We\n disagree.\n A. Standard of Review\n Summary judgment is proper if the pleadings,\n depositions, answers to interrogatories, and\n admissions on file, together with the\n affidavits, if any, show that there is no\n genuine issue as to any material fact and that\n any party is entitled to a judgment as a\n matter of law. The party moving for summary\n judgment ultimately has the burden of\n establishing the lack of any triable issue of\n fact.\n A defendant may show entitlement to summary\n judgment by (1) proving that an essential\n element of the plaintiff’s case is\n non-existent, or (2) showing through discovery\n that the plaintiff cannot produce evidence to\n support an essential element of his of her\n claim, or (3) showing that the plaintiff\n cannot surmount an affirmative defense.\n Summary judgment is not appropriate where\n matters of credibility and determining the\n weight of the evidence exist.\n Once the party seeking summary judgment makes\n the required showing, the burden shifts to the\n nonmoving party to produce a forecast of\n evidence demonstrating specific facts, as\n opposed to allegations, showing that he can at\n least establish a prima facie case at trial.\n We review an order allowing summary judgment\n de novo. If the granting of summary judgment\n can be sustained on any grounds, it should be\n affirmed on appeal.\n -5-\n Wilkins v. Safran, 185 N.C. App. 668, 672, 649 S.E.2d 658, 661\n (2007) (internal citations and quotations omitted).\n B. Analysis\n Our Supreme Court has “emphasized that summary judgment is a\n drastic measure, and it should be used with caution[,]” especially\n in negligence cases in which a jury ordinarily applies a reasonable\n person standard to the facts of each case. Williams v. Power &\n Light Co., 296 N.C. 400, 402, 250 S.E.2d 255, 257 (1979) (citation\n omitted). Summary judgment has been held to be proper in\n negligence cases “where the evidence fails to show negligence on\n the part of defendant, or where contributory negligence on the part\n of plaintiff is established, or where it is established that the\n purported negligence of defendant was not the proximate cause of\n plaintiff’s injury.” Hale v. Power Co., 40 N.C. App. 202, 203, 252\n S.E.2d 265, 267 (citation omitted), cert. denied, 297 N.C. 452, 256\n S.E.2d 805 (1979).\n 1. Causation\n [1] Our Supreme Court has addressed the issue of causation in\n several cases that involve negligence actions arising from fire\n damage. See Snow v. Power Co., 297 N.C. 591, 256 S.E.2d 227\n (1979); Phelps v. Winston-Salem, 272 N.C. 24, 157 S.E.2d 719\n (1967); Maharias v. Storage Co., 257 N.C. 767, 127 S.E.2d 548\n (1962). In both Maharias and Phelps, our Supreme Court affirmed\n judgments, which dismissed the plaintiffs’ negligence claims based\n upon a lack of evidence tending to show a causal link between the\n defendants’ alleged negligence and the origin of the fire. 257\n -6-\n N.C. at 767-68, 127 S.E.2d at 549; 272 N.C. at 24, 157 S.E.2d at\n 719.\n In Maharias, a fire originated at the defendant’s warehouse\n and caused significant damage to the plaintiff’s adjacent\n restaurant and the contents within. 257 N.C. at 767, 127 S.E.2d at\n 549. An Assistant Fire Chief inspected the defendant’s building\n and opined that it was possible the fire had been caused by\n spontaneous combustion of a pile of furniture-polishing rags, but\n that “[the] fire could have happened from any one of a number of\n causes.” Id. Our Supreme Court held that non-suit entered at the\n close of the plaintiff’s evidence was proper because “[t]he\n evidence raised a mere conjecture, surmise and speculation as to\n the cause of the fire.” Id. at 768, 127 S.E.2d at 549. Our\n Supreme Court further stated, “[a] cause of action must be based on\n something more than a guess.” Id.\n In Phelps, the plaintiffs were tenants in a building owned and\n operated by the City of Winston-Salem. 272 N.C. at 25, 157 S.E.2d\n at 720. A fire originated in the defendant’s building and\n destroyed a substantial amount of the plaintiffs’ belongings. Id.\n at 26, 157 S.E.2d at 720. The plaintiffs alleged the defendant had\n negligently allowed combustible materials to accumulate in the\n building and had failed to provide fire safety equipment. Id. at\n 26, 157 S.E.2d at 721. The fire chief and the captain in charge of\n the Fire Prevention Bureau both testified they were unable to\n determine the cause of the fire. Id. at 27, 157 S.E.2d at 721.\n -7-\n Applying the reasoning in Maharias, our Supreme Court held that the\n cause of the fire was “unexplained” and stated:\n [p]roof of the burning alone is not sufficient\n to establish liability, for if nothing more\n appears, the presumption is that the fire was\n the result of accident or some providential\n cause. There can be no liability without\n satisfactory proof, by either direct or\n circumstantial evidence, not only of the\n burning of the property in question but that\n it was the proximate result of negligence and\n did not result from natural or accidental\n causes.\n Id. at 31, 157 S.E.2d at 724 (citation omitted). Our Supreme Court\n further stated that the plaintiff bears the burden to\n “affirmatively fix [responsibility] upon the defendant by the\n greater weight of the evidence.” Id. at 28, 157 S.E.2d at 722.\n Several years later in Snow, our Supreme Court limited the\n holdings in Maharias and Phelps to the particular facts presented\n in both cases and acknowledged that the circumstances in Snow\n presented a “very different factual context.” 297 N.C. at 598, 256\n S.E.2d at 232. In Snow, the plaintiffs filed a negligence action\n against Duke Power Company and alleged a fire originated at a\n faulty electrical meter attached to a barn. Id. at 597, 256 S.E.2d\n at 232. The plaintiffs presented evidence tending to show:\n (1) that the fire originated at a point where\n the wiring connecting the weatherhead to the\n meter box was “hot” with electrical current;\n (2) that the initially compact and\n concentrated nature of the flames was\n consistent with an electrical fire; [and] (3)\n that the fire took some time to spread from\n the front of the barn -- where the “hot wires”\n were located -- to the back of the barn.\n -8-\n Id. at 598, 256 S.E.2d at 232. The plaintiffs also presented\n evidence which tended to negate the likelihood of other causes of\n the fire. Id.\n Our Supreme Court affirmed the trial court’s denial of the\n defendant’s motion for a directed verdict on the theory of res ipsa\n loquitur and stated “[i]f the facts proven establish the more\n reasonable probability that the fire was electrical in origin, then\n the case cannot be withdrawn from the jury though all possible\n causes have not been eliminated.” Id. at 597, 256 S.E.2d at 232.\n Plaintiffs argue that Snow is “controlling.” We disagree and\n find the factual backgrounds and analyses presented in Maharias and\n Phelps to be directly on point with the facts at bar.\n Here, Greensboro Fire Department Inspector Myron E. Kenan\n (“Inspector Kenan”) arrived at the scene of the fire and observed\n the building fully engulfed in flames. After the fire was\n extinguished, the building was deemed unsafe to enter. An\n investigation could not be immediately completed to determine the\n cause or origin of the fire. A month later, on 25 November 2003,\n Inspector Kenan returned to the site and conducted his\n investigation. Inspector Kenan discovered a portion of the second\n floor was severely damaged and opined that the fire had originated\n in the right rear corner of that floor. Further investigation\n revealed “three generations of electrical wiring design” within the\n building.\n Inspector Kenan photographed and examined all of the\n electrical panels located in the vicinity of the damaged portion of\n -9-\n the second floor. Inspector Kenan’s report specifically stated\n that he “did not find any prevalent indications of an electrical\n cause of the fire. However, with the extent of fire damage [he]\n [could not] determine that this fire was not electrical in nature.”\n Another member of the investigation team, “who ha[d] extensive\n knowledge of electrical service and equipment[,]” agreed with\n Inspector Kenan’s findings. The fire investigation report further\n stated, “In addition to not being able to deduct all possible\n accidental causes[,] I cannot make a determination that this fire\n was or was not incendiary in nature.” The cause of the fire was\n ultimately listed as “undetermined.”\n Although our Supreme Court has acknowledged that “the origin\n of a fire may be established by circumstantial evidence[,]” it has\n also stated, “[w]hether the circumstantial evidence is sufficient\n to take the case out of the realm of conjecture and into the field\n of legitimate inference from established facts, must be determined\n in relation to the attendant facts and circumstances of each case.”\n Snow, 297 N.C. at 597, 256 S.E.2d at 232 (citations and quotations\n omitted). Here, plaintiffs’ assertion that the evidence points to\n “an electrical fire originating from the right rear of\n [d]efendants’ building” is “a mere conjecture, surmise and\n speculation as to the cause of the fire.” Maharias, 257 N.C. at\n 768, 127 S.E.2d at 549.\n The record is completely devoid of any evidence tending to\n support plaintiffs’ assertion. Inspector Kenan and two other\n experts were unable to determine the origin of the fire.\n -10-\n Plaintiffs’ unsubstantiated and self-serving allegation that\n immediately prior to the fire, defendants’ rear gutters could have\n allowed water to come into contact with electrical wiring is\n insufficient to submit the issue of defendants’ negligence to the\n jury. See Phelps, 272 N.C. at 31, 157 S.E.2d at 724 (“In order to\n go to the jury on the question of defendant’s negligence causing\n the fire, plaintiffs must not only show that the fire might have\n been started due to the defendant’s negligence, but must show by\n reasonable affirmative evidence that it did so originate.”\n (Emphasis original)).\n Plaintiffs have failed to establish any inference that the\n alleged negligence by defendants was the actual or proximate cause\n of their injury. Because “[a] cause of action must be based on\n something more than a guess[,]” the trial court properly granted\n defendants’ motion for summary judgment regarding this issue.\n Maharias, 257 N.C. at 768, 127 S.E.2d at 549. This assignment of\n error is overruled.\n 2. Corrective Action\n [2] Plaintiffs also argue genuine issues of material fact\n exist regarding whether defendants negligently delayed taking\n corrective action to remedy the condition of their building after\n the fire occurred. Plaintiffs assert “water was seeping into\n Plaintiffs’ property on account of Defendants’ failure to demolish\n or repair what remained of their building[,]” which impeded repairs\n to the first floor of plaintiffs’ building and caused a loss of\n potential rental income.\n -11-\n “In order to prevail in a negligence action, plaintiffs must\n offer evidence of the essential elements of negligence: duty,\n breach of duty, proximate cause, and damages.” Camalier v.\n Jeffries, 340 N.C. 699, 706, 460 S.E.2d 133, 136 (1995) (citation\n omitted). Presuming arguendo, that plaintiffs could establish a\n duty and breach thereof, plaintiffs have once again failed to\n produce any evidence tending to show or to raise any inference that\n defendants’ alleged negligence was the proximate cause of\n plaintiffs’ injury.\n It is a well established principle that “all damages must flow\n directly and naturally from the wrong, and that they must be\n certain both in their nature and in respect to the cause from which\n they proceed.” People’s Center, Inc. v. Anderson, 32 N.C. App.\n 746, 748, 233 S.E.2d 694, 696 (1977) (citation omitted) (emphasis\n supplied). “[N]o recovery is allowed when resort to speculation or\n conjecture is necessary to determine whether the damage resulted\n from the unlawful act of which complaint is made or from some other\n source.” Id. at 748-49, 233 S.E.2d at 696 (citation omitted).\n During William Heroy’s deposition he testified, “[a]ccording\n [to] the inspector, the water intrusion from [defendants’] existing\n building there penetrated the walls top to bottom.” Plaintiffs’\n attorney also wrote a letter to defendants that asserted the\n condition of defendants’ building after the fire was “allowing\n water runoff onto the property at 320 S. Elm Street[.]”\n Plaintiffs have presented no evidence to support these\n allegations. The record on appeal contains no sworn affidavit from\n -12-\n plaintiffs’ supposed inspector nor any inspection report. The\n record is also completely devoid of any other evidence that tends\n to establish that defendants’ delay in demolishing and/or repairing\n their building caused the moisture problems of which plaintiffs now\n complain.\n Further, William Heroy, perhaps unknowingly, contradicted his\n earlier allegations by testifying that at the time of his\n deposition, the “water intrusion” was continuing to impact\n reconstruction even after the total demolition of defendants’\n former building was completed. Viewing the evidence in the light\n most favorable to plaintiffs, the trial court properly granted\n defendants’ motion for summary judgment because speculation or\n conjecture would be required to determine whether the damage to\n plaintiffs’ building resulted from defendants’ delay in the\n demolition, plaintiffs’ repairs to their building, or from some\n other source. People’s Center, Inc., 32 N.C. App. at 748-49, 233\n S.E.2d at 696.\n Plaintiffs’ unsubstantiated and contradictory allegations are\n insufficient to establish any inference that defendants’ alleged\n negligence was the actual or proximate cause of plaintiffs’ injury.\n The trial court properly entered summary judgment in favor of\n defendants. This assignment of error is overruled.\n IV. Conclusion\n Plaintiffs failed to establish defendants’ purported\n negligence, before or after the fire, provided a causal connection\n to plaintiffs’ alleged damages. The trial court properly granted\n -13-\n defendants’ motion for summary judgment. The trial court’s order\n is affirmed.\n Affirmed.\n Judges HUNTER and JACKSON concur."""
err10 = """IN THE SUPREME COURT OF NORTH CAROLINA\n No. 385PA13\n DOUGLAS KIRK LUNSFORD\n v.\n THOMAS E. MILLS, JAMES W. CROWDER, III, and SHAWN T. BUCHANAN\n On discretionary review pursuant to N.C.G.S. § 7A-31 of a unanimous\n decision of the Court of Appeals, ___ N.C. App. ___, 747 S.E.2d 390 (2013),\n affirming an order of summary judgment entered on 13 November 2012 by Judge\n James U. Downs in Superior Court, McDowell County. Heard in the Supreme\n Court on 15 April 2014.\n Abrams & Abrams, P.A., by Noah B. Abrams, Douglas B. Abrams, Margaret\n S. Abrams, and Melissa N. Abrams, for plaintiff-appellee.\n Nelson Levine de Luca & Hamilton, by David L. Brown, Brady A. Yntema,\n and David G. Harris, II, for unnamed defendant-appellant North Carolina\n Farm Bureau Mutual Insurance Company.\n White & Stradley, PLLC, by J. David Stradley; and Whitley Law Firm, by\n Ann C. Ochsner, for North Carolina Advocates for Justice, amicus curiae.\n Sparkman Larcade, PLLC, by George L. Simpson, IV, for North Carolina\n Association of Defense Attorneys and Property Casualty Insurers Association\n of America, amici curiae.\n BEASLEY, Justice.\n The primary issue in this appeal is whether an insured may, in a situation in\n which there is more than one at-fault driver responsible for the accident causing the\n insured’s injuries, recover under his or her underinsured motorist (UIM) policy\n LUNSFORD V. MILLS\n Opinion of the Court\n -2-\n before exhausting the liability insurance policies of all the at-fault drivers. We\n conclude that the insured is only required to exhaust the liability insurance\n coverage of a single at-fault motorist in order to trigger the insurer’s obligation to\n provide UIM benefits. Accordingly, we affirm the Court of Appeals’ decision on this\n issue. Because, however, the trial court’s award of interest and costs against the\n insurer in this case exceeds the amount the insurer contractually promised to pay\n under the terms of its policy with the insured, the Court of Appeals erred in\n upholding that portion of the award. In this respect, we reverse the Court of\n Appeals.\n Facts\n The parties to this appeal have stipulated to the material facts, which tend to\n establish that on 18 September 2009, defendant Thomas E. Mills was operating a\n tractor-trailer owned by his employer, defendant James W. Crowder, III. Mills was\n traveling eastbound on Interstate Highway 40 in McDowell County when he lost\n control while rounding a curve, causing his vehicle to collide with the concrete\n median barrier and flip. Plaintiff Douglas Kirk Lunsford, a volunteer firefighter,\n responded first to the scene and found that Mills was injured and that diesel fuel\n was leaking from the tractor-trailer. Lunsford, who was standing in the highway\n median, attempted to lift Mills over the concrete divider so that he could carry Mills\n to safety and assess his injuries. As Lunsford was doing so, defendant Shawn T.\n Buchanan was driving westbound on Interstate Highway 40. When the vehicle in\n LUNSFORD V. MILLS\n Opinion of the Court\n -3-\n front of Buchanan slowed down because of the tractor-trailer accident, Buchanan\n swerved to the left to avoid the vehicle and struck Lunsford. Lunsford was dragged\n underneath Buchanan’s car and suffered severe injuries, including multiple broken\n bones, lacerations, and internal injuries.\n At the time of the accidents, Mills and Crowder were insured through\n Crowder’s business motor vehicle policy with United States Fire Insurance\n Company (U.S. Fire), which provided a liability coverage limit of $1 million. The\n second driver, Buchanan, was insured under a policy written by Allstate Insurance\n Company (Allstate), providing liability coverage of $50,000. Lunsford maintained\n two policies with unnamed defendant North Carolina Farm Bureau Mutual\n Insurance Company (Farm Bureau): (1) a business policy with UIM coverage of\n $300,000; and (2) a personal policy with UIM coverage of $100,000.\n Lunsford subsequently filed a negligence action against Mills, Crowder, and\n Buchanan (named defendants), claiming that they were jointly and severally liable\n for his injuries. All named defendants filed answers, which included crossclaims for\n indemnification and contribution. Farm Bureau, as an unnamed defendant, also\n filed an answer in which it claimed that it would be entitled to an offset with\n respect to Lunsford’s UIM policies for any damages he recovered through the\n insurance policies held by the named defendants.\n LUNSFORD V. MILLS\n Opinion of the Court\n -4-\n On 24 May 2011, Allstate tendered to Lunsford the $50,000 liability coverage\n limit for Buchanan’s policy. Lunsford’s attorney notified Farm Bureau the next day\n of Allstate’s tender and demanded that Farm Bureau tender payment on Lunsford’s\n UIM claim. In a letter dated 7 June 2011, Farm Bureau indicated that (1) it would\n not advance the liability policy limits tendered to Lunsford by Allstate; and (2) it\n would review its legal options regarding Lunsford’s UIM claim and respond “at a\n later date.” On 15 November 2011, Lunsford’s attorney informed Farm Bureau that\n Lunsford had tentatively settled his claims against Mills and Crowder for $850,000,\n which was to be paid through Crowder’s policy with U.S. Fire. At the time of these\n settlements, Farm Bureau had not provided UIM coverage to Lunsford.\n On 12 January 2012, the trial court entered an order approving the\n settlement agreements. On 19 July 2012, Farm Bureau filed a motion for summary\n judgment on Lunsford’s UIM claim, arguing that he was not entitled to UIM\n coverage because the total amount of his settlements with Buchanan, Mills, and\n Crowder ($50,000 + $850,000 = $900,000) exceeded the aggregate amount of\n Lunsford’s UIM policies ($300,000 + $100,000 = $400,000). Lunsford also moved for\n summary judgment, maintaining that his UIM policies stacked and that he was\n entitled to recover $350,000 from Farm Bureau—the amount of his aggregated UIM\n coverage limit ($400,000) minus the $50,000 he recovered through his settlement\n with Buchanan.\n LUNSFORD V. MILLS\n Opinion of the Court\n -5-\n After conducting a hearing on the parties’ motions, the trial court entered an\n order on 13 November 2012 granting summary judgment in favor of Lunsford. The\n trial court accordingly ordered Farm Bureau to pay Lunsford $350,000, plus costs\n and pre- and post-judgment interest “as provided by law.”\n Farm Bureau appealed the trial court’s order to the Court of Appeals,\n primarily arguing that the trial court erred in granting summary judgment in favor\n of Lunsford and ordering Farm Bureau to pay $350,000 in UIM coverage because,\n under the statute governing UIM coverage, Farm Bureau “was not required to\n provide coverage until all applicable policies—meaning all policies held by all the\n named Defendants—had been exhausted.” Lunsford v. Mills, ___ N.C. App. ___,\n ___, 747 S.E.2d 390, 393 (2013). The court disagreed based on its reading of the\n UIM statute: “ ‘Underinsured motorist coverage is deemed to apply when, by\n reason of payment of judgment or settlement, all liability bonds or insurance\n policies providing coverage for bodily injury caused by the ownership, maintenance,\n or use of the underinsured highway vehicle have been exhausted.’ ” Id. at ___, 747\n S.E.2d at 393 (quoting N.C.G.S. § 20-279.21(b)(4) (emphasis added by court)). The\n court interpreted this language “to mean that UIM coverage is triggered the\n moment that an insured has recovered under all policies applicable to ‘a’—meaning\n one—‘underinsured highway vehicle’ involved in a motor vehicle accident resulting\n in injury to the insured.” Id. at ___, 747 S.E.2d at 393 (emphasis added).\n LUNSFORD V. MILLS\n Opinion of the Court\n -6-\n Noting that the issue of when UIM coverage is triggered in situations\n involving multiple potential at-fault drivers is one of first impression in North\n Carolina, the Court of Appeals believed that its interpretation of the UIM statute\n was consistent with that court’s precedent suggesting that “insureds should [not] ‘be\n kept hanging in limbo as they are forced to sue any and all possible persons . . .\n before they could recover UIM benefits’ just because other potential tortfeasors also\n happen to be covered under automobile policies.” Id. at ___, 747 S.E.2d at 394\n (quoting Farm Bureau Ins. Co. of N.C. v. Blong, 159 N.C. App. 365, 373, 583 S.E.2d\n 307, 312, disc. rev. denied, 357 N.C. 578, 589 S.E.2d 125 (2003)). In light of this\n rationale, the court determined that, in such a situation, UIM carriers are obligated\n “to first provide coverage, and later seek an offset through reimbursement or\n exercise of subrogation rights.” Id. at ___, 747 S.E.2d at 394. Consequently, the\n court determined that upon the exhaustion of “all policies applicable to Mr.\n Buchanan’s vehicle,” Lunsford’s “UIM coverage was triggered,” and “Farm Bureau\n was not at liberty to withhold coverage until [Lunsford] reached settlement\n agreements with Mr. Mills and Mr. Crowder.” Id. at ___, 747 S.E.2d at 394.\n Farm Bureau alternatively argued that, even it were required to provide UIM\n coverage, the trial court nevertheless erred in ordering it to pay pre- and postjudgment\n interest and costs. In support of this contention, Farm Bureau cited our\n decision in Sproles v. Greene, 329 N.C. 603, 613, 407 S.E.2d 497, 503 (1991), in\n which we concluded that North Carolina’s compulsory motor vehicle insurance laws\n LUNSFORD V. MILLS\n Opinion of the Court\n -7-\n do not impose an obligation on liability insurers to pay interest on a judgment in\n excess of the insurer’s policy limits, but rather, such an obligation “is governed by\n the terms of the [insurance] policy.” The Court of Appeals believed that Sproles was\n distinguishable on the ground that Sproles held that a “UIM carrier is not required\n to pay pre and post-judgment interest on behalf of the insured where the judgment\n has been entered against the insured.” Lunsford, ___ N.C. App. at ___, 747 S.E.2d\n at 395 (citing Sproles, 329 N.C. at 605, 407 S.E.2d at 498). Here, in contrast, “the\n judgment was entered against Farm Bureau itself, not against its insured\n (Plaintiff).” Id. at ___, 747 S.E.2d at 395. Thus the court concluded that Sproles\n “ha[d] no bearing on the case at hand” and upheld the trial court’s award of interest\n and costs. Id. at ___, 747 S.E.2d at 395 (2013).\n Farm Bureau petitioned this Court for discretionary review of the Court of\n Appeals’ decision regarding both the UIM coverage and the judgment interest\n issues. We allowed Farm Bureau’s petition with respect to both questions. 367\n N.C. 259, 749 S.E.2d 843 (2013).\n Standard of Review\n Under Rule 56(c) of the North Carolina Rules of Civil Procedure, summary\n judgment is appropriate when the record establishes that there are no genuine\n issues of material fact and that any party is entitled to judgment as a matter of law.\n N.C. R. Civ. P. 56(c); e.g., In re Will of Jones, 362 N.C. 569, 573, 669 S.E.2d 572, 576\n (2008). Here the parties have stipulated to the material facts, and therefore, the\n LUNSFORD V. MILLS\n Opinion of the Court\n -8-\n only question for our consideration is whether either party is entitled to judgment\n as a matter of law. Answering this question primarily involves interpretation of the\n Motor Vehicle Safety and Financial Responsibility Act of 1953 (commonly referred\n to as the “FRA”), N.C.G.S. §§ 20-279.1 through -279.39 (2013), and examination of\n the terms of Farm Bureau’s motor vehicle insurance policy, each a question of law.\n See Brown v. Flowe, 349 N.C. 520, 523, 507 S.E.2d 894, 896 (1998) (“A question of\n statutory interpretation is ultimately a question of law for the courts.”); Wachovia\n Bank & Trust v. Westchester Fire Ins. Co., 276 N.C. 348, 354, 172 S.E.2d 518, 522\n (1970) (observing that the interpretation of “the language used in [a] policy of\n insurance” is “a question of law”). This Court reviews questions of law de novo,\n meaning that we consider the matter anew and freely substitute our judgment for\n the judgment of the lower court. In re Greens of Pine Glen Ltd. P’ship, 356 N.C.\n 642, 647, 576 S.E.2d 316, 319 (2003) (citation omitted).\n Underinsured Motorist Coverage\n The parties’ principal dispute centers on the proper interpretation of\n subdivision 20-279.21(b)(4), the FRA’s provision governing UIM coverage. The\n primary objective of statutory interpretation is to ascertain and effectuate the\n intent of the legislature. Burgess v. Your House of Raleigh, Inc., 326 N.C. 205, 209,\n 388 S.E.2d 134, 137 (1990). “If the language of the statute is clear and is not\n ambiguous, we must conclude that the legislature intended the statute to be\n implemented according to the plain meaning of its terms.” Hyler v. GTE Prods. Co.,\n LUNSFORD V. MILLS\n Opinion of the Court\n -9-\n 333 N.C. 258, 262, 425 S.E.2d 698, 701 (1993) (citations omitted), superseded in part\n by statute, Workers’ Compensation Reform Act of 1994, ch. 679, sec. 2.5, 1993 N.C.\n Sess. Laws 394, 399-400, as recognized in N.C. Ins. Guar. Ass’n v. Bd. of Trs., 364\n N.C. 102, 691 S.E.2d 694 (2010). Thus, in effectuating legislative intent, it is our\n duty to give effect to the words actually used in a statute and not to delete words\n used or to insert words not used. N.C. Dep’t of Corr. v. N.C. Med. Bd., 363 N.C. 189,\n 201, 675 S.E.2d 641, 649 (2009); accord In re Banks, 295 N.C. 236, 239, 244 S.E.2d\n 386, 388-89 (1978) (“[C]ourts must give [a clear and unambiguous] statute its plain\n and definite meaning, and are without power to interpolate, or superimpose,\n provisions and limitations not contained therein.”).\n The first paragraph of subdivision 20-279.21(b)(4) defines the term\n “underinsured highway vehicle” as\n a highway vehicle with respect to the ownership,\n maintenance, or use of which, the sum of the limits of\n liability under all bodily injury liability bonds and\n insurance policies applicable at the time of the accident is\n less than the applicable limits of underinsured motorist\n coverage for the vehicle involved in the accident and\n insured under the owner’s policy.\n N.C.G.S. § 20-279.21(b)(4). The statute further sets out when UIM coverage is\n triggered:\n Underinsured motorist coverage is deemed to apply when,\n by reason of payment of judgment or settlement, all\n liability bonds or insurance policies providing coverage for\n bodily injury caused by the ownership, maintenance, or\n use of the underinsured highway vehicle have been\n LUNSFORD V. MILLS\n Opinion of the Court\n -10-\n exhausted.\n Id. (“triggering provision”).\n Farm Bureau reads the reference to “all bodily injury liability bonds and\n insurance policies applicable at the time of the accident” in the definition of an\n underinsured highway vehicle to mean that, in determining whether UIM coverage\n is triggered, the insured’s UIM coverage limit must be compared to the sum of all of\n the liability limits of all the at-fault motorists. Thus, according to Farm Bureau, as\n a prerequisite to receiving UIM benefits, Lunsford was required to exhaust not only\n Buchanan’s liability limits, but also the policy limits of Mills and Crowder to the\n extent that they are liable as joint tortfeasors. We read subdivision 20-279.21(b)(4)\n differently.\n As an initial matter, the reference to “all bodily injury liability bonds and\n insurance policies applicable at the time of the accident” is found in the UIM\n statute’s definition of an “underinsured highway vehicle,” not in the triggering\n provision. The location of the clause in a separate and distinct provision of the UIM\n statute indicates that the clause relates solely to an underinsured highway vehicle\n and not, as Farm Bureau suggests, to all the vehicles involved in an accident. See\n Colonial Penn Ins. Co. v. Salti, 84 A.D.2d 350, 352, 446 N.Y.S.2d 77, 79 (N.Y. App.\n Div. 1982) (“[T]he [UIM] endorsement affords coverage for bodily injury arising out\n of the use of an underinsured highway vehicle and the clause ‘the limits of liability\n LUNSFORD V. MILLS\n Opinion of the Court\n -11-\n under all bodily injury liability bonds or insurance policies applicable at the time of\n the accident’ . . . should be read to relate . . . to [the underinsured] vehicle only, and\n not, as [the insurer] contends, to the total number of vehicles involved in the\n accident.” (emphasis added)).\n An examination of the actual language of the triggering provision further\n undermines Farm Bureau’s reading of the statute to provide that UIM coverage is\n not triggered until “all liability limits applicable ‘at the time of the accident’ ” are\n exhausted. The plain language of the triggering provision identifies the liability\n bonds and insurance policies relevant to determining whether UIM coverage is\n triggered as those bonds and policies relating to “the ownership, maintenance, or\n use of the underinsured highway vehicle.” N.C.G.S. § 20-279.21(b)(4) (emphasis\n added). A statute’s use of the definite article—“the”—indicates that the legislature\n intended the term modified to have a singular referent. See Renz v. Grey Adver.,\n Inc., 135 F.3d 217, 222 (2d Cir. 1997) (“Placing the article ‘the’ in front of a word\n connotes the singularity of the word modified.”); see also Gen. Accident Ins. Co. v.\n Wheeler, 221 Conn. 206, 211, 603 A.2d 385, 387 (1992) (concluding, under an\n insurance regulation providing that “the ‘insurer shall undertake to pay on behalf of\n the insured all sums which the insured shall be legally entitled to recover as\n damages from the owner or operator of an uninsured [or underinsured] motor\n vehicle because of bodily injury sustained by the insured caused by an accident\n involving the uninsured [or underinsured] motor vehicle,’ ” that an insured is\n LUNSFORD V. MILLS\n Opinion of the Court\n -12-\n required to exhaust “the insurance coverage of only one tortfeasor” in order to\n recover UIM benefits (brackets in original)).\n Farm Bureau’s interpretation effectively rewrites the triggering provision to\n provide that UIM coverage applies only once all liability bonds or insurance policies\n providing coverage for any party potentially liable for the insured’s bodily injuries\n have been exhausted. But that is not what the statute says. The plain language of\n the triggering provision establishes that when an insured suffers bodily injury\n caused by the ownership, maintenance, or use of an underinsured highway vehicle,\n and when the liability bonds or insurance policies providing coverage for that\n vehicle have been exhausted, UIM coverage is triggered. Accordingly, a UIM\n carrier’s statutory obligation to provide UIM benefits is triggered when the insurer\n of a single vehicle meeting the definition of an underinsured highway vehicle\n tenders its liability limits to the UIM claimant through an offer of settlement or in\n satisfaction of a judgment. See Register v. White, 358 N.C. 691, 698, 599 S.E.2d 549,\n 555 (2004) (“Exhaustion occurs when the liability carrier has tendered the limits of\n its policy in a settlement offer or in satisfaction of a judgment.”).\n Farm Bureau contends, however, that this interpretation of subdivision 20-\n 279.21(b)(4) has been “expressly rejected by the legislature.” In support of this\n argument, Farm Bureau points to the General Assembly’s consideration and\n ultimate rejection of a bill proposed in the 1983 legislative session that was\n LUNSFORD V. MILLS\n Opinion of the Court\n -13-\n designed “to clarify the law concerning UIM coverage.” The proposed bill would\n have completely repealed subdivision 20-279.21(b)(4), the FRA’s provision governing\n UIM coverage, and amended subdivision 20-279.21(b)(3), the provision governing\n uninsured motorist coverage, so that an underinsured motor vehicle would have\n constituted an “uninsured motor vehicle” to the extent of “the difference between\n the limits of the bodily injury liability insurance and property damage liability\n insurance coverages on such motor vehicle and the limits of the uninsured motorist\n coverage provided under the insured’s motor vehicle liability insurance policy.” H.\n 60, 135th Gen. Assemb., Reg. Sess. (N.C. 1983) (emphasis added).\n The fact that this proposed bill was not enacted is unavailing. When, as here,\n “the language of a statute expresses the legislative intent in clear and unambiguous\n terms, the words employed must be taken as the final expression of the meaning\n intended unaffected by its legislative history.” Piedmont Canteen Serv., Inc. v.\n Johnson, 256 N.C. 155, 161, 123 S.E.2d 582, 586 (1962) (citations omitted); accord\n Wake Cares, Inc. v. Wake Cnty Bd. of Educ., 190 N.C. App. 1, 25, 660 S.E.2d 217,\n 232 (2008) (explaining that “[l]egislative history cannot . . . be relied upon to force a\n construction on [a] statute inconsistent with the plain language”), aff’d, 363 N.C.\n 165, 675 S.E.2d 345 (2009).\n Farm Bureau’s construction of the UIM statute also undermines the statute’s\n purpose. Section 20-279.21 “was passed to address circumstances where ‘ “the\n LUNSFORD V. MILLS\n Opinion of the Court\n -14-\n tortfeasor has insurance, but his [or her] coverage is in an amount insufficient to\n compensate the injured party for his [or her] full damages.” ’ ” Progressive Am. Ins.\n Co. v. Vasquez, 350 N.C. 386, 390, 515 S.E.2d 8, 10-11 (1999) (first alteration in\n original) (quoting Harris v. Nationwide Mut. Ins. Co., 332 N.C. 184, 189, 420 S.E.2d\n 124, 127 (1992), superseded by statute, Act of July 12, 1991, ch. 646, secs. 1, 2, 1991\n N.C. Sess. Laws 1550, 1559). We have recognized the remedial nature of\n subdivision 20-279.21(b)(4) and explained that the statute should be “liberally\n construed” in order to accomplish its purpose of “protect[ing] . . . innocent victims\n who may be injured by financially irresponsible motorists.” Proctor v. N.C. Farm\n Bureau Mut. Ins. Co., 324 N.C. 221, 224-25, 376 S.E.2d 761, 763 (1989). To that\n end, subdivision 20-279.21(b)(4)—as well as the FRA as a whole—should be\n “interpreted to provide the innocent victim with the fullest possible protection.” Id.\n at 225, 376 S.E.2d at 764.\n If Farm Bureau’s interpretation were adopted, insureds would be required to\n pursue all claims, including weak, tenuous ones, against all potentially liable\n parties, no matter how impractical, before being eligible to collect their contractedfor\n UIM benefits. Placing this burden on insureds—who, like Lunsford, commonly\n suffer serious injuries and need prompt payment of benefits to pay medical\n expenses and other costs—would substantially delay the recovery of UIM benefits\n and promote litigation expenses that reduce insureds’ overall recovery. See\n Wheeler, 221 Conn. at 213, 603 A.2d at 388 (observing that if the insured is not\n LUNSFORD V. MILLS\n Opinion of the Court\n -15-\n permitted to recover UIM benefits until exhausting all liability limits of all joint\n tortfeasors, “the insured could be required to pursue claims of weak liability\n against third parties, thereby fostering marginal and costly litigation in our\n courts”). Because Farm Bureau’s interpretation of subdivision 20-279.21(b)(4)\n would fail to provide innocent victims “the fullest possible protection,” we reject\n Farm Bureau’s proposed construction. See Proctor, 324 N.C. at 225-26, 376 S.E.2d\n at 764 (rejecting insurer’s construction of subdivision 20-279.21(b)(4) that\n “result[ed] in the least possible protection for the innocent victim of an\n underinsured tortfeasor” and thus “undermine[d] the intent and purpose of the\n statute”).\n Our conclusion that an insured may recover UIM benefits upon exhausting\n the liability limits of a single at-fault motorist is further buttressed by examining\n the subrogation provision of section 20-279.21. See Faizan v. Grain Dealers Mut.\n Ins. Co., 254 N.C. 47, 53, 118 S.E.2d 303, 307 (1961) (construing provisions of the\n FRA in pari materia). The third paragraph of subdivision 20-279.21(b)(4) states in\n pertinent part:\n An underinsured motorist insurer may at its\n option, upon a claim pursuant to underinsured motorist\n coverage, pay moneys without there having first been an\n exhaustion of the liability insurance policy covering the\n ownership, use, and maintenance of the underinsured\n highway vehicle. In the event of payment, the\n underinsured motorist insurer shall be either: (a) entitled\n to receive by assignment from the claimant any right or\n (b) subrogated to the claimant’s right regarding any claim\n LUNSFORD V. MILLS\n Opinion of the Court\n -16-\n the claimant has or had against the owner, operator, or\n maintainer of the underinsured highway vehicle, provided\n that the amount of the insurer’s right by subrogation or\n assignment shall not exceed payments made to the\n claimant by the insurer.\n N.C.G.S. § 20-279.21(b)(4); see also id. § 20-279.21(b)(3) (providing an insurer a\n right to reimbursement from settlement proceeds to the extent the insurer has\n made a “payment to any person under the coverage required by this section and\n subject to the terms and conditions of coverage”).\n If, as Farm Bureau argues, insureds were required to exhaust the liability\n policies of all at-fault motorists as a prerequisite to recovering UIM coverage, there\n would be no need to provide UIM carriers subrogation or reimbursement rights, and\n consequently, these provisions would be rendered meaningless. See Leslie v. W.H.\n Transp. Co., 338 F. Supp. 2d 684, 689 (S.D. W. Va. 2004) (“The reservation of a\n subrogation right indicates that [the insurer] foresees situations in which an\n insured receives UIM benefits and [the insurer] then pursues a claim against a\n tortfeasor who is legally liable for the damages suffered by the insured. If the\n insured were required to exhaust every tortfeasor’s policy limit before receiving\n UIM benefits, it is hard to imagine a UIM scenario in which subrogation rights\n would arise.”). Yet it is a fundamental principle of statutory interpretation that\n courts should “evaluate [a] statute as a whole and . . . not construe an individual\n section in a manner that renders another provision of the same statute\n meaningless.” Polaroid Corp. v. Offerman, 349 N.C. 290, 297, 507 S.E.2d 284, 290\n LUNSFORD V. MILLS\n Opinion of the Court\n -17-\n (1998) (citation omitted), cert. denied, 526 U.S. 1098, 119 S. Ct. 1576, 143 L. Ed. 2d\n 671 (1999), abrogated on other grounds by Lenox, Inc. v. Tolson, 353 N.C. 659, 548\n S.E.2d 513 (2001).\n Moreover, given the General Assembly’s provision of subrogation and\n reimbursement rights for the financial protection of insurers, we cannot agree with\n Farm Bureau’s argument that the trial court’s order resulted in a “windfall” for\n Lunsford. Farm Bureau could have preserved its subrogation rights by advancing\n its UIM policy limits. See State Farm Mut. Auto. Ins. Co. v. Blackwelder, 332 N.C.\n 135, 138-39, 418 S.E.2d 229, 231 (1992) (concluding that the insurer of the injured\n party’s vehicle had “preserved its subrogation rights” against the estate of the\n deceased tortfeasor by advancing the deceased tortfeasor’s liability limits to its\n insured and by advancing an additional amount to settle its insured’s UIM claim).\n Had Farm Bureau elected to do so, it would have been entitled to recoup the\n advanced funds from the proceeds of the settlements with Mills and Crowder.1\n 1 Farm Bureau further contends that Lunsford’s recovery of an amount greater than\n his contracted-for UIM coverage limit is inconsistent with the purpose of the UIM statute,\n as articulated by the Court of Appeals in Nationwide Mutual Insurance Co. v. Haight, 152\n N.C. App. 137, 566 S.E.2d 835 (2002), disc. rev. denied, 356 N.C. 675, 577 S.E.2d 627\n (2003), in which the court stated: “UIM coverage is intended to place a policy holder in the\n same position that the policy holder would have been in if the tortfeasor had had liability\n coverage equal to the amount of the . . . UIM coverage.” Id. at 142, 566 S.E.2d at 838\n (citations, emphasis, and quotation marks omitted). We perceive no inherent conflict\n between Haight’s articulation of the intended purpose of the UIM statute and the principle\n we reaffirmed in Proctor that subdivision 20-279.21(b)(4), as a remedial statute, must be\n “interpreted to provide the innocent victim with the fullest possible protection.” Proctor,\n 324 N.C. at 225, 376 S.E.2d at 764. Even if, as we have held, a UIM carrier is required to\n LUNSFORD V. MILLS\n Opinion of the Court\n -18-\n N.C.G.S. § 20-279.21(b)(3). But by not advancing its policy limits, Farm Bureau\n waived its subrogation rights. See N.C.G.S. § 20-279.21(b)(4) (prohibiting insurers\n from exercising any right of subrogation when “the insurer fails to advance a\n payment to the insured in an amount equal to the tentative settlement within 30\n days” of receiving written notice of the proposed settlement).\n In sum, we believe that the structure and plain language of subdivision 20-\n 279.21(b)(4), the purpose behind the UIM statute, and the legislature’s inclusion of\n subrogation rights for insurers, compel the conclusion that UIM coverage is\n triggered upon the exhaustion of the policy limits of a single at-fault motorist.\n Accordingly, upon Allstate’s tender of its policy limit of $50,000 on behalf of\n Buchanan, UIM coverage was triggered under subdivision 20-279.21(b)(4), and\n Lunsford was entitled to recover UIM benefits according to the terms of his policy\n with Farm Bureau. We therefore affirm the Court of Appeals’ decision on this issue.\n Judgment Interest and Costs\n Farm Bureau also challenges the Court of Appeals’ determination that\n Lunsford is entitled to pre- and post-judgment interest and costs. Farm Bureau\n contends that the award of these damages, taxed in excess of Lunsford’s UIM\n provide UIM coverage upon exhaustion of the liability limits of a single tortfeasor, the\n carrier may still seek recovery of any overpayment through the exercise of its rights to\n subrogation or reimbursement. Through these mechanisms, insurers are able to recoup\n any overpayment and insureds are divested of any so-called “windfall.”\n LUNSFORD V. MILLS\n Opinion of the Court\n -19-\n coverage limits, conflicts with our decision in Baxley v. Nationwide Mutual\n Insurance Co., 334 N.C. 1, 430 S.E.2d 895 (1993). We agree.\n We have established that “when a statute is applicable to the terms of a\n policy of insurance, the provisions of that statute become terms of the policy to the\n same extent as if they were written in it, and if the terms of the policy conflict with\n the statute, the provisions of the statute prevail.” Id. at 6, 430 S.E.2d at 898 (citing\n Sutton v. Aetna Cas. & Surety Co., 325 N.C. 259, 263, 382 S.E.2d 759, 762 (1989)).\n Section 20-279.21 is silent with respect to pre- and post-judgment interest, and thus\n subsection 24-5(b), the statute governing judgment interest, “is not a part of the\n Financial Responsibility Act so as to be written into every liability policy.” Id.\n (citing Sproles, 329 N.C. at 613, 407 S.E.2d at 503). When, as here, no statutory\n provision dictates a liability insurer’s obligation to pay interest in excess of its\n policy limits, such an obligation “is governed by the language of the policy.” Id.\n (citing Sproles, 329 N.C. at 612-13, 407 S.E.2d at 502-03) (emphasis omitted).\n The pertinent language in Lunsford’s business and personal policies states\n that Farm Bureau promises to pay, up to its UIM policy limit,\n all sums the “insured” is legally entitled to recover as\n compensatory damages from the owner or driver of:\n a. An “uninsured motor vehicle” because of “bodily\n injury” sustained by the “insured” and caused\n by an “accident”; and\n b. b. An “uninsured motor vehicle” as defined in\n LUNSFORD V. MILLS\n Opinion of the Court\n -20-\n Paragraphs a. and c. of the definition of\n “uninsured motor vehicle”, because of “property\n damage” caused by an “accident”.\n The owner’s or driver’s liability for these damages must\n result from the ownership, maintenance or use of the\n “uninsured motor vehicle”.\n (Emphasis added.) The policies’ definition of an “uninsured motor vehicle” includes\n an “underinsured motor vehicle.”\n In Baxley, 334 N.C. at 6-7, 430 S.E.2d at 899, we examined substantially\n similar policy language:\n The contractual language [at issue] is [the UIM\n carrier]’s promise to pay, up to its UIM policy limit,\n damages which a covered person is legally entitled to\n recover from the owner or operator of an uninsured motor\n vehicle because of:\n 1. Bodily injury sustained by a covered person and\n caused by an accident; and\n 2. Property damage caused by an accident.\n Holding that interest is an element of “damages,” id. at 11, 430 S.E.2d at 901, we\n held that, based on the pertinent policy language, the UIM carrier in Baxley was\n “obligated to pay pre-judgment interest up to its policy limits.” Id. at 6, 430 S.E.2d\n at 898 (emphasis omitted). Our reasoning in Baxley regarding judgment interest\n has similarly been applied to costs. See Wiggins v. Nationwide Mut. Ins. Co., 112\n N.C. App. 26, 35-36, 434 S.E.2d 642, 648 (1993) (rejecting, based on Baxley, the\n insurer’s contention “that ‘damages’ does not include costs or interest”).\n LUNSFORD V. MILLS\n Opinion of the Court\n -21-\n The relevant language in Farm Bureau’s policy is, we believe, materially\n indistinguishable from the policy language at issue in Baxley. Accordingly, the\n Court of Appeals erred in concluding that Farm Bureau was required to pay preand\n post-judgment interest and costs in excess of its remaining UIM policy limit of\n $350,000. Because Farm Bureau contractually capped its obligation to pay\n “compensatory damages” at its UIM coverage limit, Farm Bureau is not required to\n pay interest and costs over and above the $350,000 coverage amount. See Baxley,\n 334 N.C. at 11, 430 S.E.2d at 901 (“Since [the UIM carrier] promised to pay [the\n insured]’s resulting damages, it must now do so, up to, but not in excess of, its UIM\n policy limits.”).\n Lunsford nonetheless claims that Farm Bureau should be required to pay\n pre- and post-judgment interest because the judgment “was entered directly against\n Farm Bureau” due to Farm Bureau’s “breach of its obligations under its insurance\n contract.” This argument is misplaced. There is no underlying breach of contract\n claim against Farm Bureau in this case, and thus, such a claim could not have been\n the basis for the trial court’s award of interest and costs. Rather, the basis for the\n award is Farm Bureau’s promise to pay, up to its UIM coverage limit, the\n “compensatory damages” resulting from the named defendants’ negligence. In such\n circumstances, our precedent “clearly establish[es]” that the extent to which a UIM\n carrier is required to pay judgment interest is controlled by “the specific terms of\n [the] policy.” Nationwide Mut. Ins. Co. v. Mabe, 342 N.C. 482, 491, 467 S.E.2d 34,\n LUNSFORD V. MILLS\n Opinion of the Court\n -22-\n 40 (1996). Farm Bureau was permitted to, and did in fact, cap its liability for\n damages, including interest, at the amount of its UIM coverage limit. We\n accordingly reverse the Court of Appeals’ decision with respect to interest and costs.\n Conclusion\n We affirm that part of the decision of the Court of Appeals holding than an\n insured is only required to exhaust the liability insurance coverage of a single atfault\n motorist in order to trigger the insurer’s obligation to provide UIM benefits.\n We reverse the decision of the Court of Appeals awarding interest and costs against\n the insurer in an amount that exceeds the amount the insurer contractually\n promised to pay under the terms of its policy with the insured. This case is\n remanded to the Court of Appeals for further remand to the Superior Court,\n McDowell County, for proceedings not inconsistent with this opinion.\n AFFIRMED IN PART; REVERSED IN PART, AND REMANDED.\n Justice HUNTER did not participate in the consideration or decision of this\n case.\n No. 385PA13 – Lunsford v. Mills\n Justice NEWBY dissenting in part and concurring in part.\n The purpose of underinsured motorist (UIM) coverage in our state is to serve\n as a safeguard when tortfeasors’ liability policies do not provide sufficient\n recovery—that is, when the tortfeasors are “under insured.” This is simply not the\n case here. Plaintiff incurred damages amounting to $900,000. He brought suit\n jointly and severally against responsible tortfeasors whose total liability limits were\n $1,050,000. Those combined liability limits were more than sufficient to satisfy\n plaintiff’s damages and were more than twice as high as plaintiff’s $400,000 UIM\n limits. Not only does the majority incorrectly hold that UIM coverage was\n necessary in this instance, but the majority’s outcome also leaves plaintiff with\n $350,000 in excess of his agreed-to damages. By contrast, I would hold that UIM\n coverage was not activated in this case. Rather, under the UIM statute, coverage\n only applies when the policyholder’s UIM limits are more than the combined limits\n of the insurance coverage of all jointly and severally liable tortfeasors against whom\n the plaintiff files suit. Consequently, I respectfully dissent.\n At the time of the accident, the jointly and severally liable tortfeasors, Mills,\n his employer Crowder, and Buchanan, carried liability policies totaling $1,050,000\n while plaintiff was covered by two UIM policies with North Carolina Farm Bureau\n Mutual Insurance Company (Farm Bureau) with combined limits of $400,000.\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -2-\n After plaintiff filed suit against Mills, Crowder, and Buchanan, Buchanan’s\n provider, Allstate, tendered to plaintiff the $50,000 limits of Buchanan’s policy. Six\n months later, plaintiff settled his claim with defendants Mills’ and Crowder’s\n coverage provider for $850,000. After the trial court approved plaintiff’s settlement\n with the named defendants, Farm Bureau, as an unnamed defendant, moved for\n summary judgment, contending that plaintiff was not entitled to UIM coverage\n because the combined policy limits of the defendants exceeded plaintiff’s UIM\n limits. Plaintiff also moved for summary judgment, insisting that Buchanan was an\n underinsured driver and that plaintiff was thus entitled to Farm Bureau’s UIM\n policy limits of $400,000 less an offset of $50,000 for Buchanan’s Allstate insurance\n payment. The trial court entered judgment in plaintiff’s favor for $350,000, plus\n costs and pre- and post-judgment interest. As a result, plaintiff received $50,000\n from Buchanan’s insurer, $850,000 from the settlement with Mills and Crowder,\n and $350,000 from his own UIM policy with Farm Bureau for a total of $1,250,000\n while settling his damages claims with the actual tortfeasors for only $900,000,\n which left untapped $150,000 of tortfeasor insurance.\n The majority’s holding is based on a fundamental misunderstanding of UIM\n coverage and the implementing statute, as well as a misunderstanding of Farm\n Bureau’s argument. UIM insurance in North Carolina developed out of uninsured\n motorist (UM) insurance. James E. Snyder, Jr., North Carolina Automobile\n Insurance Law § 30-1 (3d ed. 1999). UM insurance provides recovery for a\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -3-\n policyholder injured in an auto accident by the motor vehicle of a tortfeasor who has\n no liability insurance. Id. By comparison, UIM coverage provides a secondary\n source of recovery for an insured when the tortfeasor has insurance, but the\n tortfeasor’s liability limits are insufficient to compensate the injured party. Sutton\n v. Aetna Cas. & Sur. Co., 325 N.C. 259, 263, 382 S.E.2d 759, 762 (1989), superseded\n on other grounds by statute, Act of July 12, 1991, ch. 646, 1991 N.C. Sess. Laws\n 1550 (captioned “An Act to Prohibit the Stacking of Uninsured and Underinsured\n Motorist Coverage”). The UM and UIM statute is part of North Carolina’s Motor\n Vehicle Safety and Financial Responsibility Act of 1953 (Act). N.C.G.S. §§ 20-279.1\n to 279.39 (2013). The Act’s purpose is\n to compensate the innocent victims of financially\n irresponsible motorists. The Act is remedial in nature\n and is to be liberally construed so that the beneficial\n purpose intended by its enactment may be accomplished.\n The purpose of the Act, we have said, is best served when\n [every provision of the Act] is interpreted to provide the\n innocent victim with the fullest possible protection.\n Liberty Mut. Ins. Co. v. Pennington, 356 N.C. 571, 573-74, 573 S.E.2d 118, 120\n (2002) (alteration in original) (citations and internal quotation marks omitted).\n Even though the Act is intended to provide “the fullest possible protection,” id. at\n 574, 573 S.E.2d at 120, it is only activated when a plaintiff is “under insured.” A\n plaintiff cannot, under the statute, obtain UIM proceeds if the tortfeasors’ insurance\n is greater than the UIM coverage or is sufficient to compensate his damages.\n N.C.G.S. § 20-279.21(b)(4). The recovery provided by UIM coverage is only meant to\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -4-\n augment inadequate recoveries obtained from underinsured tortfeasors. Id.\n (reducing UIM amounts by amounts received by the plaintiff from a tortfeasor’s\n exhausted policy or policies). In other words, UIM coverage puts the insured\n claimant back in the position he would have occupied had the tortfeasor been\n insured at limits equal to the claimant’s UIM limits. See Nationwide Mut. Ins. Co.\n v. Haight, 152 N.C. App. 137, 142, 566 S.E.2d 835, 838 (2002) (noting the statute’s\n goal of putting a policy holder “in the same position that the policy holder would\n have been in if the tortfeasor had had liability coverage equal to the amount of the\n UM/UIM coverage” (citations and emphasis omitted)), disc. rev. denied, 356 N.C.\n 675, 577 S.E.2d 627 (2003).\n Two provisions in the UIM statute in particular demonstrate this intent by\n the legislature to make UIM coverage a source of compensation secondary to\n tortfeasors’ liability policies. Elec. Supply Co. of Durham v. Swain Elec. Co., 328\n N.C. 651, 656, 403 S.E.2d 291, 294 (1991) (observing that, inter alia, “we are guided\n by the structure of the statute” in determining legislative intent (citations omitted)).\n The first is the reduction provision, which states:\n In any event, the limit of underinsured motorist\n coverage applicable to any claim is determined to be the\n difference between the amount paid to the claimant under\n the exhausted liability policy or policies and the limit of\n underinsured motorist coverage applicable to the motor\n vehicle involved in the accident.\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -5-\n N.C.G.S. § 20-279.21(b)(4) (“reduction provision”). Under the reduction provision, a\n UIM carrier reduces its applicable policy limits by amounts paid to the claimant\n from tortfeasors’ exhausted policies.\n The second supporting provision is the offset or recovery provision found in\n N.C.G.S. § 20-279.21(b)(3), which is incorporated by reference into subdivision 20-\n 279.21(b)(4):\n In the event of payment to any person under the\n coverage required by this section and subject to the terms\n and conditions of coverage, the insurer making payment\n shall, to the extent thereof, be entitled to the proceeds of\n any settlement for judgment resulting from the exercise of\n any limits of recovery of that person against any person or\n organization legally responsible for the bodily injury for\n which the payment is made, including the proceeds\n recoverable from the assets of the insolvent insurer.\n Id. at § 20-279.21(b)(3). This provision entitles a UIM carrier to use a claimant’s\n judgment proceeds to recoup the UIM carrier’s payments to the claimant. The\n presence of the reduction and offset provisions in the statute evinces a legislative\n intent for UIM coverage to be applicable only to the extent that other sources of\n recovery fail to compensate for the injury up to the UIM limits.2 Elec. Supply Co.,\n 328 N.C. at 656, 403 S.E.2d at 294 (“An analysis utilizing the plain language of the\n statute and the canons of construction must be done in a manner which harmonizes\n 2 By contrast, some states apply an “excess coverage” approach whereby UIM\n coverage is activated when a tortfeasor’s liability limits are exceeded by the insured’s\n damages. 3 Irvin E. Schermer & William J. Schermer, Automobile Liability Insurance §\n 38:9, at 38-31 (4th ed. Dec. 2004).\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -6-\n with the underlying reason and purpose of the statute.” (citation omitted)); State v.\n Hart, 287 N.C. 76, 80, 213 S.E.2d 291, 295 (1975) (“A construction which operates to\n defeat or impair the object of the statute must be avoided if that can reasonably be\n done without violence to the legislative language.” (citation omitted)). The insured’s\n UIM limits, not the insured’s total damages, provide the ceiling for recovery. See\n Fasulo v. State Farm Mut. Auto. Ins. Co., 108 N.M. 807, 810-11, 780 P.2d 633, 636-\n 37 (1989) (discussing a UIM statute similar to subsection 20-279.21(b)(4)). Thus, an\n insured plaintiff’s UIM recovery “is controlled contractually by the amount of the\n UIM policy limits purchased and available to her, not fortuitously by the number of\n tortfeasors involved in the accident.” Nikiper v. Motor Club of Am. Cos., 232 N.J.\n Super. 393, 398-99, 557 A.2d 332, 335, certification denied, 117 N.J. 139, 564 A.2d\n 863 (1989). The majority’s holding runs contrary to the nature and purpose of UIM\n coverage.\n With this understanding of the UIM statute’s purpose in mind, it is necessary\n to consider closely the statute’s controlling provision in this case—the activation\n provision. As an initial matter, the majority misreads Farm Bureau’s argument.\n Farm Bureau is not insisting that the statute requires plaintiff “to exhaust not only\n Buchanan’s liability limits, but also the policy limits of Mills and Crowder to the\n extent that they are liable as joint tortfeasors” in order for plaintiff to receive UIM\n benefits. Rather, Farm Bureau is asserting that UIM coverage is not applicable at\n all because plaintiff implicated $1,050,000 in liability coverage when he sued the\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -7-\n three tortfeasors. As a result of this mischaracterization, the majority errs in its\n approach to the statute by focusing on the UIM’s triggering (exhaustion) provision\n without first fully considering subdivision (b)(4)’s activation provision.3 The\n distinction between the activation and triggering provisions is critical because if no\n vehicle meets the definition of an underinsured vehicle under the activation\n provision, then consideration of the subsequent triggering provision is unnecessary.\n The activation provision is found in subdivision (b)(4), which is the portion of\n the statute governing UIM coverage. N.C.G.S. § 20-279.21(b)(4). A UIM carrier\n pays on its policy to an injured claimant when (1) the auto accident involves a\n tortfeasor who meets the statute’s definition of an underinsured highway vehicle\n (the activation provision); and (2) the underinsured highway vehicle’s liability\n coverage has been exhausted (triggering provision). Id.4 The UIM statute’s\n activation provision defines an underinsured highway vehicle as:\n [A] highway vehicle with respect to the ownership,\n maintenance, or use of which, the sum of the limits of\n liability under all bodily injury liability bonds and\n insurance policies applicable at the time of the accident is\n less than the applicable limits of underinsured motorist\n 3 The majority’s analysis and interpretation of the activation provision is relegated\n to one paragraph with a citation to a case from New York interpreting, against the insurer,\n a provision in a claimant’s insurance policy. That case did not interpret a statute and offers\n no support for an interpretation of North Carolina’s statute.\n 4 The relevant portions of the current version of this statute are identical to the 2009\n version of the statute, which is the version applicable to this case. White v. Mote, 270 N.C.\n 544, 555, 155 S.E.2d 75, 82 (1967) (“Laws in effect at the time of issuance of a policy of\n insurance become a part of the contract . . . .”).\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -8-\n coverage for the vehicle involved in the accident and\n insured under the owner’s policy.\n Id. The activation provision applies a comparison of limits approach—UIM\n coverage is activated when the insured’s UIM policy limits are greater than the\n liability limits of policies connected with the tortfeasor’s ownership, maintenance, or\n use of a highway vehicle. 3 Irvin E. Schermer & William J. Schermer, Automobile\n Liability Insurance § 38:7 (4th ed. Dec. 2004) [hereinafter Automobile Liability\n Insurance]. In a scenario involving a single insured claimant and a single\n tortfeasor, application of the statute’s activation provision is straightforward. If the\n insured’s UIM limits are greater than the tortfeasor’s liability limits, the insured’s\n UIM coverage is activated. N.C.G.S. § 20-279.21(b)(4). Only then does subdivision\n (b)(4)’s triggering provision become relevant.\n Under the triggering provision, once the tortfeasor’s liability limits have been\n paid out to the insured, if the injuries have not been adequately compensated, the\n insured can collect from the UIM carrier up to the maximum amount of the UIM\n coverage limits minus the amount paid to the claimant under the tortfeasor’s\n exhausted policy. Id. The net effect is that UIM coverage puts the insured claimant\n back in the position he would have occupied had the tortfeasor been insured at\n limits equal to the claimant’s UIM limits. See Haight, 152 N.C. App. at 142, 566\n S.E.2d at 838.\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -9-\n Though the activation provision’s application is clear when only one\n tortfeasor is involved, we have not previously addressed whether, in a multiple\n tortfeasor scenario, the insured’s UIM policy limits should be compared individually\n to each tortfeasor’s liability limits or compared to the sum of the liability limits of\n all tortfeasors. When read in the broader context of the statute, the UIM’s\n activation provision instructs comparing the insured’s policy limits to the sum of the\n liability of all jointly and severally liable tortfeasors. More specifically, a vehicle is\n underinsured when “the sum of the limits of liability under all bodily injury liability\n bonds and insurance policies applicable at the time of the accident” with respect to\n the use of the vehicle is less than an insured’s UIM limits. N.C.G.S. § 20-\n 279.21(b)(4).\n This interpretation of the activation provision is in consonance with the\n surrounding provisions of the statute and in keeping with the overall legislative\n intent of requiring UIM coverage to provide a limited source of compensation when\n a claimant is injured by tortfeasors who are collectively underinsured.5 Automobile\n Liability Insurance § 41.3 at 41-42 (noting that under “comparison of limits”\n statutes like North Carolina’s, “an underinsured motorist carrier may defeat\n underinsured motorist coverage by pointing to other liability coverages available to\n 5 The legislative history of the statute asserted by Farm Bureau and addressed by\n the majority provides additional support for this interpretation. Because the activation\n provision is susceptible to multiple interpretations, the majority’s dismissive “plain\n meaning” response to Farm Bureau’s argument is unavailing.\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -10-\n the tortfeasor which, when aggregated, produce a totality of limits in excess of the\n underinsured motorist insured’s limits, or by aggregating the liability coverages of\n joint tortfeasors.” (emphasis added) (footnote call number omitted)); see Nikiper, 232\n N.J. Super. at 397, 557 A.2d at 334 (“We conclude that where the amount paid by\n the insurors for the multiple tortfeasors equals or exceeds the amount of the UIM\n coverage, plaintiff has no UIM claim.”); see also Sutton, 325 N.C. at 265, 382 S.E.2d\n at 763 (observing that “[l]egislative intent can be ascertained not only from the\n phraseology of the statute but also from the nature and purpose of the act and the\n consequences which would follow its construction one way or the other”). In the\n case at hand it is contrary to the purpose of the statute to conclude that Buchanan’s\n vehicle alone activates UIM coverage when the combined liability limits of the\n jointly and severally liable tortfeasors is $1,050,000 and plaintiff’s UIM coverage is\n $400,000. Likewise, it is nonsensical to say a party is “underinsured” when the\n injured party settles with the tortfeasors for $150,000 less than their policies’\n coverage. State v. Beck, 359 N.C. 611, 614, 614 S.E.2d 274, 277 (2005) (“[W]here a\n literal interpretation of the language of a statute will lead to absurd results, or\n contravene the manifest purpose of the Legislature, as otherwise expressed, the\n reason and purpose of the law shall control and the strict letter thereof shall be\n disregarded.” (citations and internal quotation marks omitted)).\n Interpreting the first portion of the activation provision to require comparing\n UIM limits to the combined limits of jointly and severally liable tortfeasors is in\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -11-\n harmony with the immediately succeeding portion of the activation provision, which\n addresses UIM coverage in the context of multiple victims. State ex rel. Comm’r of\n Ins. v. N. C. Auto. Rate Admin. Office, 287 N.C. 192, 202, 214 S.E.2d 98, 104 (1975)\n (“We are further guided by rules of construction that statutes in pari materia, and\n all parts thereof, should be construed together and compared with each other.”\n (citation omitted)). The succeeding portion of the provision states:\n For purposes of an underinsured motorist claim asserted\n by a person injured in an accident where more than one\n person is injured, a highway vehicle will also be an\n “underinsured highway vehicle” if the total amount\n actually paid to that person under all bodily injury\n liability bonds and insurance policies applicable at the\n time of the accident is less than the applicable limits of\n underinsured motorist coverage for the vehicle involved in\n the accident and insured under the owner’s policy.\n N.C.G.S. § 20-279.21(b)(4) (emphasis added). This provision unambiguously\n contemplates comparing an insured plaintiff’s UIM limits broadly to payments the\n plaintiff has received under all liability policies applicable at the time of the\n accident. It does not restrict the comparison of limits test to a single tortfeasor.\n Because this second portion of the activation provision requires aggregation of\n liability limits for the purposes of comparison in a multiple victim scenario, under\n the first portion of the activation provision, in a multiple tortfeasor scenario, the\n same aggregation of liability limits must apply. Otherwise, in a multiple victim,\n multiple tortfeasor scenario, the activation provision would produce conflicting\n determinations as to the existence of an underinsured highway vehicle, with the\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -12-\n first portion requiring a one-to-one comparison and the second portion requiring a\n one-to-all comparison. An interpretation of the activation provision that limits\n policy comparisons to a single tortfeasor violates a basic rule of statutory\n interpretation by creating this conflict. Nationwide Mut. Ins. Co. v. Chantos, 293\n N.C. 431, 440, 238 S.E.2d 597, 603 (1977) (“Obviously, the Court will, whenever\n possible, interpret a statute so as to avoid absurd consequences.” (citations\n omitted)).\n The majority contends that under Farm Bureau’s approach, “insureds would\n be required to pursue all claims, including weak, tenuous ones, against all\n potentially liable parties, no matter how impractical, before being eligible to collect\n their contracted-for UIM benefits.” As noted above, this conclusion arises from\n mischaracterizing Farm Bureau’s argument as stating that UIM benefits should\n only be paid after plaintiff exhausts all applicable policies. The majority’s policy\n concern disappears, however, when Farm Bureau’s position is correctly understood\n to be that UIM coverage is not activated when the sum of the jointly and severally\n liable tortfeasors’ policy limits is higher than plaintiff’s UIM limits. In the instant\n case plaintiff chose to bring suit against the three defendants jointly and severally;\n no one was being “forced to sue any and all possible persons,” Lunsford v. Mills, ___\n N.C. App. ___, ___, 747 S.E.2d 390, 394 (2013), or “required to pursue all claims,” as\n the majority insists. An attempt by a UIM carrier to demand that plaintiff pursue\n the other tortfeasors before being eligible for UIM benefits “would be in the realm of\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -13-\n bad faith.” Farm Bureau Ins. Co. of N.C. v. Blong, 159 N.C. App. 365, 373, 583\n S.E.2d 307, 312, disc. rev. denied, 357 N.C. 578, 589 S.E.2d 125 (2003). Our\n General Statutes already prohibit such actions. N.C.G.S. § 58-63-15(11)(f) (2013)\n (“Unfair Claim Settlement Practices”); see also Gray v. N.C. Ins. Underwriting\n Ass’n, 352 N.C. 61, 71, 529 S.E.2d 676, 683 (2000) (concluding that “the act or\n practice of ‘[n]ot attempting in good faith to effectuate prompt, fair and equitable\n settlements of claims in which liability has become reasonably clear,’ also engages\n in conduct that embodies the broader standards of N.C.G.S. § 75-1.1” (alteration in\n original) (quoting N.C.G.S. § 58-63-15(11)(f))). The decision whether to pursue\n further litigation is within the control of the plaintiff unless he subrogates his\n claims to the insurer; a UIM carrier “cannot require an insured to pursue [other\n alleged tortfeasors] before exhaustion can occur.” Blong, 159 N.C. App. at 373, 583\n S.E.2d at 312. If plaintiff in this case had preferred to sue Buchanan alone and\n collect on his $50,000 policy limits, plaintiff’s UIM coverage would have been\n activated and triggered. Having chosen, however, to pursue simultaneously claims\n against multiple tortfeasors whose combined liability limits far exceeded plaintiff’s\n own UIM coverage, plaintiff was no longer able to access his UIM policy limits.\n The majority further asserts, again under a misunderstanding of Farm\n Bureau’s position, that requiring exhaustion before the receipt of UIM benefits\n would render “meaningless” the provisions granting UIM carriers subrogation and\n reimbursement rights. Under a proper consideration of Farm Bureau’s position and\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -14-\n based on a proper reading of the activation provision, the provisions in question\n would not be surplusage. The subrogation provision noted by the majority is\n applicable when (a) underinsured motorist coverage is activated, and (b) a UIM\n carrier voluntarily pays out to the insured before the triggering provision has been\n satisfied. N.C.G.S. § 20-279.21(b)(4). This subrogation right is a necessary\n assurance to a UIM carrier who voluntarily, id. (“at its option”), chooses to pay its\n insured before exhaustion of a tortfeasor’s policy limits. Granted, this scenario is\n not likely to occur. George L. Simpson, III, North Carolina Uninsured and\n Underinsured Motorist Insurance § 4:2, at 351 (2013-2014 ed.) (noting that these\n occasions are likely to be few). Nevertheless, this does not make the provision\n superfluous.\n Lastly, the majority misapprehends subdivision (b)(4)’s thirty-day\n advancement-of-payment provision. The majority is incorrect in concluding that\n Farm Bureau has forfeited its rights to recovery from the proceeds of the Mills and\n Crowder settlement, N.C.G.S. § 20-279.21(b)(3) (incorporated into subdivision (b)(4)\n and entitling the UIM carrier to “the proceeds of any settlement for judgment”\n related to the plaintiff’s injuries), because it failed to “preserve its subrogation\n rights” by not advancing its policy limits to plaintiff in a timely manner. When a\n UIM carrier fails to advance payment within thirty days of notice of a settlement\n with an underinsured motorist, it only forfeits its subrogation rights as to the\n underinsured motorist under N.C.G.S. § 20-279.21(b)(4) (“No insurer shall exercise\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -15-\n any right of subrogation or any right to approve settlement with the original owner,\n operator, or maintainer of the underinsured highway vehicle under a policy\n providing coverage against an underinsured motorist where the insurer has been\n provided with written notice before a settlement between its insured and the\n underinsured motorist and the insurer fails to advance a payment to the insured in\n an amount equal to the tentative settlement within 30 days following receipt of that\n notice.” (emphasis added)). That thirty-day deadline does not affect the UIM\n carrier’s recovery rights against remaining tortfeasors. Furthermore, the offset\n provision in N.C.G.S. § 20-279.21(b)(3) contains no requirement that a UIM carrier\n first pay out its limits before being entitled to a recovery against the proceeds paid\n by tortfeasors. Nothing in the statute dictates that a UIM carrier forfeits its rights\n to offset against judgment recoveries from other parties by not paying out benefits\n in a timely manner.\n The case relied on by the majority in support of its forfeiture conclusion, State\n Farm Mutual Automobile Insurance Co. v. Blackwelder, determined that the insurer\n preserved subrogation rights against the underinsured tortfeasor; it does not\n address a UIM carrier’s right to recover proceeds paid by other tortfeasors. 332\n N.C. 135, 418 S.E.2d 229 (1992). In Blong, upon which the Court of Appeals relied\n in arriving at a conclusion similar to that of the majority, a UIM carrier paid out its\n policy limits to an insured and then argued it was entitled to an offset against any\n amounts received by the insured in subsequent actions against additional parties.\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -16-\n 159 N.C. App. at 367-68, 583 S.E.2d at 308-09. Noting the UM/UIM statute’s\n remedial nature, Blong nonetheless concluded that “the Act appears to allow for the\n type of subrogation that plaintiff claims.” Id. at 373, 583 S.E.2d at 312. Blong\n answered the question whether the UIM carrier was entitled to an offset after\n having already paid out its UIM limits and gave a sequence of “how the procedure\n may play out.” Id. (emphasis added). The holding in Blong does not “clearly\n obligate[ ] the UIM carrier to first provide coverage, and later seek [recovery]”.\n Lunsford, ___ N.C. App. at ___, 747 S.E.2d at 394. Neither the UIM statute nor\n case law provides the necessary support for the majority’s timing and forfeiture\n determination regarding Farm Bureau’s entitlement to recovery. Furthermore,\n reading the UIM statute as requiring Farm Bureau to pay out its UIM limits\n promptly in order to protect the UIM policyholder is unnecessary; a UIM claimant\n is already protected by the Unfair Claim Settlement Practices statute from delayed\n payment, as noted above. Regardless whether UIM coverage was activated in this\n case, Farm Bureau should nevertheless be entitled to recovery.\n The majority’s insistence on reading the activation provision as limited only\n to a comparison of the UIM policy limits and an individual tortfeasor’s policy limits\n in this case allows plaintiff to collect from his $400,000 UIM policy even though he\n has already settled damages claims for $900,000 with the tortfeasors, which is\n $150,000 less than the maximum primary insurance coverage available. The\n legislature never intended for UIM coverage to serve this role, providing plaintiff an\n LUNSFORD V. MILLS\n Newby, J., dissenting in part and concurring in part\n -17-\n excess recovery of $350,000. Rather the legislature intended for plaintiff’s UIM\n policy to serve as a safeguard to protect plaintiff in the event the tortfeasors’\n liability policies failed to compensate plaintiff for injuries up to $400,000. This\n legislative intent is best carried out by first comparing plaintiff’s UIM limits to the\n combined limits of all the auto policies implicated in the lawsuit. Even though the\n majority’s holding provides “the fullest possible protection,” Pennington, 356 N.C. at\n 574, 573 S.E.2d at 120, it contravenes the activation provision’s requirements and\n the legislature’s intent to reduce UIM payouts by amounts recovered from all liable\n parties. Accordingly, the trial court erred in requiring Farm Bureau to make the\n $350,000 payment. Nevertheless, were UIM coverage properly implicated, I agree\n with the majority that the awarding of costs and interests against the insurer is\n limited contractually by the terms of the insured’s policy. Thus, I respectfully\n concur in part and dissent in part."""
err11 = """An unpublished opinion of the North Carolina Court of Appeals does not constitute controlling legal authority. Citation is disfavored, but may be permitted in accordance with the provisions of Rule 30(e)(3) of the North Carolina Rules of Appellate Procedure.\n IN THE COURT OF APPEALS OF NORTH CAROLINA\n No. COA 20-112\n Filed: 31 December 2020\n Wake County, No. 17 CVD 013381\n DAWN REYNOLDS-DOUGLASS, Plaintiff,\n v.\n KARI TERHARK, Defendant.\n Appeal by defendant from judgment entered 17 October 2019 by Judge Ned W. Mangum in Wake County District Court. Heard in the Court of Appeals 12 August 2020.\n Omer Law Firm, PLLC, by David G. Omer, for plaintiff-appellee.\n Kari Terhark, pro se, for defendant-appellant.\n YOUNG, Judge.\n This appeal arises out of a breach of contract. The trial court did not err in granting summary judgment in Plaintiff’s favor, nor did the trial court err in awarding Plaintiff attorney’s fees and damages. Accordingly, we find no error.\n I. Factual and Procedural History\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 2 -\n In July 2017, Dawn Reynolds-Douglass (“Plaintiff”) placed her home (“the property”) on the market for sale. Plaintiff hired real estate agent Dee Love (“Agent”), who advised Douglass to complete a North Carolina Residential Property and Owner’s Association Disclosure Statement (“Disclosure Statement”) pursuant to Chapter 47E of the North Carolina General Statutes. When Plaintiff did so, she inadvertently left two items unanswered: Item Number 27, inquiring as to whether there existed any easements, driveways, shared walls, or encroachments from or on adjacent property, and Item Number 33, asking whether any conveyance or transfer fees would be levied by the community homeowner’s association. Plaintiff signed the Disclosure Statement on 14 July 2017.\n On 24 July 2017, Kari Terhark (“Defendant”) reviewed a copy of the Disclosure Statement and signed each page, including the pages containing Items Numbers 27 and 33. Defendant then reviewed and signed an Offer to Purchase and Contract (“Agreement”) for $250,000. Plaintiff accepted Defendant’s offer by signing the Agreement on 24 July 2017 and took the property off the market.\n The Agreement included several provisions that are significant to this litigation. Paragraph 1(d) of the Agreement, entitled “Purchase Price,” provides for (i) a “Due Diligence Fee” of $2,000.00, delivered to the Seller by the Effective Date (in this case 24 July 2017), as well as (ii) an “Earnest Money Deposit” of $2,500.00 delivered no later than 14 August 2017. The Agreement also provided in Paragraph\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 3 -\n 1(e) that the Earnest Money Deposit was to be paid to the Seller in the event of the Buyer’s breach of the Agreement, “as liquidated damages and as Seller’s sole and exclusive remedy for such breach, but without limiting . . . Seller’s right to retain the Due Diligence Fee.”\n Further, Paragraph 1(e) of the Agreement provides that “[i]f legal proceedings are brought by Buyer or Seller against the other to recover the Earnest Money Deposit, the prevailing party in the proceeding shall be entitled to recover from the non-prevailing party reasonable attorney’s fees and court costs incurred in connection with the proceeding.” Paragraph 1(i) of the Agreement provides that the Due Diligence Fee “shall be the property of Seller upon the Effective Date and shall be a credit to Buyer at Closing.” The same paragraph continues, “[t]he Due Diligence Fee shall be non-refundable except in the event of a material breach of this Contract by seller …”\n Defendant did not deliver the Due Diligence Fee on the Effective Date of 24 July 2017 as required by the Agreement. Defendant later informed Plaintiff, through Agent, that she refused to tender the fee until Plaintiff agreed to reduce the purchase price by $5,500.00, in order to allow Defendant to purchase new appliances and carpet for the property. Plaintiff refused, and Defendant attempted to terminate the Agreement by sending an email to Agent on 27 July 2017. Plaintiff terminated the Agreement on 28 July 2017 by serving Defendant with a “Notice to Buyer That Seller\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 4 -\n is Exercising Their Unilateral Right to Terminate the Offer to Purchase and Contract.”\n Plaintiff relisted the property. Defendant ended her relationship with Agent, obtained her own buyer’s agent, and made a second offer. Plaintiff countered this offer with an asking price of $245,000.00, no due diligence period and all major appliances not included. There were no further negotiations.\n On 29 September 2017, Plaintiff sued Defendant in small claims court for the $2,000.00 Due Diligence Fee and the $2,500.00 Earnest Money Deposit included in the Agreement. On 30 October 2017, the presiding magistrate found that Defendant had breached the Agreement and entered judgment in favor of Plaintiff for $2,000.00.\n Defendant appealed to Wake County District Court for de novo review. The matter was referred to arbitration, resulting in an arbitrator’s decision affirming the small claims judgment. Defendant then sought trial de novo in a district court trial. Following discovery, the parties filed cross motions for summary judgment. On 26 February 2019, the Court granted Plaintiff’s motion for summary judgment. Plaintiff then filed a Motion for Determination of Damages.\n On 20 September 2019, the Court entered an Order awarding Plaintiff damages as follows: $2,000.00 for the Due Diligence Fee; $2,500.00 for the Earnest Money Deposit; $4,500.00 in prejudgment interest; and $13,067.70 in attorney fees, for Plaintiff’s claim of breach of contract but not for Plaintiff’s claim for promissory\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 5 -\n estoppel or for fraud and punitive damages. Defendant filed timely written notice of appeal.\n Plaintiff filed a Motion to Strike Portions of Defendant’s Reply Brief, and a Motion to Dismiss the appeal. This Court denied both motions.\n II. Summary Judgment\n “Our standard of review of an appeal from summary judgment is de novo; such judgment is appropriate only when the record shows that ‘there is no genuine issue as to any material fact and that any party is entitled to a judgment as a matter of law.’” In re Will of Jones, 362 N.C. 569, 573, 669 S.E.2d 572, 576(2008) (quoting Forbis v. Neal, 361 N.C. 519, 524, 649 S.E.2d 382, 385 (2007)).\n Defendant contends that the trial court erred in granting summary judgment in Plaintiff’s favor, because there were genuine issues of material fact that Plaintiff failed to comply with the Residential Property Disclosure Act. We disagree.\n North Carolina requires most sellers of residential property to provide potential purchasers with a disclosure statement containing disclosures regarding the property being sold. N.C. Gen. Stat. § 47E-4 (2020). The Disclosure Statement that Plaintiff provided was a standard form that the North Carolina Association of Realtors provided in compliance with Chapter 47E. Chapter 47E further provides that when the required disclosures are not timely provided, potential purchasers are vested with the ability to cancel any resulting contract, without penalty and under\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 6 -\n limited circumstances. N.C. Gen. Stat. § 47E-5(b) (2020). When the disclosure is timely received, however, the purchaser’s right to cancel the contract is subject to the terms of the contract itself.\n Furthermore, North Carolina law provides that “[o]ne who signs a written contract without reading it, when he can do so understandingly is bound thereby unless the failure to read is justified by some special circumstance.” Davis v. Davis, 256 N.C. 468, 472, 124 S.E.2d 130, 133 (1962).\n Here, Defendant attended the “Open House” on 24 July 2017, where she was given both the Disclosure Statement and Agreement. Defendant had as much time as needed to review both documents. Defendant reviewed both documents, attested that she had received and examined Disclosure Statement by signing each page, including the pages upon which Items Number 27 and 33 appeared. Defendant voluntarily made an offer to purchase the property subject to the terms described therein and attested once again that she had received and reviewed a copy of the Disclosure Statement before signing the Agreement. When Defendant breached the Agreement by refusing to tender the Due Diligence Fee, her purpose for doing so was entirely unrelated to any perceived deficiency with regard to the Disclosure Statement. In fact, Defendant did not argue that the Disclosure Statement was invalid until well after litigation had commenced in this matter.\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 7 -\n Defendant was given the opportunity to read and review both documents and she attested that she did so. She had the opportunity to ask for clarification, but she failed to do so. Defendant had no justification for any failure to read either document, nor the presence of any mistake, fraud, or oppression, nor any evidence that a special circumstance exists in this case which should operate to excuse any such failure. As such, the requirements of Chapter 47E were satisfied, and Defendant did not have the ability to terminate the Agreement. Defendant failed to show that there were any genuine issues of material fact that would have made summary judgment in Plaintiff’s favor an error. Accordingly, the trial court did not err in granting summary judgment in Plaintiff’s favor.\n III. Attorney’s Fees\n “Recovery of attorney’s fees, even when authorized by statute is within the trial court’s discretion and will only be reviewed for an abuse of that discretion. In order to demonstrate an abuse of discretion, the party challenging an award of attorney’s fees must show that the trial court’s ruling was manifestly unsupported by reason, or could not be the product of a reasoned decision.” In re Clark, 202 N.C. App. 151, 168, 688 S.E.2d 484, 494 (2009).\n Defendant contends that the trial court committed reversible error in granting Plaintiff’s award for attorney’s fees and damages. Defendant does not challenge the\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 8 -\n amount of the attorney’s fees award, only the award itself. After careful review, we affirm the trial court.\n A. The Agreement\n Paragraph 1(d) of the Agreement provides that the Due Diligence Fee of $2,000.00 is due “by the Effective Date,” in this case July 24, 2017. Paragraph 1(e) of the Agreement further provides that upon a breach of the Agreement by the buyer, the Earnest Money Deposit of $2,500.00 is to be “paid to Seller as liquidated damages as Seller’s sole and exclusive remedy for such breach, but without limiting . . . Seller’s right to retain the Due Diligence Fee.” Finally, Paragraph 1(e) provides that “If legal proceedings are brought by Buyer or Seller against the other to recover the Earnest Money Deposit, the prevailing party in the proceeding shall be entitled to recover from the non-prevailing party reasonable attorney’s fees and costs incurred in connection with the proceeding.”\n When Defendant refused to deliver the Due Diligence Fee, she breached the Agreement. Plaintiff then sued to recover both the Due Diligence Fee and the Earnest Money Deposit. Since Plaintiff was required to institute a legal proceeding in order to recover the money, which included the Earnest Money Deposit, and prevailed, Plaintiff was entitled to recover attorney’s fees incurred in the proceeding. Id.\n Our dissenting colleague argues that Plaintiff is not entitled to recover attorneys’ fees because she also sought to recover the Due Diligence Fee in the same\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 9 -\n proceeding. Given language in the Agreement expressly providing for recovery of both the Due Diligence Fee and the Earnest Money Deposit as liquidated damages for breach by the buyer, we disagree with that analysis.\n B. Statutory Authority\n Generally, “a party may not recover it’s attorney’s fees unless authorized by statute.” Martin Architectural Prods., Inc. v. Meridian Constr. Co., 155 N.C. App. 176, 181 574 S.E.2d 189, 192 (2002). However, North Carolina law permits parties to “any note, conditional sale contract or other evidence of indebtedness” to recover attorney’s fees resulting from a breach of the same, “not in excess of fifteen percent (15%) of the outstanding balance owing.” N.C. Gen. Stat. § 6-21.2 (2020).\n Our dissenting colleague argues this case falls outside the scope of Section 6-21.2 so that regardless of the terms of the Agreement, the trial court had no authority to award attorney’s fees.\n In Stillwell Enter. v. Interstate Equipm. Co., 300 N.C. 286, 266 S.E.2d 812 (1980), our Supreme Court examined what constituted evidence of indebtedness. “The term ‘evidence of indebtedness’ as used in N.C. Gen. Stat. § 6-21.2 refers to any printed or written instrument, signed or otherwise executed by the obligor(s), which evidences on its face a legally enforceable obligation to pay money.” Id. at 294, 266 S.E.2d at 817.\n Viewed in light of this definition, defendant’s lease agreement with plaintiff is obviously an “evidence of\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 10 -\n indebtedness.” The contract acknowledges a legally enforceable obligation by plaintiff-lessee to remit rental payments to defendant-lessor as they become due, in exchange for the use of the property which is the subject of the lease. The contract . . . is in writing and is executed by the parties obligated under its terms. . . . Under these circumstances, we see no reason why the obligation by plaintiff to pay attorneys’ fees incurred by defendant upon collection of the debts arising from the contract itself should not be enforced to the extent allowed by [N.C. Gen. Stat. § 6-21.2.\n Id. at 294-95, 266 S.E.2d at 818. Thus, in Stillwell, our Supreme Court held that a contract acknowledging a legally enforceable obligation constituted “evidence of indebtedness.”\n In the present case, the Agreement was a printed instrument signed by both parties. The Agreement on its face evidenced a legally enforceable obligation for Defendant to pay the Due Diligence fee and Earnest Money Deposit to Plaintiff. We hold that, as in Stillwell, the Agreement served as “evidence of indebtedness” within the meaning of N.C. Gen. Stat. § 6-21.2.\n The dissent argues that this case falls outside the scope of Section 6-21.2 because the sale of a residence by owner is not a commercial transaction, citing the “homeowner exception” to claims for unfair and deceptive trade practices. We are not persuaded that the exception should be extended to the application of Section 6-21.2, based on the text of the statute and its remedial nature. Section 6-21.2 provides for the recover of attorney’s fees “upon any note, conditional sale contract or other\n REYNOLDS-DOUGLASS V. TERHARK\n Opinion of the Court\n - 11 -\n evidence of indebtedness.” (emphasis added). The word “any” is inconsistent with an exception not otherwise provided in the same statute. See also Stillwell, 300 N.C. at 294, 266 S.E.2d at 817 (“[W]e hold that the term ‘evidence of indebtedness’ as used in [N.C. Gen. Stat. § 6-21.2 has reference to any printed or written instrument . . .which evidences on its face a legally enforceable obligation to pay money. Such a definition . . .accords well with its general purpose to validate a debt collection remedy expressly agreed upon by contracting parties.” (emphasis added)). And, as noted in Stillwell, Section 6-21.2 should be construed liberally by this Court because it is a remedial statute. Id. at 293, 266 S.E.2d at 817. As of the date of Defendant’s breach by refusal to pay the Due Diligence Fee, she owed Plaintiff $2,000.00 plus another $2,500.00 for an Earnest Money Deposit.\n The award of attorney’s fees was authorized by both statute and the Agreement. There is no evidence to suggest that the trial court’s decision was manifestly unsupported by reason, or that the award could not be the product of a reasoned decision. Accordingly, the trial court did not err in granting Plaintiff’s award for attorney’s fees and damages, and thus, we uphold the decision of the trial court.\n NO ERROR.\n Judge INMAN concurs.\n Judge MURPHY concurs in part, dissents in part.\n Report per Rule 30(e).\n No. COA20-112 – Reynolds-Douglass v. Terhark\n MURPHY, Judge, concurring in part and dissenting in part.\n While I concur fully with the Majority’s conclusion in Part II that the trial court did not err in granting Plaintiff summary judgment, I respectfully dissent as to Part III regarding the award of attorney’s fees. If published, the Majority opinion would enact a sweeping change to the recoverability of attorney’s fees in North Carolina. This would be a change to public policy, and whether such a change is good or bad, it is a decision properly left to the General Assembly and not for this panel to make under the guise of statutory interpretation. N.C. CONST. art. I, § 6; id. art. II, § 1.\n Generally, a party’s attorney’s fees are not recoverable. Martin Architectural Prods., Inc. v. Meridian Constr. Co., 155 N.C. App. 176, 181, 574 S.E.2d 189, 192 (2002). However, this common law principle is subject to statutory exceptions. Id. One such exception is N.C.G.S. § 6-21.2, which allows for the recovery of attorney’s fees up to 15% of the disputed amount when there is an obligation to pay attorney’s fees upon any note, conditional sale contract, or other evidence of indebtedness. N.C.G.S. § 6-21.2 (2019). However, N.C.G.S. § 6-21.2 is inapplicable here and thetrial court erred by ordering attorney’s fees to be paid as there was nothing in the Offer to Purchase and Contract (“Agreement”) to authorize the award and no statutory authority to support the amount of the award.\n A.The Agreement Does Not Authorize Attorney’s Fees\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 2\n The dispute between the parties involved only the due diligence fee. However, the Agreement only provides for the recovery of attorney’s fees by the prevailing party when either party initiates a legal proceeding to recover the earnest money deposit. Paragraph 1(d) of the Agreement contains the following provisions:\n $2,000.00 BY DUE DILIGENCE FEE made payable and delivered to Seller by the Effective Date.\n . . .\n $2,500.00 BY (ADDITIONAL) EARNEST MONEY DEPOSIT made payable and delivered to Escrow Agent named in Paragraph 1(f) by cash, official bank check, wire transfer or electronic transfer no later than [14 August 2017], TIME BEING OF THE ESSENCE with regard to said date.\n Further, paragraph 1(e) of the Agreement, in relevant part, allows for the recovery of attorney’s fees in a limited provision:\n “Earnest Money Deposit”: The Initial Earnest Money Deposit, the Additional Earnest Money Deposit and any other earnest monies paid or required to be paid in connection with this transaction, collectively the “Earnest Money Deposit”, shall be deposited and held in escrow by Escrow Agent until Closing, at which time it will be credited to Buyer, or until this Contract is otherwise terminated. . . . In the event of breach of this Contract by Seller, the Earnest Money Deposit shall be refunded to Buyer upon Buyer’s request, but such return shall not affect any other remedies available to Buyer for such breach. In the event of breach of this Contract by Buyer, the Earnest Money Deposit shall be paid to Seller as liquidated damages and as Seller’s sole and exclusive remedy for such breach . . . If legal proceedings are brought by Buyer or Seller against the other to recover the Earnest Money Deposit, the prevailing party in the proceeding shall\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 3\n be entitled to recover from the non-prevailing party reasonable attorney fees and court costs incurred in connection with the proceeding.\n (Emphasis added). These paragraphs specify attorney’s fees will be recovered only when there is a legal proceeding to recover the earnest money deposit. When the trial court awarded Plaintiff “$13,067.70 as reasonable attorney’s fees incurred[,]” it was in response to the lawsuit brought by Plaintiff to recover the due diligence fee. The Agreement did not provide for attorney’s fees in an action such as this and therefore the order must be vacated. As the Agreement itself did not authorize the recovery of attorney’s fees in an action related to the due diligence fee, the trial court erred in awarding any attorney’s fees.\n B. N.C.G.S. § 6-21.2 Does Not Apply\n In quoting N.C.G.S. § 6-21.2, the Majority states “North Carolina law permits parties to ‘any note, conditional sale contract or other evidence of indebtedness’ to recover attorney’s fees resulting from a breach of the same, ‘not in excess of fifteen percent (15%) of the outstanding balance owing.’” Supra at 9. N.C.G.S. § 6-21.2 provides, in relevant part:\n Obligations to pay attorney[’s] fees upon any note, conditional sale contract or other evidence of indebtedness, in addition to the legal rate of interest or finance charges specified therein, shall be valid and enforceable, and collectible as part of such debt, if such note, contract or other evidence of indebtedness be collected by or through an attorney at law after maturity, subject to the following provisions:\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 4\n (1)If such note, conditional sale contract or other evidenceof indebtedness provides for attorney[’s] fees in somespecific percentage of the “outstanding balance” as hereindefined, such provision and obligation shall be valid andenforceable up to but not in excess of fifteen percent (15%)of said “outstanding balance” owing on said note, contractor other evidence of indebtedness.\n (2)If such note, conditional sale contract or other evidenceof indebtedness provides for the payment of reasonableattorney[’s] fees by the debtor, without specifying anyspecific percentage, such provision shall be construed tomean fifteen percent (15%) of the “outstanding balance”owing on said note, contract or other evidence ofindebtedness.\n N.C.G.S. § 6-21.2 (2019) (emphasis added). While the Majority accurately describesthis statute, it is inapplicable here.\n 1.The Agreement Is Not a Note or Conditional Sale Contract\n As a preliminary matter, the Agreement does not fall within the definition of a note or a conditional sale contract. A note is defined as “[a] written promise by one party . . . to pay money to another party . . . . A note is a two-party negotiable instrument. . . .” Note, BLACK’S LAW DICTIONARY (11th ed. 2019). A conditional sale contract is “[a] contract for the sale of goods under which the buyer makes periodic payments and the seller retains title to or a security interest in the goods.” Conditional Sales Contract, Retail Installment Contract, BLACK’S LAW DICTIONARY (11th ed. 2019). The Agreement is neither a note nor a conditional sale contract because it does not involve a negotiable instrument or the seller retaining title to the real property.\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 5\n 2.The Agreement Is Not Evidence of Indebtedness\n Our caselaw has previously defined “evidence of indebtedness” as used in N.C.G.S. § 6-21.2 to mean “a writing which acknowledges a debt or obligation andwhich is executed by the party obligated thereby.” Stillwell Enters., Inc. v. Interstate Equip. Co., 300 N.C. 286, 294, 266 S.E.2d 812, 817 (1980); see also State Wholesale Supply, Inc. v. Allen, 30 N.C. App. 272, 277, 227 S.E.2d 120, 124 (1976). In the case before us, the Majority characterizes the Agreement as “evidence of indebtedness” using the definition from a limited portion of Stillwell and holds N.C.G.S. § 6-21.2 applies to this agreement. Supra at 9-10. I disagree with this characterization of the Agreement and the application of N.C.G.S. § 6-21.2. I do not read Stillwell through the same lens as the Majority.\n In Stillwell, our Supreme Court discussed the proper scope of the term “evidence of indebtedness” and held “[its] application must [] be gleaned from the context of the statute in which it appears and the factual circumstances surrounding the instrument or transaction to which it is sought to be applied.” Stillwell, 300 N.C. at 292, 266 S.E.2d at 816. While the Majority is correct in stating “[t]he term ‘evidence of indebtedness’ as used in [N.C.G.S.] § 6-21.2 refers to any printed or written instrument, signed or otherwise executed by the obligor(s), which evidences on its face a legally enforceable obligation to pay money[,]” the Majority applies this definition out of context. Supra at 9. The full passage reads:\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 6\n [W]e hold that the term “evidence of indebtedness” as usedin [N.C.]G.S. 6-21.2 has reference to any printed or writteninstrument, signed or otherwise executed by the obligor(s),which evidences on its face a legally enforceable obligationto pay money. Such a definition, we believe, does noviolence to any of the statute’s specific provisions andaccords well with its general purpose to validate a debtcollection remedy expressly agreed upon by contractingparties. Viewed in light of this definition, [the] defendant’slease agreement with [the] plaintiff is obviously an“evidence of indebtedness.” The contract acknowledges alegally enforceable obligation by [the] plaintiff-lessee toremit rental payments to [the] defendant-lessor as theybecome due, in exchange for the use of the property whichis the subject of the lease.\n Stillwell, 300 N.C. at 294, 266 S.E.2d at 817-18 (emphasis added). It is clear when reading Stillwell as a whole that N.C.G.S. § 6-21.2 applies to “supplement those principles of law generally applicable to commercial transactions” and is only relevant for commercial transactions. Id. at 293, 266 S.E.2d at 817. Under the Majority’s application of Stillwell, every contract where one party is to pay money would be evidence of indebtedness, an interpretation that is overbroad by its terms.\n In Stillwell, the lease agreement at issue was a contract between two corporations for the lease of specific goods. Stillwell, 300 N.C. at 287, 266 S.E.2d at 813.The Agreement here is a form agreement created and approved by the NorthCarolina Association of Realtors, Inc. and the North Carolina Bar Association. According to the Guidelines for Completing the Offer to Purchase and Contract Form, the form is for use “in a variety of real estate sales transactions, but it was developed primarily for use in the sale of existing single-family residential properties. Do not\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 7\n use this form as a substitute for a lease-option agreement, lease-purchase agreement or installment land contract.” GUIDELINES FOR COMPLETING THE OFFER TO PURCHASE AND CONTRACT, https://www.ncrealtors.org/wp-content/uploads/markup0717-2G.pdf (last visited Oct. 27, 2020). At no point in the terms of the Agreement is there an acknowledgment of a debt. The Agreement is not the same as an acknowledgement of debt, but rather a\n basic agreement whereby [seller] agrees to sell and [buyer] agrees to purchase real property. . . . It is not a financing device, but rather contemplates that the [buyer] will pay the balance due on the property at the closing date, either by securing permanent mortgage financing or by simply using the [buyer’s] own funds.\n James A. Webster, Jr., Webster’s Real Estate Law in North Carolina § 9.05 (Patrick K. Hetrick & James B. McLaughlin, Jr. eds., 6th ed. 2020). The Majority’s characterization of the Agreement as evidence of indebtedness is overbroad by its terms and I further dissent from its use of N.C.G.S. § 6-21.2 in this context.\n 3. The Agreement Does Not Fall Within the Purpose of N.C.G.S. § 6-21.2\n Further, on the basis of the intended purpose and function of the statute, the Agreement does not fall within the acknowledgment of debt nor the evidence of indebtedness definition. The general purpose of N.C.G.S. § 6-21.2 is to “validate a debt collection remedy expressly agreed upon by contracting parties.” Stillwell Enters., 300 N.C. at 294, 266 S.E.2d at 817-18. A contract to purchase residential real estate for personal use does not achieve this purpose. According to the drafter of\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 8\n the boilerplate agreement, when the form was revised in 2010, the purpose of adding the attorney’s fees sentence to paragraph 1(e) was “the threat of additional costs will help dissuade persons who clearly are not entitled to the earnest money deposit from refusing to consent to its release to the other party.” 2010-2011 Update Course, Revised Offer to Purchase and Contract, Attorney Fees, NORTH CAROLINA REALESTATE COMMISSION, https://www.ncrec.gov/Pdfs/bicar/RevisedOfferToPurchaseAndContract.pdf (last visited Oct. 27, 2020). Incentivizing parties to not withhold a deposit they are not legally entitled to is not of the same caliber as a debt collection remedy. Therefore, the use of N.C.G.S. § 6-21.2 in the present case is inappropriate.\n Even if the Agreement fell under the definition of evidence of indebtedness, N.C.G.S. § 6-21.2 still would not apply. The use of the word “upon” in the statuteindicates there must be an explicit obligation to pay attorney’s fees in the contract itself. In the Agreement, the obligation for attorney’s fees only relates to the specific term “earnest money deposit.” Therefore, even under N.C.G.S. § 6-21.2, the Agreement does not allow for recovery of attorney’s fees in any context other than the recovery of the earnest money deposit.\n C.Amount of Attorney’s Fees\n Even assuming, arguendo, N.C.G.S. § 6-21.2 applies to the due diligence portion of the Agreement, attorney’s fees would be capped at 15% of the due diligence\n REYNOLDS-DOUGLASS V. TERHARK\n Murphy, J., concurring in part and dissenting in part\n 9\n fee. Here, the due diligence fee was $2,000.00, 15% of which totals only $300.00. Therefore, Plaintiff’s attorney’s fees award would be limited to $300.00. The trial court’s award was unlawfully in excess of what would be statutorily authorized.\n CONCLUSION\n The trial court erred in awarding Plaintiff attorney’s fees. The attorney’s fees award should be reversed and vacated because there is no language in the Agreement to authorize the award, as well as no statutory authority to justify an award. Further, even assuming, arguendo, the award was authorized, attorney’s fees would be capped at $300.00. I respectfully dissent in part."""

In [55]:
error_pdfs = [err1, err2, err3, err4, err5, err6, err7, err8, err9, err10, err11]
error_df = pd.DataFrame(columns=["Opinion"])

In [56]:
for n, text in enumerate(error_pdfs):
    error_df.loc[n] = text
error_df.head()

,Opinion
0,STATE OF NORTH CAROLINA v. MARIO DEANDRE TAYLO...
1,Pinpoint added 191 #3 p. 411\n CHARLES W. STON...
2,"NATIONWIDE MUTUAL FIRE INSURANCE COMPANY, Plai..."
3,"1 Bruce-Terminix Co. v. Zurich Ins. Co., 130 N..."
4,"ST. REGIS OF ONSLOW COUNTY, NORTH CAROLINA OWN..."


In [57]:
df = pd.concat([df, error_df], ignore_index=True)

In [58]:
df.describe(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28783 entries, 0 to 28782
Data columns (total 1 columns):
Opinion    28783 non-null object
dtypes: object(1)
memory usage: 225.0+ KB


(                                                  Opinion
 count                                               28783
 unique                                              28674
 top      \n \n \n\nIN THE COURT OF APPEALS OF NORTH CA...
 freq                                                    4, None)

In [59]:
df.head()

,Opinion
0,\n\nAn unpublished opinion of the North ...
1,An unpublished opinion of the North Caro...
2,An unpublished opinion of the North Caro...
3,NO. COA11-246 \n\nNORTH CAROLINA COURT OF APPE...
4,An unpublished opinion of the North Caro...


In [60]:
# Save the combined DF to a data file
with open('combined_df.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(df, filehandle)

## 3. With the Master DataFrame Created, Select Only the Relevant Opinions
This project is only looking at opinions containing the phrase "summary judgment." This will be further reduced later (some cases might reference "summary judgment" outside of the relevant context), but this initial reduction will remove those opinions lacking any relevance at all. 

In [ ]:
# Open the dataframe
infile = open('combined_df.data', 'rb')
df = pickle.load(infile)
infile2.close()

In [ ]:
# Create new DF that only has "summary judgment" or "Summary Judgment"